In [1]:
import sys
sys.path.insert(0, r"C:\Users\Rui Campos\Dropbox\MontyCarlo")

# numpy.polynomial.polynomial.Polynomial.fit

In [2]:
import MontyCarlo.materials.database as db

____INIT_____
>>>> SETTINGS.Py
_________________________________________________________________
INSTALL PATH:  C:\Users\Rui Campos\Dropbox\MontyCarlo\MontyCarlo
----
SURFACE_THICKNESS: 1e-07
---------------
DEBUG: False
---------------
PHOTON CUT OFF: 5110.0 MeV
----
ELECTRON CUT OFF: 10000.0 MeV
---------------------------------------------
Wcc: 10 eV
---------------


/materials.__init__
from .materials import Material
pyRelax
48
>>>> importing database.py
material init electron
>>>> materials.photon folder
>>>> IMPORTING material.photon.CrossSection.pyx
>>>> IMPORTING material.photon.pyx
>>>> IMPORTING main.pyx
>>>> IMPORTING materials.pyx
END
END


<frozen importlib._bootstrap>:219: RuntimeWarning: MontyCarlo.materials.electron.GOSfinal.gosMolecule size changed, may indicate binary incompatibility. Expected 448 from C header, got 456 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: MontyCarlo.materials.electron.GOSfinal.gosShell size changed, may indicate binary incompatibility. Expected 928 from C header, got 960 from PyObject


In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Dec  1 14:20:30 2020

@author: Rui Campos
"""

from numpy import *
from numba import *

#dealing with paths >.<
#from ....settings import __montecarlo__
#__coherent__ = __montecarlo__/'materials'/'photon'/'coherent'
#directory = str(__coherent__)

#path = "/pickles/"

def getFF(Z):
    import dill as pickle
    path = "pickles/" + str(Z)
    with open(path, 'rb') as file:
        param = pickle.load(file)

    a1, a2, a3, a4, a5 = param


    #depending on Z, the fit is calculated differently
    if Z<11:
        @njit
        def _eval(x2):
            x = sqrt(x2)
            A = 1 + a1*x**2 + a2 * x**3 + a3 * x**4
            B = 1 + a4*x**2 + a5 * x**4
            f = Z*A/B**2
            return f**2

    else:

        @njit
        def _eval(x2):
            x = sqrt(x2)
            A = 1 + a1*x**2 + a2 * x**3 + a3 * x**4
            B = 1 + a4*x**2 + a5 * x**4
            f = Z*A/B**2

            if f > 2: return f**2

            alpha = 1/137.03599908421  #fine structure constant
            a = (Z - 5/6)*alpha

            Q = x/20.6074/2/a #changing variables to suit formula in paper

            gamma = (1 - a**2)**.5

            #calculating Fk
            num = sin(2*gamma*arctan(Q))
            den = gamma*Q*(1 + Q**2)**gamma
            Fk  = num/den
            return max(f, Fk)**2
        
    return _eval


FFlist = [getFF(Z) for Z in range(1, 70)]

In [4]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from matplotlib.pyplot import *

from MontyCarlo.tools.data import getAxis

from numpy import *

from numba import *


def graph(Z, x1, x2, y1, y2):
    FFdata = db.EPDL[Z-1][(7, 93, 0, 0, 0, 941)]
    
    FFx, FFy = FFdata.X, FFdata.Y
    FFx = FFx*1e-8
    FFy = FFy/Z
    
    fig = figure(figsize=(14, 14))
    
    #x1, x2 = map(lambda x: 10**x, (x1, x2))
    #y1, y2 = map(lambda x: 10**x, (y1, y2))
    
    x = logspace(x1, x2, num=150)#, (x2 - x1)/((x1+x2)*100))
    
    
    
    f = FFlist[Z-1]
    
    y = [Z**-1 *  f(X**2)**.5 for X in x]
    #print(y)
    plot(x, y)
    
    
    ax = gca()
    
    scatter(FFx*1e14, FFy, s = 5, color="red")
    
    ax.set_yscale("log"); ax.set_xscale("log")
    ax.set_title("Form Factor of Element " + str(Z))
    ax.set_xlabel("x (cm^-1)")
    ax.grid(which='both')
    ax.set_ylim(10**y1, 10**y2)
    ax.set_xlim(10**x1, 10**x2)
    
    Y = [Z**-1 *  f(X**2)**.5 for X in FFx*1e14]
    Y = array(Y)
    print(sum((FFy - Y)**2))
    #display(f(1))
    


In [5]:
from ipywidgets import IntSlider
a = IntSlider(min=1, max=100, step=1, continuous_update = False)

x2 = IntSlider(min=-10, max=4, step=1, continuous_update = False)
x1 = IntSlider(min=-10, max=4, step=1, continuous_update = False)

y2 = IntSlider(min=-10, max=1, step=1, continuous_update = False)
y1 = IntSlider(min=-10, max=1, step=1, continuous_update = False)

w = interact(graph, Z = a, 
             y1=y1, y2=y2, 
             x1=x1, x2=x2) 



interactive(children=(IntSlider(value=1, continuous_update=False, description='Z', min=1), IntSlider(value=0, …

<Figure size 1008x1008 with 0 Axes>

In [6]:
def graph_error(Z):
    f = getFF(Z)
    
    
    FFdata = db.EPDL[Z-1][(7, 93, 0, 0, 0, 941)]
    
    FFx, FFy = FFdata.X, FFdata.Y
    FFx = FFx*1e-8
    FFy = FFy/Z
    
    Yfit = [Z**-1 * f(x**2)**.5 for x in FFx*1e14]
    Yfit = array(Yfit)
    fig = figure(figsize=(10, 10))
    err = Yfit - FFy
  
    
    print(f"""
    max err = {max(err)}
    min err = {min(err)}
    """)
    
    hist(err, bins = 6000)
    ylim(0, 8)
    title("Error distribution for fit of element " + str(Z))

In [7]:
w = interact(graph_error, Z = (1, 70))

interactive(children=(IntSlider(value=35, description='Z', max=70, min=1), Output()), _dom_classes=('widget-in…

In [8]:
help(logspace)

Help on function logspace in module numpy:

logspace(start, stop, num=50, endpoint=True, base=10.0, dtype=None, axis=0)
    Return numbers spaced evenly on a log scale.
    
    In linear space, the sequence starts at ``base ** start``
    (`base` to the power of `start`) and ends with ``base ** stop``
    (see `endpoint` below).
    
    .. versionchanged:: 1.16.0
        Non-scalar `start` and `stop` are now supported.
    
    Parameters
    ----------
    start : array_like
        ``base ** start`` is the starting value of the sequence.
    stop : array_like
        ``base ** stop`` is the final value of the sequence, unless `endpoint`
        is False.  In that case, ``num + 1`` values are spaced over the
        interval in log-space, of which all but the last (a sequence of
        length `num`) are returned.
    num : integer, optional
        Number of samples to generate.  Default is 50.
    endpoint : boolean, optional
        If true, `stop` is the last sample. Otherwise, 

<html><head>


<!-- Load require.js. Delete this if your page already loads require.js -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" crossorigin="anonymous"></script>
<script src="https://unpkg.com/@jupyter-widgets/html-manager@*/dist/embed-amd.js" crossorigin="anonymous"></script>
<script type="application/vnd.jupyter.widget-state+json">
{
    "version_major": 2,
    "version_minor": 0,
    "state": {
        "da201b6c63ef42958ce0b9699ebf5e29": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "bcfbc21b47b44526813b1ca8d8fb8a21": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_2a113460df4f41c09e4c66550095d001",
                    "IPY_MODEL_13b2c9ab21b64454b0a46d00309237d0",
                    "IPY_MODEL_6cbfd16b7b3540d6a6d94095e3fc3ea7",
                    "IPY_MODEL_db542944757d48eb8d83eac352011cdd",
                    "IPY_MODEL_31d1bb5dde5549a2acf2a68d6a8aa2e0",
                    "IPY_MODEL_ba240848279847e0b72bfd725733a471"
                ],
                "layout": "IPY_MODEL_da201b6c63ef42958ce0b9699ebf5e29"
            }
        },
        "9428531de6a247b28622e0d504989964": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "5c8cb60e603d4104a1cb928302cdbce4": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "2a113460df4f41c09e4c66550095d001": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_9428531de6a247b28622e0d504989964",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_5c8cb60e603d4104a1cb928302cdbce4",
                "value": 10
            }
        },
        "54833e8aaf6c40ed99cd6af71de3f438": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "660047fe0ce34bd596e125f459df22ca": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "13b2c9ab21b64454b0a46d00309237d0": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x1",
                "layout": "IPY_MODEL_54833e8aaf6c40ed99cd6af71de3f438",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_660047fe0ce34bd596e125f459df22ca",
                "value": -1
            }
        },
        "f2112f596b304ed88becefb6e947a447": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "d8d9c814ed434783b99115b98f34e9be": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "6cbfd16b7b3540d6a6d94095e3fc3ea7": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x2",
                "layout": "IPY_MODEL_f2112f596b304ed88becefb6e947a447",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_d8d9c814ed434783b99115b98f34e9be",
                "value": 2
            }
        },
        "19c1039e6d3c48509931212e36600f8b": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "af247caf3d654fd2ad3f6c4647260bd1": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "db542944757d48eb8d83eac352011cdd": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y1",
                "layout": "IPY_MODEL_19c1039e6d3c48509931212e36600f8b",
                "max": 1,
                "min": -15,
                "style": "IPY_MODEL_af247caf3d654fd2ad3f6c4647260bd1",
                "value": -7
            }
        },
        "b6dfda7e331842608e02f6b1f87ec7ed": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "1be4fa4b08c24831a4169c86b11a8d32": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "31d1bb5dde5549a2acf2a68d6a8aa2e0": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y2",
                "layout": "IPY_MODEL_b6dfda7e331842608e02f6b1f87ec7ed",
                "max": 1,
                "min": -15,
                "style": "IPY_MODEL_1be4fa4b08c24831a4169c86b11a8d32",
                "value": 1
            }
        },
        "d582b8b2b3f34cb0bc987a59f0c50293": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "ba240848279847e0b72bfd725733a471": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_d582b8b2b3f34cb0bc987a59f0c50293",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "...\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "9a3e13a7192e45679e605e2a5bf08956": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "d42b8961a03e4350a4952d8edc4ce871": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_7485a8a3f1a34992be867344fc0e338b",
                    "IPY_MODEL_8a96c7c7d83d4198ba48a028ba02f19c",
                    "IPY_MODEL_e1cb48e51db1438282a368f0b100452c",
                    "IPY_MODEL_fdf31007f5a947429bad9c90a18730cb",
                    "IPY_MODEL_d2f0a694cdaa42a6b4d6a2f2a37a6e73",
                    "IPY_MODEL_144423b5e48d48a88014cdd9aca7c12d"
                ],
                "layout": "IPY_MODEL_9a3e13a7192e45679e605e2a5bf08956"
            }
        },
        "9147430305d24664b8a2bbc2769eba69": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "6f35348a32b0455eaf5a26078fc4f08d": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "7485a8a3f1a34992be867344fc0e338b": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_9147430305d24664b8a2bbc2769eba69",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_6f35348a32b0455eaf5a26078fc4f08d",
                "value": 10
            }
        },
        "e84eae3f45e24050a2c1123de907f160": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "07f68a46dad84a859bc4ad3f4e1f24ed": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "8a96c7c7d83d4198ba48a028ba02f19c": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x1",
                "layout": "IPY_MODEL_e84eae3f45e24050a2c1123de907f160",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_07f68a46dad84a859bc4ad3f4e1f24ed",
                "value": -5
            }
        },
        "420742318e174a29a682669df27ddeac": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "03e02078a283451381e4304673de727f": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "e1cb48e51db1438282a368f0b100452c": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x2",
                "layout": "IPY_MODEL_420742318e174a29a682669df27ddeac",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_03e02078a283451381e4304673de727f",
                "value": 10
            }
        },
        "47983747e3d14494b3b9ff09b85e7778": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "4f3c62b055d74406a8631ed3a8a5c429": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "fdf31007f5a947429bad9c90a18730cb": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y1",
                "layout": "IPY_MODEL_47983747e3d14494b3b9ff09b85e7778",
                "max": 1,
                "min": -15,
                "style": "IPY_MODEL_4f3c62b055d74406a8631ed3a8a5c429",
                "value": -15
            }
        },
        "86d4a95226ab4bf0b075233a184750ff": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "c9373e56d80a4142a225911b90bd5d8a": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "d2f0a694cdaa42a6b4d6a2f2a37a6e73": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y2",
                "layout": "IPY_MODEL_86d4a95226ab4bf0b075233a184750ff",
                "max": 1,
                "min": -15,
                "style": "IPY_MODEL_c9373e56d80a4142a225911b90bd5d8a",
                "value": 1
            }
        },
        "efa8484094af4822acc65464ebccde51": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "144423b5e48d48a88014cdd9aca7c12d": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_efa8484094af4822acc65464ebccde51",
                "outputs": [
                    {
                        "output_type": "stream",
                        "text": "<ipython-input-9-67c16fe9bae3>:40: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.\nInvalid limit will be ignored.\n  ax.set_ylim(y1, y2)\n<ipython-input-9-67c16fe9bae3>:41: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.\nInvalid limit will be ignored.\n  ax.set_xlim(x1, x2)\n",
                        "name": "stderr"
                    },
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAmkAAAJgCAYAAADRfvQaAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAA0qUlEQVR4nO3deZhcVZ3/8fe3E0gCzZpAgCQQlhCBoECzKLgkMy5RERwHFdTMbxTlwfllRscRhNHfiCuI4+gIuMwg4EYig6gxRhE0kVFRQkAkGFBAJU2EJBCWQAJZvr8/uqJF0Ut19+306fT79Tz1JPd7zrl17uHe9IdbXVWRmUiSJKksLYM9AUmSJD2bIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SWoQEX8TEcsjYm1EHNnLsedFxNcGam6Shg9DmrQNiYg/RMS6WrjY8thnKM8jIqZHRHvVc+zBvwOzM7M1M2/tZE4ZEU80HN/ZW3mOvVKb80HdtO8dEfMiYkWt7+SG9lERcVlEPBYRD0TEewZ80tIwN3KwJyCpcq/JzOv7OjgiRmbmxsGeR1X6eDz7AXf00Od5mXl3H6dVos3AD4DzgZ930n4eMIWOtdkLWBgRv8nMH2y1GUrDjHfSpGGgdhfkM7W7JCtqfx9Va5seEe0R8b6IeAC4vPaS3f9ExNci4vGIuD0iDo6IcyNiZe2lwJf3cg67RcT8iFgVEWtqf59Y1757RFxem9+aiPh2ROwIfB/Yp/6OXG+Pp5O5tETEByLij7Xj+UpE7FLb71pgBHBbRNzT91X/83M9PyJ+HhGPRMRtETG9rm1RRHy01r42Ir4bEWMj4uu1O1aL6+9oRcRzIuK6iHg4Iu6KiDfUtV0REZdExPdq/81+GREH1tpuqHW7rfY8b2ycZ2Y+mJmfAxZ3cSh/B3wkM9dk5jLgv4G/79/qSOqOIU0aHt4PPB84AngecCzwgbr2vYDd6bhLckat9hrgq8BuwK3AtXT8mzEB+DDwxV7OoYWOwLQfsC+wDri4rv2rwA7AYcCewKcz8wnglcCK2kuPrZm5oo/HU+/va48ZwAFAK3BxZj6Vma21Ps/LzAN7eYzPEBETgO8BH63N573ANyNij7pupwKz6FjXA4Eb6Vin3YFlwAdr+9oRuA64ko71OQ34XEQcVrev04AP0fHf7G7gYwCZ+eK6Y2rNzG/08jh2A/YBbqsr30bHfytJA8SQJm17vl27a/NIRHy7Vnsz8OHMXJmZq+j4QT6rbsxm4IO1kLKuVvvfzLy29lLh/wB7ABdk5gZgLjA5InZtdh6Z+VBmfjMzn8zMx+kIEC+Bjt+HoiOMnVm7U7MhM3/Szb77cjyN4/8jM+/NzLXAucCpEdGbXwG5pe74HomIV3TS5y3AgsxckJmbM/M64GbgVXV9Ls/MezLzUTruGt6TmdfXrfuWNy6cCPwhMy/PzI2ZeQvwTeCUun1dk5k31cZ+nY4QW4UtwfXRutqjwE4V7V9SJ/ydNGnb89pOfhdsH+CPddt/rNW2WJWZ6xvGPFj393XA6szcVLcNHT+8H2lmHhGxA/BpYCYdd3oAdoqIEcAk4OHMXNPlUT1TX46np/EjgfHA/U3O4agmfidtP+D1EfGautp2wMK67cZ1btzeEpD2A46LiEfq2kfScQdyiwfq/v5k3dj+Wlv7c2dgfd3fH69o/5I64Z00aXhYQccP+S32rdW2yK0wh38BpgLHZebOwJaX4AJYDuzexZ25zubW3+PpbPxGnhmQqrAc+Gpm7lr32DEzL+jjvn7SsK/WzHxnxXN+llp4/hMdLy1v8Tx6fnOFpH4wpEnDwxzgAxGxR0SMA/4N2Nqf5bUTHXeGHomI3an9rhVAZv6Jjpf6Pld7g8F2EbElxD0IjI2IXer21d/jmQP8c0TsHxGtwMeBb1T0rtZ6XwNeExGviIgRETG69saGiT2OfLb5wMERMau2PttFxDERcUiT4x+k4/fvuhQRo4FRtc1Rte0tvkLHmu8WEc8B3gFc0btDkNQbhjRpePgoHb8L9WvgduCWWm1r+gwwBlgN/IKOj3uoNwvYANwJrATeDZCZd9IRqu6t/e7XPvT/eC6j42XCG4Df0/ES3j/28ni2vFNyy+MzjR0yczlwMvCvwCo67oadRR/+7a39Ht/L6XijwQo6Xtr8BH8JVT05D/hybQ3f0EWfdfzlpc07+cvL2tARqu+h46XhnwCf9OM3pIEVmVvjVQ5JkiT1hnfSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAINqQ+zHTduXE6ePPkZtSeeeIIdd9yx1/tqdlxP/bpr76qtsd5Zvy21h594mvsf6ewD0yVJW0Tt0dIStETQEhAbN9IyciQtmzbQst12xIYNtIwe1dG+bh0xZjQt69fRMmYMLeuepGWnnYiRI2nZvImWxx6lZdddO7ZbOvbXEjHgP3Oa6dvsz5bu6j39HOrrcfbGtv7zG2DJkiWrM3MP+mhIhbTJkydz8803P6O2aNEipk+f3ut9NTuup37dtXfV1ljvrN+W2qPrNvD9H/8vxx57bI9z7Y+bbrqpz8/R7Nie+nXX3llbM7WetgdCX5+jN+P6upa9qXe7dmvWcNONN3LsC14Au+3GgOjHczS1lj3tv4v2Le+Hv+nnP+fYZcvgda/7S3vjmE62ueYabjrkEI49/vjOn2PNGvKaa+BvXtfNcSeseQSuuab2/Ls+u8eWia5Zw02/+AXHPv/5nR5HPrwG5swhgTztNNh11z+PzUfWcPMvb6Lt2GNh113/vN+sjf5Lv0dYsngxRx1zDLnLLs9sy2TTmke4ZcktPPfII9m8885s2pxkwqbNyeZMfn37Ug497DA2fWcem6+8ks1HHsVvDj2UKZuTnDmTTZthUyZ33fVbDvzjfWz+7jw2H3oYvztoCpMz2XTCi1j/05/xO0YwdnOw/tBprLtjGevu/C3r9tyb9Y8/wbrdxrFu/dOs320c6zJ4MoOMjheTNtcePWllE2NHw8HjdmLiqnYmHPc8Juw8igk3XMeEN7+OPfbdm4jodGxvflb19edOb+o9/Rzq68/W3tjWf34DRMQf6YchFdKGo13GbMdeO7ZwwB5VfbtL5+7rx3M0O7anft21d9bWTK2n7YHQ1+fozbi+rmVv6t2u3R6t3LfiHg44eFJT8+2TfjxHU2vZ0/57aF++y/YceM67njVm+Yp7OHDLmE62OeddLF+0iAP3aH12e12fHu25E5zbRL/xO3H/A/cy5Tn7dtnOh9/Xeds+O7Nq5e+Zdth+nbdvMWlXHnnoDxx5+OTO2yfvzrpH7+OEts6/r377VXcy/fC9Ye+/hVGPwUknsduvfsX0v/5rGDfuz/0Wrf890199Mox5GE46iUX1fZ63G4t+9KO/bK+eCJe3w0kvgXnz4KRjO/5868kA5GWX8/SJr2H9d7/HupfNZN0Pr2fdSa9l3Y47s371w6z75KdYd9tS1r3pLax/6ctZe92PefB/b+K2F0xnxe/WsDhG8/gNy2sz2x8+fyvbj/gVE3I9EyftwYRRdIS3V/01E5YuZvWBE9m47E5Gzv8uvPWtzzguqSuGNElSGcaNg7PO6vj7n/7UeZDpqs+4cTB+/DO3t/Rr/BOIs89iFDDq0OewC8BRh/3lOfbbDS7+MFx+Obz1dR37mnISbL6PRVOT6Uc9H+bN47E3zeL+x57m/mu+x/3HvIj7f76E9l/dyf3RxrL1yeq9joNb1gKHwDJ4329+x16PjWXCedcw8XnPYcKKe5lw1KFMuOmnTIyn2PsfTmf0XntWtJjaFhjSJElqVB/ytmyfdx4sWgRTp8JZZ7EzsPMEOOT9ta9PPWovuHwlvHU6PPQQ6//lLO5/1/u4/4Zf8uPx+9K6+z7c/5Pfcf9jG/jlshX8aeTubP7ZKjq+0hb4zGImto5k/Ih1LF1wMYeN2sC0t5/KHvvtvTWPXAUxpEmSVIX6YDduHKPnz+NA4MCXncDmRYuYPv14ePnBHXfoTjqSDd+ZxwN/9Urun3ct99/1R+5vX8VvDz+OxWP3499b9u/4krTP38L4HUYwbf1DHBZPcNhL2lj74Epy1Spijz7/PrqGCEOaJElbS12Q2+7ss5gETDp6GqxeXQtvr2TRLbfQdutN/OZ3f2Jp+xruOOol3LFxFAvHTmLzjx8AduSjt/2EafvuzqEPL2facYcx7bafsbnt0EE9NFXPkCZJ0mBr+F27nU47jeNWr+a4yy+Hk14Nc+awLh/izpedzLd+eQdPX/8Llm44ksu3242nf/IgcBCjf7ye5y79EdNW/4HDWp5k2ptO5sCDJw7qYal/DGmSJJWoPriddx5jgCOBR9evYvrmSXDS8Tz9nXn87oWv4I7PfZlrDzqSNfeu5Mrtd2X9dnvBl29nFLcxYcxmrv/+f3HYdk8z7dTXcPBzDG5DhSFNkqShZOTIP4e37c8+i8OAww4+lz1/9COmH3EMm66cw+9zNEtzB5b+7Nf89ISX852nduFr0QpfW0pwO7ttlxx064+Y+MgDTDpkf9Y+/BCjd7mXSfvvzV47jx7c49OfDWpIi4gDgPcDu2TmKYM5F0mShqwtH0EydSojPnQeBwEHrV7Na0c8zKIpG3jJbbexfPvdWJo7cNfCm7j5r1/Nhnsf4BebRvKt9Q+TMZov3bcMWMZINrPb9nDwbT9m0sMrmHTM4UycOI6Ju+3ApN3HsEfrqC4/tFfV6nNIi4jLgBOBlZk5ra4+E/hPYARwaWZe0NU+MvNe4PSIuLqv85AkSZ2o+9iQePOb2RfYd/VqXjViDYumBtOPejHMm8fTr34B31p0I/t88rMsP+GltC9fyZLpM3ninpVcn6NY/dN2oP3Pux01IpjIeibttRuTVrcz6flHMHHn7Zl0ww+Z9JbXs8vE8Ya4ivTnTtoVwMXAV7YUImIEcAnwMjr+iy6OiHl0BLbzG8a/LTNX9uP5JUlSb3TyeW/bA+NXtPOif3gTnHQSzJnDoqkw/agXwrx5rFt7L+2XXMryl57I8t8tZ/kRz6f94SdYvvZgbokxPPaT+2o7PwAuWcJO27UwaePjPLflSY56wWEcvfTn7P/aVxDf9dsWeqvPIS0zb4iIyQ3lY4G7a3fIiIi5wMmZeT4dd90kSVJp6n7PrTHEjVm9mimtY5hy0km1r9ja8ucLYN48Hj1tFu2PPcXyj3yS9nvuZ/lzj+X3j21gwT5TmfvTlcBB7HbpUtp+/zhH3fYp2vZp5Xk8Dkc+r+OjRwxtXYr887fw9mFwR0ibv+Xlzog4BZiZmW+vbc8CjsvM2V2MHwt8jI47b5fWwlxjnzOAMwDGjx/fNnfu3Ge0r127ltbW3n9PYrPjeurXXXtXbY31zvrV1/p6jL3Rn+cYrLVsptbMWldtoM/JZvo2e+51Vx8Oa+n1Xd1Yr+/qxg3Z63vjRnjoIdh1V3j4YTYn/Gn0btz9pye4e8MYfvfQBh7Y0HFvaATJxB02M3W7dRy0awtTJuzEbusehbFjOwJjN4bS9T1jxowlmXl0j5PtSmb2+QFMBpbWbb+ejrC1ZXsWcFF/nqP+0dbWlo0WLlz4rFozmh3XU7/u2rtqa6x31q++1tdj7I3+PMdgrWUztWbWumoDfU4207fZc6+7+nBYS6/v6sZ6fVc3bpu9vletyof+7SN53VkX5AVnfy5f/sFv5sHv+Wbu9775ud/75ufxZ34p//HcL+cV778kbz/rQ7nhrLMyP/jBzFWr+jS3Eq5v4ObsR+6p+t2d7cCkuu2JwIqKn0OSJA0148ax+4c+wEuBlwLHXX89x//vz/hN7sCSI17MLb96lF8+PYZ5m8dCy37s8PQ6jrjzt7R9+lu0tTzOkU+tYhc2wZFHDpuXSasOaYuBKRGxP3A/cCrwpoqfQ5IkDXUjR7L9hz7IEcARwOmvO45ctYr7L/5vlqzbjlu224Ul+x/E5zbtwKZN+xAxhSmrl7P3XZtZ/V+zaTtmKpPPmr1Nf4dpfz6CYw4wHRgXEe3ABzPzSxExG7iWjnd0XpaZd1QyU0mStE2LPfZg4of+lYnAybXaEyse5LbPf40lT41iycjkpqd24ifHzQLg4I8u4O07Pc5Js9/A6L32HLR5D5T+vLvztC7qC4AFfZ6RJElSzY77jOf4j/wLxwOsXs2Pf3Atk772TX6xy75cucOBnD3mAC68cBGzHryVt0xoYezZ/7zNvBTq10JJkqShYdw4WiZOYMoPrmHK6tW85aKLufH2W7i0ZV8+fdALuWTj07zu0/M5fcMf4MC9YcECOPvsIRvaDGmSJGnoGTeO+NB5HL96NcdfeCF333oplx3xKr65aS/mtuzB4e1rGfE/1/HCW28lTjgBZs8ecmGtZbAnIEmS1GfjxsGFF3LQtd/i4594Bzf+w9G8d/PvuW/zGGa98aPM3Pe1XHXNz3jqhBd23FVbvXqwZ9w076RJkqRtxu777s3sC2cz9ccLebRlLy699knOftW7uXDtGv7uZ9/jzW98C2Nf9Hx44QsHe6o98k6aJEna5mzXEpwy/RC+//FT+Por9uHwx1fwHy96C8cfcQbn/uIhVtx+D9x112BPs1veSZMkSdusiOCEGUdywuGTuPuzl/KlBzdzzWEzmPPgKH744W/y9glwwlnvKPLz1ryTJkmStn3jxnHQh8/h/E+ewc9HLOFvWh9i6e778paW5/HKf/s2/3P2p3jqgZWDPctnMKRJkqThY9w4xl74MU4+Zjw/vetrXLj46/DUU5zV8hxOuHAhF73pHB7/44oi3mDgy52SJGn4GTWK0T/8AW+46y5eP3s2P1u9D5eOOoBPHfgitlu2mZ9/79Oc/i+nctBxhw/aFL2TJkmShq+pU4nrruOFl32KK/Z/gut+8DGOH/0Y39z3aF76rft4+5mf5Z6bbh+UqRnSJEmSap+3NuX6ebx1Cvz8Bx/m3T/9Or8csxevuPr3fOStH+GJteu36pQMaZIkSVuMGwcTJjBu0XW8+/iJLLzufF7/6+u4bM8jOOfnT3Pluz/Bpg+et1V+Z82QJkmS1Kh2Z23cous4/7lj+O63P8jeI57iX0dP48T7xvKLV7+p43PWBvBbDAxpkiRJXamFtWnz5nDupEe4eP2tPNa6K6fOeBeXrNiZ5V/8Mvzt38J551Ue1gxpkiRJPZk6lTh4Cid+5gNcf+bR/PPyn3Lb06289O2f51O5L09+/AI47jh46qnKntKQJkmS1AtjDjuEd339fM4/YTQzR63louNP5a/e8UW+PXpfctmdlb0EakiTJEnqg7Gt2/GfH/87rn7VBPaIjbz7Ne/lY6v34bavfhtOOYWR/fw8WkOaJElSPxz94iP4zmffxoUv3JOVm7fn5P/zad674xEcBAf0Z7+GNEmSpH5qaQnecOIxfOKEUZx523zmHTKdHWGnfu2zqslJkiQNd2N2HM05n3k3P7z+E/3elyFNkiSpSlOnMvn677IKHujPbgxpkiRJVRs3jvvg/v7swpAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFGjmYTx4RLcBHgJ2BmzPzy4M5H0mSpFL0+U5aRFwWESsjYmlDfWZE3BURd0fEOT3s5mRgArABaO/rXCRJkrY1/bmTdgVwMfCVLYWIGAFcAryMjtC1OCLmASOA8xvGvw2YCtyYmV+MiKuBH/VjPpIkSduMPoe0zLwhIiY3lI8F7s7MewEiYi5wcmaeD5zYuI+IaAeerm1u6utcJEmStjWRmX0f3BHS5mfmtNr2KcDMzHx7bXsWcFxmzu5i/A7ARcCTwJ2ZeUknfc4AzgAYP35829y5c5/RvnbtWlpbW3s992bH9dSvu/au2hrrnfWrr/X1GHujP88xWGvZTK2Zta7aQJ+TzfRt9tzrrj4c1tLru7qxXt/VjfP6rmZcCdf3jBkzlmTm0T1OtiuZ2ecHMBlYWrf9euDSuu1ZwEX9eY76R1tbWzZauHDhs2rNaHZcT/26a++qrbHeWb/6Wl+PsTf68xyDtZbN1JpZ66oN9DnZTN9mz73u6sNhLb2+qxvr9V3dOK/vasaVcH3T8abIPueeqj+Cox2YVLc9EVhR8XNIkiRt86oOaYuBKRGxf0RsD5wKzKv4OSRJkrZ5/fkIjjnAjcDUiGiPiNMzcyMwG7gWWAZclZl3VDNVSZKk4aM/7+48rYv6AmBBn2ckSZIkvxZKkiSpRIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAWy2kRcQBEfGliLi6u5okSZKaDGkRcVlErIyIpQ31mRFxV0TcHRHndLePzLw3M0/vqSZJkiQY2WS/K4CLga9sKUTECOAS4GVAO7A4IuYBI4DzG8a/LTNX9nu2kiRJw0RkZnMdIyYD8zNzWm37BcB5mfmK2va5AJnZGNAa93N1Zp7SU62u7QzgDIDx48e3zZ079xnta9eupbW1talj6Mu4nvp1195VW2O9s371tb4eY2/05zkGay2bqTWz1lUb6HOymb7Nnnvd1YfDWnp9VzfW67u6cV7f1Ywr4fqeMWPGksw8usfJdiUzm3oAk4GlddunAJfWbc8CLu5m/FjgC8A9wLld1bp7tLW1ZaOFCxc+q9aMZsf11K+79q7aGuud9auv9fUYe6M/zzFYa9lMrZm1rtpAn5PN9G323OuuPhzW0uu7urFe39WN8/quZlwJ1zdwczaZszp7NPtyZ2eis8zXTRh8CDizp5okSZL69+7OdmBS3fZEYEX/piNJkiToX0hbDEyJiP0jYnvgVGBeNdOSJEka3pr9CI45wI3A1Ihoj4jTM3MjMBu4FlgGXJWZdwzcVCVJkoaPpn4nLTNP66K+AFhQ6YwkSZLk10JJkiSVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBVo5NZ6oog4AHg/sEtmnlKrtQAfAXYGbs7ML2+t+UiSJJWsqTtpEXFZRKyMiKUN9ZkRcVdE3B0R53S3j8y8NzNPbyifDEwANgDtvZm4JEnStqzZlzuvAGbWFyJiBHAJ8ErgUOC0iDg0Ig6PiPkNjz272O9U4MbMfA/wzr4dgiRJ0rYnMrO5jhGTgfmZOa22/QLgvMx8RW37XIDMPL+H/Vxd93LnW4CnM/OqiPhGZr6xk/5nAGcAjB8/vm3u3LnPaF+7di2tra1NHUNfxvXUr7v2rtoa6531q6/19Rh7oz/PMVhr2UytmbWu2kCfk830bfbc664+HNbS67u6sV7f1Y3z+q5mXAnX94wZM5Zk5tE9TrYrmdnUA5gMLK3bPgW4tG57FnBxN+PHAl8A7gHOrdV2AL4EXAT8357m0NbWlo0WLlz4rFozmh3XU7/u2rtqa6x31q++1tdj7I3+PMdgrWUztWbWumoDfU4207fZc6+7+nBYS6/v6sZ6fVc3zuu7mnElXN90/L5901mr8dGfNw5EZ5mvmzD4EHBmQ+1JoPH31CRJkoa9/nwERzswqW57IrCif9ORJEkS9C+kLQamRMT+EbE9cCowr5ppSZIkDW/NfgTHHOBGYGpEtEfE6Zm5EZgNXAssA67KzDsGbqqSJEnDR1O/k5aZp3VRXwAsqHRGkiRJ8muhJEmSSmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKtNVCWkS8NiL+OyK+ExEvr9UOiIgvRcTVW2sekiRJQ0FTIS0iLouIlRGxtKE+MyLuioi7I+Kc7vaRmd/OzHcAfw+8sVa7NzNP7+PcJUmStlkjm+x3BXAx8JUthYgYAVwCvAxoBxZHxDxgBHB+w/i3ZebK2t8/UBsnSZKkLkRmNtcxYjIwPzOn1bZfAJyXma+obZ8LkJmNAW3L+AAuAK7LzOsb2q7OzFO6GHcGcAbA+PHj2+bOnfuM9rVr19La2trUMfRlXE/9umvvqq2x3lm/+lpfj7E3+vMcg7WWzdSaWeuqDfQ52UzfZs+97urDYS29vqsb6/Vd3Tiv72rGlXB9z5gxY0lmHt3jZLuSmU09gMnA0rrtU4BL67ZnARd3M/6fgCXAF4Aza7Wxte17gHN7mkNbW1s2Wrhw4bNqzWh2XE/9umvvqq2x3lm/+lpfj7E3+vMcg7WWzdSaWeuqDfQ52UzfZs+97urDYS29vqsb6/Vd3Tiv72rGlXB9Azdnkzmrs0ezL3d2JjrLfN2Ewc8Cn22oPQSc2Y85SJIkbZP68+7OdmBS3fZEYEX/piNJkiToX0hbDEyJiP0jYnvgVGBeNdOSJEka3pr9CI45wI3A1Ihoj4jTM3MjMBu4FlgGXJWZdwzcVCVJkoaPpn4nLTNP66K+AFhQ6YwkSZLk10JJkiSVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUypEmSJBXIkCZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJkmSVCBDmiRJUoEMaZIkSQUaubWeKCJeC7wa2BO4JDN/GBEtwEeAnYGbM/PLW2s+kiRJJWvqTlpEXBYRKyNiaUN9ZkTcFRF3R8Q53e0jM7+dme8A/h54Y618MjAB2AC093r2kiRJ26hmX+68AphZX4iIEcAlwCuBQ4HTIuLQiDg8IuY3PPasG/qB2jiAqcCNmfke4J39ORBJkqRtSVMvd2bmDRExuaF8LHB3Zt4LEBFzgZMz83zgxMZ9REQAFwDfz8xbauV24Ona3zf1fvqSJEnbpsjM5jp2hLT5mTmttn0KMDMz317bngUcl5mzuxj/T8D/ARYDv8rML0TEDsBFwJPAnZl5SSfjzgDOABg/fnzb3Llzn9G+du1aWltbmzqGvozrqV937V21NdY761df6+sx9kZ/nmOw1rKZWjNrXbWBPieb6dvsudddfTispdd3dWO9vqsb5/VdzbgSru8ZM2Ysycyje5xsVzKzqQcwGVhat/164NK67VnARc3ury+Ptra2bLRw4cJn1ZrR7Lie+nXX3lVbY72zfvW1vh5jb/TnOQZrLZupNbPWVRvoc7KZvs2ee93Vh8Naen1XN9bru7pxXt/VjCvh+qbjTZF9zj39+QiOdmBS3fZEYEU/9idJkqSa/oS0xcCUiNg/IrYHTgXmVTMtSZKk4a3Zj+CYA9wITI2I9og4PTM3ArOBa4FlwFWZecfATVWSJGn4aPbdnad1UV8ALKh0RpIkSfJroSRJkkpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCrTVQlpEHBIRX4iIqyPinbXaARHxpYi4emvNQ5IkaShoKqRFxGURsTIiljbUZ0bEXRFxd0Sc090+MnNZZp4JvAE4ula7NzNP7+vkJUmStlXN3km7AphZX4iIEcAlwCuBQ4HTIuLQiDg8IuY3PPasjTkJ+Cnwo8qOQJIkaRs0splOmXlDRExuKB8L3J2Z9wJExFzg5Mw8Hzixi/3MA+ZFxPeAK/s8a0mSpG1cZGZzHTtC2vzMnFbbPgWYmZlvr23PAo7LzNldjJ8OvA4YBfw6My+JiLHAx4CXAZfWAl7juDOAMwDGjx/fNnfu3Ge0r127ltbW1qaOoS/jeurXXXtXbY31zvrV1/p6jL3Rn+cYrLVsptbMWldtoM/JZvo2e+51Vx8Oa+n1Xd1Yr+/qxnl9VzOuhOt7xowZSzLz6B4n25XMbOoBTAaW1m2/no5gtWV7FnBRs/vry6OtrS0bLVy48Fm1ZjQ7rqd+3bV31dZY76xffa2vx9gb/XmOwVrLZmrNrHXVBvqcbKZvs+ded/XhsJZe39WN9fqubpzXdzXjSri+gZuzH7mnP+/ubAcm1W1PBFb0Y3+SJEmq6U9IWwxMiYj9I2J74FRgXjXTkiRJGt6a/QiOOcCNwNSIaI+I0zNzIzAbuBZYBlyVmXcM3FQlSZKGj2bf3XlaF/UFwIJKZyRJkiS/FkqSJKlEhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUAjt9YTRcQhwLuAccCPMvPzEdECfATYGbg5M7+8teYjSZJUsqbupEXEZRGxMiKWNtRnRsRdEXF3RJzT3T4yc1lmngm8ATi6Vj4ZmABsANp7P31JkqRtU7Mvd14BzKwvRMQI4BLglcChwGkRcWhEHB4R8xsee9bGnAT8FPhRbTdTgRsz8z3AO/t/OJIkSduGyMzmOkZMBuZn5rTa9guA8zLzFbXtcwEy8/wm9vW9zHx1RLwFeDozr4qIb2TmGzvpewZwBsD48ePb5s6d+4z2tWvX0tra2tQx9GVcT/26a++qrbHeWb/6Wl+PsTf68xyDtZbN1JpZ66oN9DnZTN9mz73u6sNhLb2+qxvr9V3dOK/vasaVcH3PmDFjSWYeTV9lZlMPYDKwtG77FODSuu1ZwMXdjJ8OfBb4IvB/a7UdgC8BF22pdfdoa2vLRgsXLnxWrRnNjuupX3ftXbU11jvrV1/r6zH2Rn+eY7DWsplaM2tdtYE+J5vp2+y51119OKyl13d1Y72+qxvn9V3NuBKubzp+377prNX46M8bB6KzzNdNGFwELGqoPQmc3o85SJIkbZP68xEc7cCkuu2JwIr+TUeSJEnQv5C2GJgSEftHxPbAqcC8aqYlSZI0vDX7ERxzgBuBqRHRHhGnZ+ZGYDZwLbAMuCoz7xi4qUqSJA0fTf1OWmae1kV9AbCg0hlJkiTJr4WSJEkqkSFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgpkSJMkSSqQIU2SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFNkiSpQIY0SZKkAhnSJEmSCmRIkyRJKpAhTZIkqUCGNEmSpAIZ0iRJkgoUmTnYc2haRKwC/thQ3gV4tA+7a3ZcT/26a++qrbHeWb/62jhgdY8z7Z++rmNvxla9ls3UGrdLXsvejOvrWvamPhzW0uu7urFe39WN8/quZlwJ1/fUzNyp56l2ITOH9AP4r4Ec11O/7tq7amusd9avvgbcXOo6DuZaNlPrZLvYtezNuL6uZW/qw2Etvb7LXcvhek72Zy29vqtZx96s5UBf39vCy53fHeBxPfXrrr2rtsZ6Z/36elx91Z/nG6y1bKa2tdexP8/Zm3F9Xcve1IfDWnp9VzfW67u6cV7f1Ywb8tf3kHq5c7iKiJsz8+jBnse2wLWsjmtZDdexOq5ldVzLavR3HbeFO2nDwX8N9gS2Ia5ldVzLariO1XEtq+NaVqNf6+idNEmSpAJ5J02SJKlAhjRJkqQCGdIkSZIKZEiTJEkqkCFtiIuI6RHxvxHxhYiYPtjzGeoiYseIWBIRJw72XIaqiDikdj5eHRHvHOz5DGUR8dqI+O+I+E5EvHyw5zOURcQBEfGliLh6sOcy1NT+Xfxy7Vx882DPZyjr7XloSBtEEXFZRKyMiKUN9ZkRcVdE3B0R5/SwmwTWAqOB9oGaa+kqWkuA9wFXDcwsy1fFOmbmssw8E3gDMGw/Z6mitfx2Zr4D+HvgjQM43aJVtJb3ZubpAzvToaOXa/o64OrauXjSVp9s4Xqzlr09D/0IjkEUES+mI2B9JTOn1WojgN8CL6MjdC0GTgNGAOc37OJtwOrM3BwR44H/yMxh+X85Fa3lc+n4vrrRdKzr/K0z+3JUsY6ZuTIiTgLOAS7OzCu31vxLUtVa1sZ9Cvh6Zt6ylaZflIrX8urMPGVrzb1UvVzTk4HvZ+avIuLKzHzTIE27SL1Zy8z8Ta29qfNw5IDNWj3KzBsiYnJD+Vjg7sy8FyAi5gInZ+b5QHcvwa0BRg3IRIeAKtYyImYAOwKHAusiYkFmbh7YmZelqnMyM+cB8yLie8CwDGkVnZMBXEDHD8hhGdCg8n8rRe/WlI6QMRH4Fb4C9yy9XMvf9GbfLnZ5JgDL67bba7VORcTrIuKLwFeBiwd4bkNNr9YyM9+fme+mI1T893ALaN3o7Tk5PSI+WzsvFwz05IaYXq0l8I/AS4FTIuLMgZzYENTb83JsRHwBODIizh3oyQ1RXa3pNcDfRsTnGZzv+ByKOl3L3p6H3kkrT3RS6/I16cy8ho4LSM/Wq7X8c4fMK6qfypDW23NyEbBooCYzxPV2LT8LfHbgpjOk9XYtHwIMut3rdE0z8wngrVt7MkNcV2vZq/PQO2nlaQcm1W1PBFYM0lyGOteyGq5jdVzL6riW1XNNq1PJWhrSyrMYmBIR+0fE9sCpwLxBntNQ5VpWw3WsjmtZHdeyeq5pdSpZS0PaIIqIOcCNwNSIaI+I0zNzIzAbuBZYBlyVmXcM5jyHAteyGq5jdVzL6riW1XNNqzOQa+lHcEiSJBXIO2mSJEkFMqRJkiQVyJAmSZJUIEOaJElSgQxpkiRJBTKkSZIkFciQJmmbEBFjIuInETGiov2NjIjvRcTqiJjWQ9/LImJlRCxtqP97RPxVFfORNPwY0iRtK94GXJOZmyra3+eBu4CTgW9ExMRu+l4BzOykfhFwTkXzkTTMGNIkFS0ijomIX0fE6IjYMSLu6OLO1puB79SNOzsibo+I2yLiglptUUR8OiJuiIhltX1fExG/i4iP1o39IPBoZr4nM38GvB2YExG7dDbHzLwBeLiT+h+BsRGxV78WQdKwNHKwJyBJ3cnMxRExD/goMAb4WmY2vqy4PXBAZv6htv1K4LXAcZn5ZETsXtf96cx8cUS8i45Q10ZHwLonIj6dmQ9l5oca5nAj8KI+HsItwAnAN/s4XtIwZUiTNBR8mI4vLF4P/FMn7eOAR+q2XwpcnplPAmRm/V2uLV9yfDtwR2b+CSAi7gUmAQ9VOnNYCexT8T4lDQO+3ClpKNgdaAV2AkZ30r6uoR5AV19M/FTtz811f9+y3eP/uEbEpIj4Ve1xZk/9a/Na10Q/SXoGQ5qkoeC/gP8HfB34RGNjZq4BRkTElqD2Q+BtEbEDQMPLnf2Smcsz84ja4wtNDDkYWNpjL0lqYEiTVLSI+DtgY2ZeCVwAHNPFx1r8EHghQGb+gI6XNW+OiF8B7x3gOc4BbgSmRkR7RJxeq28HHATcPJDPL2nbFJldvSIgSUNHRBwJvCczZw32XLaIiL8BjsrM/zfYc5E09HgnTdI2ITNvBRZW9WG2FRkJfGqwJyFpaPJOmiRJUoG8kyZJklQgQ5okSVKBDGmSJEkFMqRJkiQVyJAmSZJUoP8PCnva6GRJE3sAAAAASUVORK5CYII=\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "30571da147ea417889001bf1baee1892": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "787a6042616a4f759ddff99e75303ee1": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_86616a516ac84d4e94d57172e3988879",
                    "IPY_MODEL_c7de3e1ea6dd440ab43e48ba40ec27d3",
                    "IPY_MODEL_0b44e4bd5e694aff810dad71b0cb81af",
                    "IPY_MODEL_11eca9aaf1734666971724d8042e57e3",
                    "IPY_MODEL_54fb09366a834e94a06a7e498f9b1cb2",
                    "IPY_MODEL_027b39d71b7340a6ab80912a76e0cce9"
                ],
                "layout": "IPY_MODEL_30571da147ea417889001bf1baee1892"
            }
        },
        "3906d4d2813249d6a55cc0240ea82e65": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "9b113739a2cd4f5dac1bc6775e659109": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "86616a516ac84d4e94d57172e3988879": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_3906d4d2813249d6a55cc0240ea82e65",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_9b113739a2cd4f5dac1bc6775e659109",
                "value": 10
            }
        },
        "1bc2840daa7f44078cfc501f9351c128": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "29e7b8edbe594e8fa37ca2c2a429e2c1": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "c7de3e1ea6dd440ab43e48ba40ec27d3": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x1",
                "layout": "IPY_MODEL_1bc2840daa7f44078cfc501f9351c128",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_29e7b8edbe594e8fa37ca2c2a429e2c1",
                "value": -5
            }
        },
        "91b4d4af0cd74b3bbd1a0f1208d130a9": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "8fe0cb4e51704cb1804b9d651a481987": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "0b44e4bd5e694aff810dad71b0cb81af": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x2",
                "layout": "IPY_MODEL_91b4d4af0cd74b3bbd1a0f1208d130a9",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_8fe0cb4e51704cb1804b9d651a481987",
                "value": 10
            }
        },
        "443e432df86a4bd4bcab24ceecae0202": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "4f1d9197dd9f41c8b58ecfa69e5db485": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "11eca9aaf1734666971724d8042e57e3": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y1",
                "layout": "IPY_MODEL_443e432df86a4bd4bcab24ceecae0202",
                "max": 1,
                "min": -15,
                "style": "IPY_MODEL_4f1d9197dd9f41c8b58ecfa69e5db485",
                "value": -15
            }
        },
        "ed09248f15844d138f9e5e3cb6b53c5c": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "b9a5e67154c148f08e5bf0a06bd8cb55": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "54fb09366a834e94a06a7e498f9b1cb2": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y2",
                "layout": "IPY_MODEL_ed09248f15844d138f9e5e3cb6b53c5c",
                "max": 1,
                "min": -15,
                "style": "IPY_MODEL_b9a5e67154c148f08e5bf0a06bd8cb55",
                "value": 1
            }
        },
        "3a37d6eee57e42b2ab501419573309fb": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "027b39d71b7340a6ab80912a76e0cce9": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_3a37d6eee57e42b2ab501419573309fb",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "...\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "134858da6c014c1e976b9864f73df639": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "f68cb24bbfef40e99a672d106c157a4f": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_5c946ad535f740f8b9b86e9815b01983",
                    "IPY_MODEL_db1dc199c72a47d2919cbf5149376122",
                    "IPY_MODEL_40486172a6eb490cb040826a50971ffb",
                    "IPY_MODEL_d169ae46f7234cfa9be75122819f28d6",
                    "IPY_MODEL_9486f6e58693403f8c9a70026e0068e8",
                    "IPY_MODEL_a6204041645b479c899badd63be3a8c2"
                ],
                "layout": "IPY_MODEL_134858da6c014c1e976b9864f73df639"
            }
        },
        "8b276c58d10a425091996a10bf9def3c": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "a93b0268860a4933bef26f5e496e9703": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "5c946ad535f740f8b9b86e9815b01983": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_8b276c58d10a425091996a10bf9def3c",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_a93b0268860a4933bef26f5e496e9703",
                "value": 10
            }
        },
        "89fc95bbdab64c5292e8040ac411ccd4": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "fb99dac578224192bbc011cf4c29dae1": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "db1dc199c72a47d2919cbf5149376122": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x1",
                "layout": "IPY_MODEL_89fc95bbdab64c5292e8040ac411ccd4",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_fb99dac578224192bbc011cf4c29dae1",
                "value": -5
            }
        },
        "bba6392f5450437f916403f6635b2e56": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "b0faaa5ff9984bf38ece6edaa02c7eff": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "40486172a6eb490cb040826a50971ffb": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x2",
                "layout": "IPY_MODEL_bba6392f5450437f916403f6635b2e56",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_b0faaa5ff9984bf38ece6edaa02c7eff",
                "value": 10
            }
        },
        "117bb4c1d871440faec331d4fe52588d": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "0f8fdc634afc417894e0178d8b49fd56": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "d169ae46f7234cfa9be75122819f28d6": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y1",
                "layout": "IPY_MODEL_117bb4c1d871440faec331d4fe52588d",
                "max": 1,
                "min": -15,
                "style": "IPY_MODEL_0f8fdc634afc417894e0178d8b49fd56",
                "value": -15
            }
        },
        "50f252e644774d7cb6102f3699bd21ec": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "3da40775a8684dda8e83e04e377f04a8": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "9486f6e58693403f8c9a70026e0068e8": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y2",
                "layout": "IPY_MODEL_50f252e644774d7cb6102f3699bd21ec",
                "max": 1,
                "min": -30,
                "style": "IPY_MODEL_3da40775a8684dda8e83e04e377f04a8",
                "value": 1
            }
        },
        "8b7156b04ca8422096bed7a09532fdee": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "a6204041645b479c899badd63be3a8c2": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_8b7156b04ca8422096bed7a09532fdee",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "...\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "2df50a7d07d144e49a4aea704322924b": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "4f9104b5cefc49e889099b5b5705ee23": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_ed968db3ec014d3eb1756c2176d619a2",
                    "IPY_MODEL_537552be67eb47c2ba882ec1e755f7fe",
                    "IPY_MODEL_251040c08d1744bc9153df0e6f3309f0",
                    "IPY_MODEL_a3bf5f119b8d4f9aaea523480502e3c5",
                    "IPY_MODEL_eeac8705b2d0475bb6c614f05634aedd",
                    "IPY_MODEL_50f0877749944077b312ff0eae3c8523"
                ],
                "layout": "IPY_MODEL_2df50a7d07d144e49a4aea704322924b"
            }
        },
        "95ea01408aa7411a93a8e4114cff674c": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "1715f063c2e9484cbecd939027abcc1b": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "ed968db3ec014d3eb1756c2176d619a2": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_95ea01408aa7411a93a8e4114cff674c",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_1715f063c2e9484cbecd939027abcc1b",
                "value": 16
            }
        },
        "86af1d68cb77482199f635f750e630c5": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "e65a0ed6861f4396aae618c9a1d962ee": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "537552be67eb47c2ba882ec1e755f7fe": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x1",
                "layout": "IPY_MODEL_86af1d68cb77482199f635f750e630c5",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_e65a0ed6861f4396aae618c9a1d962ee",
                "value": -5
            }
        },
        "b4600aba9d3b4254bd01208303dc67c9": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "12954ec1c20443b89974d141fc4cadbf": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "251040c08d1744bc9153df0e6f3309f0": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x2",
                "layout": "IPY_MODEL_b4600aba9d3b4254bd01208303dc67c9",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_12954ec1c20443b89974d141fc4cadbf",
                "value": 10
            }
        },
        "ba2801d230ee4170b6f11e589b7b11b3": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "6fe223c359db4ccdac3f3ad736be05ab": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "a3bf5f119b8d4f9aaea523480502e3c5": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y1",
                "layout": "IPY_MODEL_ba2801d230ee4170b6f11e589b7b11b3",
                "max": 1,
                "min": -30,
                "style": "IPY_MODEL_6fe223c359db4ccdac3f3ad736be05ab",
                "value": -30
            }
        },
        "47c04bbfcb304b038575c31e0957ff0c": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "071fff74622c4f2b8063ce07d6859c6a": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "eeac8705b2d0475bb6c614f05634aedd": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y2",
                "layout": "IPY_MODEL_47c04bbfcb304b038575c31e0957ff0c",
                "max": 1,
                "min": -30,
                "style": "IPY_MODEL_071fff74622c4f2b8063ce07d6859c6a",
                "value": 1
            }
        },
        "29dc50c88fee455bbf8eb50af75d45a3": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "50f0877749944077b312ff0eae3c8523": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_29dc50c88fee455bbf8eb50af75d45a3",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "...\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "ddbc1ecb9546466c9d2836ad4a4a677e": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "9885be12f96646bf95c87b898078f191": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_088fe2e4f5224a34bcd7e611dd1f6834",
                    "IPY_MODEL_b7e76a3c8722429598efe1512122dc94",
                    "IPY_MODEL_e289108f2b154e57b07886b20b2f32f4",
                    "IPY_MODEL_1944b6e473c045c090df7edc1c2e3ffb",
                    "IPY_MODEL_a10f1669d3854d30926dfe1658314f16",
                    "IPY_MODEL_7c3585171d5649bdab551730b75c5ea3"
                ],
                "layout": "IPY_MODEL_ddbc1ecb9546466c9d2836ad4a4a677e"
            }
        },
        "59f26ef8afdc4948a8ce9e3cf919ddef": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "add64a57a2b8479a896b32ca11d391ce": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "088fe2e4f5224a34bcd7e611dd1f6834": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_59f26ef8afdc4948a8ce9e3cf919ddef",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_add64a57a2b8479a896b32ca11d391ce",
                "value": 10
            }
        },
        "263c1a59fee14d2994f60b23565adbe5": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "49f264dedf0242c2bf7f8ed7dc03763f": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "b7e76a3c8722429598efe1512122dc94": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x1",
                "layout": "IPY_MODEL_263c1a59fee14d2994f60b23565adbe5",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_49f264dedf0242c2bf7f8ed7dc03763f",
                "value": -5
            }
        },
        "8d7ec51cdba848fe89bf9f9bc34c8b73": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "002c238623dc4f209c45719eef1f189c": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "e289108f2b154e57b07886b20b2f32f4": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x2",
                "layout": "IPY_MODEL_8d7ec51cdba848fe89bf9f9bc34c8b73",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_002c238623dc4f209c45719eef1f189c",
                "value": 10
            }
        },
        "3b3d4210b5af4aca96495e4c6b383fef": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "af569eea639b45e4810cf0ed9b3f9a79": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "1944b6e473c045c090df7edc1c2e3ffb": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y1",
                "layout": "IPY_MODEL_3b3d4210b5af4aca96495e4c6b383fef",
                "max": 1,
                "min": -30,
                "style": "IPY_MODEL_af569eea639b45e4810cf0ed9b3f9a79",
                "value": -30
            }
        },
        "c5a78e66756d4c629023bfb4cc24cbe4": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "62b87148e7ba42b6b739cee556591bce": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "a10f1669d3854d30926dfe1658314f16": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y2",
                "layout": "IPY_MODEL_c5a78e66756d4c629023bfb4cc24cbe4",
                "max": 1,
                "min": -30,
                "style": "IPY_MODEL_62b87148e7ba42b6b739cee556591bce",
                "value": 1
            }
        },
        "71e55709a70448a18bf23fd0aef56c26": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "7c3585171d5649bdab551730b75c5ea3": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_71e55709a70448a18bf23fd0aef56c26",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "...\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "4db07e8108924981b116e62258b313f7": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "5ea5d4a8f64a445d98c0ca73b42388ca": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_bf8df68164ce47f7971c4717c8d79ac7",
                    "IPY_MODEL_6b13782137134c2da969baa875320ab1",
                    "IPY_MODEL_6685cd160c17433b85a5093271e3ece3",
                    "IPY_MODEL_67762b284c2c403cbbc62a3a2be4f240",
                    "IPY_MODEL_238a93c8fdf14a70abc84d2e9e3e7058",
                    "IPY_MODEL_6c228ac9b36041ffa016ba84fabd04bb"
                ],
                "layout": "IPY_MODEL_4db07e8108924981b116e62258b313f7"
            }
        },
        "13c1b4cd34df45abb0b2c5839bf566d1": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "7a437f90b58a463db1279c5c8a800d3c": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "bf8df68164ce47f7971c4717c8d79ac7": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_13c1b4cd34df45abb0b2c5839bf566d1",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_7a437f90b58a463db1279c5c8a800d3c",
                "value": 17
            }
        },
        "ce5e331cb0c845e0b83d272f62df1986": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "bef9b65c67f646158ff21cba3e3f9f19": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "6b13782137134c2da969baa875320ab1": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x1",
                "layout": "IPY_MODEL_ce5e331cb0c845e0b83d272f62df1986",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_bef9b65c67f646158ff21cba3e3f9f19",
                "value": 3
            }
        },
        "57afc0081ff74d99ab088177815070cf": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "639150713b0049a8881e4e4678ba9d80": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "6685cd160c17433b85a5093271e3ece3": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x2",
                "layout": "IPY_MODEL_57afc0081ff74d99ab088177815070cf",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_639150713b0049a8881e4e4678ba9d80",
                "value": 5
            }
        },
        "6de0af5f70174fc98250c13ea83c7a15": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "79cfa0198e734570a730f1bc89795951": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "67762b284c2c403cbbc62a3a2be4f240": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y1",
                "layout": "IPY_MODEL_6de0af5f70174fc98250c13ea83c7a15",
                "max": 1,
                "min": -30,
                "style": "IPY_MODEL_79cfa0198e734570a730f1bc89795951",
                "value": -17
            }
        },
        "84b2865b56db4eb4a05e8a6af6bbc91a": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "889163c884d34694bac7fa79ab6e7c91": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "238a93c8fdf14a70abc84d2e9e3e7058": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y2",
                "layout": "IPY_MODEL_84b2865b56db4eb4a05e8a6af6bbc91a",
                "max": 1,
                "min": -30,
                "style": "IPY_MODEL_889163c884d34694bac7fa79ab6e7c91",
                "value": -11
            }
        },
        "7a26fc2e78f2447ebbad4b99c7b65bcf": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "6c228ac9b36041ffa016ba84fabd04bb": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_7a26fc2e78f2447ebbad4b99c7b65bcf",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "...\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "1f942366822f4d55aa8fb9dfe0eae50d": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "62b8ac902f8f432b9a34cc3ed6845c41": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_dadfc2ad1a9845b3adf9853149583b49",
                    "IPY_MODEL_99cf3bdbad22446398fd963ff003f0a3",
                    "IPY_MODEL_79826ae1e6644dc78e0194a8418e5e9a",
                    "IPY_MODEL_39b7a2ce66e24432a339daff307dd120",
                    "IPY_MODEL_fa5e63d67d174a768cec98751a6736b9",
                    "IPY_MODEL_2959247b210f45f1826473bf5ea92b8c"
                ],
                "layout": "IPY_MODEL_1f942366822f4d55aa8fb9dfe0eae50d"
            }
        },
        "0224ec62aa7449429f3c4db87e04639e": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "d420be53094e4bf791626c64a15d4cbf": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "dadfc2ad1a9845b3adf9853149583b49": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_0224ec62aa7449429f3c4db87e04639e",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_d420be53094e4bf791626c64a15d4cbf",
                "value": 10
            }
        },
        "2a76c624c2424f029b8dfe9012ec0f2b": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "69df0c75426b4602ad89b56b19c91862": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "99cf3bdbad22446398fd963ff003f0a3": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x1",
                "layout": "IPY_MODEL_2a76c624c2424f029b8dfe9012ec0f2b",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_69df0c75426b4602ad89b56b19c91862",
                "value": -5
            }
        },
        "0fc6171f718849b8a518443f887c8e4e": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "95c7e1e4e5024d0281584e70d80d223a": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "79826ae1e6644dc78e0194a8418e5e9a": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "x2",
                "layout": "IPY_MODEL_0fc6171f718849b8a518443f887c8e4e",
                "max": 10,
                "min": -5,
                "style": "IPY_MODEL_95c7e1e4e5024d0281584e70d80d223a",
                "value": 10
            }
        },
        "3410c6a525a94b05a4d3189ca6349502": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "b3cb9273efb5496892db0e22139c8937": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "39b7a2ce66e24432a339daff307dd120": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y1",
                "layout": "IPY_MODEL_3410c6a525a94b05a4d3189ca6349502",
                "max": 1,
                "min": -30,
                "style": "IPY_MODEL_b3cb9273efb5496892db0e22139c8937",
                "value": -30
            }
        },
        "36a74701b8934aadb3c4aab3a496c108": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "d467eaeaf4764a1dacae4a1d1c103589": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "fa5e63d67d174a768cec98751a6736b9": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "y2",
                "layout": "IPY_MODEL_36a74701b8934aadb3c4aab3a496c108",
                "max": 1,
                "min": -30,
                "style": "IPY_MODEL_d467eaeaf4764a1dacae4a1d1c103589"
            }
        },
        "6bb0d7d664b0482cac3ba946103a6aed": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "2959247b210f45f1826473bf5ea92b8c": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_6bb0d7d664b0482cac3ba946103a6aed",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "...\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "382370055405442293508080e51c5718": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "3c174e62b51f4e10bd886ef10b4268a3": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "layout": "IPY_MODEL_382370055405442293508080e51c5718"
            }
        },
        "c44940eeb7ad471a8f1bbce6a7ddba7d": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "57ef4069aa2b4702a85ee35ccebc2c61": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_4566bbf4476c49b693b4024914661ecf",
                    "IPY_MODEL_ec3f219f40ab42cfa7648afdb4cad4d5"
                ],
                "layout": "IPY_MODEL_c44940eeb7ad471a8f1bbce6a7ddba7d"
            }
        },
        "a7669429372941108735c53cb588f67c": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "a6ee831c563641aba10284f257974a9c": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "4566bbf4476c49b693b4024914661ecf": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_a7669429372941108735c53cb588f67c",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_a6ee831c563641aba10284f257974a9c",
                "value": 18
            }
        },
        "87f51edc2dc7450dbbf7739d4db93daa": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "ec3f219f40ab42cfa7648afdb4cad4d5": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_87f51edc2dc7450dbbf7739d4db93daa",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 432x288 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAYIAAAD4CAYAAADhNOGaAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAAQbUlEQVR4nO3df6zdd13H8efLFsZv2ezdLG1jh2nQjQSZzRyghGTqyiB0GmdKBBqdaTCbgtFoJwnwT5PhDyIYh6kwKbowKj9cww/HUiHERDYuY7B1ZbSwuV1W2gtEhpoMOt7+cb4zJ3fntnf3e3buuf08H8nJ93s+38/nfN/9nG/v636/58dNVSFJatePrXQBkqSVZRBIUuMMAklqnEEgSY0zCCSpcWtXuoDTWbduXW3evHmly5CkVWPdunXccsstt1TVtqX0n/og2Lx5M7OzsytdhiStKknWLbWvl4YkqXEGgSQ1ziCQpMYZBJLUOINAkhpnEEhS4wwCSWqcQSBJjTMIJKlxU//J4j427/7EyPb7r3vVhCuRpOnlGYEkNc4gkKTGGQSS1DiDQJIaZxBIUuMMAklqnEEgSY0zCCSpcQaBJDXutEGQ5IYkJ5LcPdR2TpJbkxzplmcPbbs2ydEk9ya5bKj955Pc1W17d5KM/58jSXqilnJG8H5g24K23cDBqtoCHOzuk+QCYAdwYTfm+iRrujHvAXYBW7rbwseUJK2A0wZBVX0O+O6C5u3Avm59H3DFUPtNVfVIVd0HHAUuTrIeeE5V/UdVFfCBoTGSpBW03NcIzquqYwDd8tyufQPw4FC/ua5tQ7e+sH2kJLuSzCaZnZ+fX2aJkqSlGPeLxaOu+9cp2keqqr1VtbWqts7MzIytOEnS4y03CI53l3volie69jlg01C/jcBDXfvGEe2SpBW23CA4AOzs1ncCNw+170hyVpLzGbwofHt3+ej7SS7p3i30hqExkqQVdNo/TJPkg8ArgHVJ5oC3AdcB+5NcBTwAXAlQVYeS7AfuAU4CV1fVo91D/R6DdyA9HfhUd5MkrbDTBkFVvXaRTZcu0n8PsGdE+yzwwidUnSTpSecniyWpcQaBJDXOIJCkxhkEktQ4g0CSGmcQSFLjDAJJapxBIEmNMwgkqXEGgSQ1ziCQpMYZBJLUOINAkhpnEEhS4wwCSWqcQSBJjTMIJKlxBoEkNc4gkKTGGQSS1DiDQJIaZxBIUuMMAklqnEEgSY0zCCSpcQaBJDXOIJCkxhkEktQ4g0CSGmcQSFLjegVBkj9McijJ3Uk+mORpSc5JcmuSI93y7KH+1yY5muTeJJf1L1+S1NeygyDJBuAPgK1V9UJgDbAD2A0crKotwMHuPkku6LZfCGwDrk+ypl/5kqS++l4aWgs8Pcla4BnAQ8B2YF+3fR9wRbe+Hbipqh6pqvuAo8DFPfcvSepp2UFQVd8E/hJ4ADgGfK+qPg2cV1XHuj7HgHO7IRuAB4ceYq5re5wku5LMJpmdn59fbomSpCXoc2nobAa/5Z8PPA94ZpLXnWrIiLYa1bGq9lbV1qraOjMzs9wSJUlL0OfS0C8D91XVfFX9EPgo8FLgeJL1AN3yRNd/Dtg0NH4jg0tJkqQV1CcIHgAuSfKMJAEuBQ4DB4CdXZ+dwM3d+gFgR5KzkpwPbAFu77F/SdIYrF3uwKq6LcmHgTuAk8CXgL3As4D9Sa5iEBZXdv0PJdkP3NP1v7qqHu1ZvySpp2UHAUBVvQ1424LmRxicHYzqvwfY02efkqTx8pPFktQ4g0CSGmcQSFLjDAJJapxBIEmNMwgkqXEGgSQ1ziCQpMYZBJLUOINAkhpnEEhS4wwCSWqcQSBJjTMIJKlxBoEkNc4gkKTGGQSS1DiDQJIaZxBIUuMMAklqnEEgSY0zCCSpcQaBJDXOIJCkxhkEktQ4g0CSGmcQSFLjDAJJapxBIEmN6xUESZ6b5MNJvprkcJKXJDknya1JjnTLs4f6X5vkaJJ7k1zWv3xJUl99zwjeBfxrVf0M8CLgMLAbOFhVW4CD3X2SXADsAC4EtgHXJ1nTc/+SpJ6WHQRJngO8HHgfQFX9oKr+C9gO7Ou67QOu6Na3AzdV1SNVdR9wFLh4ufuXJI1HnzOC5wPzwD8k+VKS9yZ5JnBeVR0D6Jbndv03AA8OjZ/r2iRJK6hPEKwFLgLeU1UvBv6H7jLQIjKirUZ2THYlmU0yOz8/36NESdLp9AmCOWCuqm7r7n+YQTAcT7IeoFueGOq/aWj8RuChUQ9cVXuramtVbZ2ZmelRoiTpdJYdBFX1LeDBJC/omi4F7gEOADu7tp3Azd36AWBHkrOSnA9sAW5f7v4lSeOxtuf43wduTPJU4BvAbzMIl/1JrgIeAK4EqKpDSfYzCIuTwNVV9WjP/UuSeuoVBFV1J7B1xKZLF+m/B9jTZ5+SpPHyk8WS1DiDQJIaZxBIUuMMAklqnEEgSY0zCCSpcQaBJDXOIJCkxhkEktQ4g0CSGmcQSFLjDAJJapxBIEmNMwgkqXEGgSQ1ziCQpMYZBJLUOINAkhpnEEhS4wwCSWqcQSBJjTMIJKlxBoEkNc4gkKTGGQSS1DiDQJIaZxBIUuMMAklqnEEgSY0zCCSpcb2DIMmaJF9K8vHu/jlJbk1ypFuePdT32iRHk9yb5LK++5Yk9TeOM4I3AYeH7u8GDlbVFuBgd58kFwA7gAuBbcD1SdaMYf+SpB56BUGSjcCrgPcONW8H9nXr+4ArhtpvqqpHquo+4ChwcZ/9S5L663tG8NfAnwA/Gmo7r6qOAXTLc7v2DcCDQ/3murbHSbIryWyS2fn5+Z4lSpJOZdlBkOTVwImq+uJSh4xoq1Edq2pvVW2tqq0zMzPLLVGStARre4x9GfCaJJcDTwOek+SfgONJ1lfVsSTrgRNd/zlg09D4jcBDPfYvSRqDZZ8RVNW1VbWxqjYzeBH436rqdcABYGfXbSdwc7d+ANiR5Kwk5wNbgNuXXbkkaSz6nBEs5jpgf5KrgAeAKwGq6lCS/cA9wEng6qp69EnYvyTpCRhLEFTVZ4HPduvfAS5dpN8eYM849ilJGg8/WSxJjTMIJKlxBoEkNc4gkKTGGQSS1DiDQJIaZxBIUuMMAklqnEEgSY0zCCSpcQaBJDXOIJCkxhkEktQ4g0CSGmcQSFLjDAJJapxBIEmNMwgkqXEGgSQ1ziCQpMYZBJLUOINAkhpnEEhS4wwCSWqcQSBJjTMIJKlxBoEkNc4gkKTGGQSS1DiDQJIat+wgSLIpyWeSHE5yKMmbuvZzktya5Ei3PHtozLVJjia5N8ll4/gHSJL66XNGcBL4o6r6WeAS4OokFwC7gYNVtQU42N2n27YDuBDYBlyfZE2f4iVJ/S07CKrqWFXd0a1/HzgMbAC2A/u6bvuAK7r17cBNVfVIVd0HHAUuXu7+JUnjMZbXCJJsBl4M3AacV1XHYBAWwLldtw3Ag0PD5rq2UY+3K8lsktn5+flxlChJWkTvIEjyLOAjwJur6uFTdR3RVqM6VtXeqtpaVVtnZmb6lihJOoVeQZDkKQxC4Maq+mjXfDzJ+m77euBE1z4HbBoavhF4qM/+JUn99XnXUID3AYer6p1Dmw4AO7v1ncDNQ+07kpyV5HxgC3D7cvcvSRqPtT3Gvgx4PXBXkju7tj8DrgP2J7kKeAC4EqCqDiXZD9zD4B1HV1fVoz32L0kag2UHQVX9O6Ov+wNcusiYPcCe5e5TkjR+frJYkhpnEEhS4wwCSWqcQSBJjTMIJKlxBoEkNc4gkKTGGQSS1DiDQJIaZxBIUuMMAklqnEEgSY0zCCSpcQaBJDXOIJCkxhkEktQ4g0CSGmcQSFLjDAJJapxBIEmNMwgkqXEGgSQ1ziCQpMYZBJLUOINAkhpnEEhS4wwCSWqcQSBJjTMIJKlxBoEkNW7tpHeYZBvwLmAN8N6qum7SNWze/YmR7fdf96oJVyKpddPw82iiQZBkDfC3wK8Ac8AXkhyoqnsmWcdinugTMg1PoCT1NekzgouBo1X1DYAkNwHbgakIgsUs9gN/XP1X0kqF3EqG6LienzMh8FfqeZjGX6LG9YvgapSqmtzOkt8AtlXV73b3Xw/8QlVds6DfLmBXd/cFwL3L3OU64NvLHLtSVlvNq61esOZJWW01r7Z6YfGavw1QVduW8iCTPiPIiLbHJVFV7QX29t5ZMltVW/s+ziSttppXW71gzZOy2mpebfXC+Gqe9LuG5oBNQ/c3Ag9NuAZJ0pBJB8EXgC1Jzk/yVGAHcGDCNUiShkz00lBVnUxyDXALg7eP3lBVh57EXfa+vLQCVlvNq61esOZJWW01r7Z6YUw1T/TFYknS9PGTxZLUOINAkhp3RgRBkm1J7k1yNMnuEduT5N3d9q8kuWgl6uxq2ZTkM0kOJzmU5E0j+rwiyfeS3Nnd3roStS6o6f4kd3X1zI7YPjVz3NXzgqH5uzPJw0nevKDPis9zkhuSnEhy91DbOUluTXKkW569yNhTHvcTrvkvkny1e+4/luS5i4w95XE0wXrfnuSbQ8/95YuMnaY5/tBQvfcnuXORsU98jqtqVd8YvOj8deD5wFOBLwMXLOhzOfApBp9juAS4bQXrXQ9c1K0/G/jaiHpfAXx8ped2QU33A+tOsX1q5niRY+RbwE9N2zwDLwcuAu4eavtzYHe3vht4xyL/plMe9xOu+VeBtd36O0bVvJTjaIL1vh344yUcN1Mzxwu2/xXw1nHN8ZlwRvD/X1tRVT8AHvvaimHbgQ/UwOeB5yZZP+lCAarqWFXd0a1/HzgMbFiJWsZsauZ4hEuBr1fVf650IQtV1eeA7y5o3g7s69b3AVeMGLqU4/5JMarmqvp0VZ3s7n6ewWeEpsIic7wUUzXHj0kS4DeBD45rf2dCEGwAHhy6P8fjf7Aupc/EJdkMvBi4bcTmlyT5cpJPJblwspWNVMCnk3yx+wqQhaZyjjs7WPw/zbTNM8B5VXUMBr84AOeO6DPN8/07DM4ORzndcTRJ13SXsm5Y5PLbtM7xLwHHq+rIItuf8ByfCUGwlK+tWNJXW0xSkmcBHwHeXFUPL9h8B4PLGC8C/gb4lwmXN8rLquoi4JXA1UlevmD71M0xQPfBxdcA/zxi8zTO81JN63y/BTgJ3LhIl9MdR5PyHuCngZ8DjjG41LLQVM4x8FpOfTbwhOf4TAiCpXxtxVR9tUWSpzAIgRur6qMLt1fVw1X13936J4GnJFk34TIX1vRQtzwBfIzBafOwqZrjIa8E7qiq4ws3TOM8d44/dlmtW54Y0Wfq5jvJTuDVwG9Vd7F6oSUcRxNRVcer6tGq+hHw94vUMY1zvBb4deBDi/VZzhyfCUGwlK+tOAC8oXtnyyXA9x479Z607vre+4DDVfXORfr8ZNePJBczeJ6+M7kqH1fPM5M8+7F1Bi8M3r2g29TM8QKL/vY0bfM85ACws1vfCdw8os9UfV1LBn9w6k+B11TV/y7SZynH0UQseP3q1xapY6rmuPPLwFeram7UxmXP8SReAX+ybwzesfI1Bq/wv6VreyPwxm49DP4gzteBu4CtK1jrLzI4vfwKcGd3u3xBvdcAhxi8S+HzwEtXeH6f39Xy5a6uqZ7jobqfweAH+48PtU3VPDMIqWPADxn8BnoV8BPAQeBItzyn6/s84JNDYx933K9gzUcZXE9/7Jj+u4U1L3YcrVC9/9gdp19h8MN9/bTPcdf+/seO36G+vefYr5iQpMadCZeGJEk9GASS1DiDQJIaZxBIUuMMAklqnEEgSY0zCCSpcf8He0auLISVavsAAAAASUVORK5CYII=\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "cfd0981381764a57b4a31cc8d67c7fee": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "dc9a361aaf8e46ec8027828444339d2e": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_464eb563de5540c3843687a5a38d052f",
                    "IPY_MODEL_13443bd67f304faab4293bf4c99595b3"
                ],
                "layout": "IPY_MODEL_cfd0981381764a57b4a31cc8d67c7fee"
            }
        },
        "f50a7f98f46f4d439cfdbcbe8a61afb8": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "5631e5cc9bc749cab79662c05ab70881": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "464eb563de5540c3843687a5a38d052f": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_f50a7f98f46f4d439cfdbcbe8a61afb8",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_5631e5cc9bc749cab79662c05ab70881",
                "value": 10
            }
        },
        "927c0b4548cc4358a56b3c41281af4ad": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "13443bd67f304faab4293bf4c99595b3": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_927c0b4548cc4358a56b3c41281af4ad",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 432x288 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAX0AAAD4CAYAAAAAczaOAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAAOUklEQVR4nO3db4hdd53H8fdnM1r7h2JKpyUmYRMhVNOC1B261YIsxqVZKqYPtjBCu0G6BJaoVQRJfNJHgTwQsQvbQmh1s1gaQiw06K5riYosLK3TP9AmMXRousmY2Iy7qMUHqanffTCn5nactJl7p/fG+b1fEO65v3vOPb+cJu97cube21QVkqQ2/MWoJyBJGh6jL0kNMfqS1BCjL0kNMfqS1JCxUU/gnVx77bW1bt26UU9Dkv6sPPPMM7+qqvH545d89NetW8fU1NSopyFJf1aS/M9C417ekaSGGH1JaojRl6SGGH1JaojRl6SGGH1JaojRl6SGGH1JaojRl6SGXPKfyB3Euh3f/+PyK7vvGOFMJOnS4Jm+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXkHaOf5FtJziR5sWfsmiRPJnmpu13Z89jOJNNJjiW5vWf8r5K80D32z0my9L8dSdLbuZgz/X8FNs8b2wEcqqoNwKHuPkk2ApPAjd02DyZZ0W3zELAN2ND9mv+ckqR32TtGv6p+CvzfvOEtwN5ueS9wZ8/4vqo6W1XHgWngliSrgKur6r+rqoB/69lGkjQk/V7Tv76qTgN0t9d146uBkz3rzXRjq7vl+eMLSrItyVSSqdnZ2T6nKEmab6l/kLvQdfp6m/EFVdWeqpqoqonx8fElm5wkta7f6L/aXbKhuz3Tjc8Aa3vWWwOc6sbXLDAuSRqifqN/ENjaLW8FnugZn0xyWZL1zP3A9unuEtBrSW7t3rXzDz3bSJKGZOydVkjyGPA3wLVJZoD7gd3A/iT3AieAuwCq6nCS/cAR4Bywvare6J7qn5h7J9DlwH90vyRJQ/SO0a+qz17goU0XWH8XsGuB8SngpkXNTpK0pPxEriQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkMGin6SLyc5nOTFJI8leV+Sa5I8meSl7nZlz/o7k0wnOZbk9sGnL0lajL6jn2Q18EVgoqpuAlYAk8AO4FBVbQAOdfdJsrF7/EZgM/BgkhWDTV+StBiDXt4ZAy5PMgZcAZwCtgB7u8f3And2y1uAfVV1tqqOA9PALQPuX5K0CH1Hv6p+AXwdOAGcBn5TVT8Erq+q0906p4Hruk1WAyd7nmKmG/sTSbYlmUoyNTs72+8UJUnzDHJ5ZyVzZ+/rgQ8AVya5++02WWCsFlqxqvZU1URVTYyPj/c7RUnSPINc3vkUcLyqZqvq98DjwMeBV5OsAuhuz3TrzwBre7Zfw9zlIEnSkAwS/RPArUmuSBJgE3AUOAhs7dbZCjzRLR8EJpNclmQ9sAF4eoD9S5IWaazfDavqqSQHgGeBc8BzwB7gKmB/knuZe2G4q1v/cJL9wJFu/e1V9caA85ckLULf0QeoqvuB++cNn2XurH+h9XcBuwbZpySpf34iV5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaMlD0k7w/yYEkP09yNMnHklyT5MkkL3W3K3vW35lkOsmxJLcPPn1J0mIMeqb/APCDqvoQ8BHgKLADOFRVG4BD3X2SbAQmgRuBzcCDSVYMuH9J0iL0Hf0kVwOfAB4BqKrXq+rXwBZgb7faXuDObnkLsK+qzlbVcWAauKXf/UuSFm+QM/0PArPAt5M8l+ThJFcC11fVaYDu9rpu/dXAyZ7tZ7qxP5FkW5KpJFOzs7MDTFGS1GuQ6I8BHwUeqqqbgd/RXcq5gCwwVgutWFV7qmqiqibGx8cHmKIkqdcg0Z8BZqrqqe7+AeZeBF5Nsgqguz3Ts/7anu3XAKcG2L8kaZH6jn5V/RI4meSGbmgTcAQ4CGztxrYCT3TLB4HJJJclWQ9sAJ7ud/+SpMUbG3D7LwCPJnkv8DLwOeZeSPYnuRc4AdwFUFWHk+xn7oXhHLC9qt4YcP+SpEUYKPpV9TwwscBDmy6w/i5g1yD7lCT1z0/kSlJDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNWTg6CdZkeS5JN/r7l+T5MkkL3W3K3vW3ZlkOsmxJLcPum9J0uIsxZn+fcDRnvs7gENVtQE41N0nyUZgErgR2Aw8mGTFEuxfknSRBop+kjXAHcDDPcNbgL3d8l7gzp7xfVV1tqqOA9PALYPsX5K0OIOe6X8T+Crwh56x66vqNEB3e103vho42bPeTDcmSRqSvqOf5NPAmap65mI3WWCsLvDc25JMJZmanZ3td4qSpHkGOdO/DfhMkleAfcAnk3wHeDXJKoDu9ky3/gywtmf7NcCphZ64qvZU1URVTYyPjw8wRUlSr76jX1U7q2pNVa1j7ge0P6qqu4GDwNZuta3AE93yQWAyyWVJ1gMbgKf7nrkkadHG3oXn3A3sT3IvcAK4C6CqDifZDxwBzgHbq+qNd2H/kqQLWJLoV9VPgJ90y/8LbLrAeruAXUuxT0nS4vmJXElqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqSN/RT7I2yY+THE1yOMl93fg1SZ5M8lJ3u7Jnm51JppMcS3L7UvwGJEkXb5Az/XPAV6rqw8CtwPYkG4EdwKGq2gAc6u7TPTYJ3AhsBh5MsmKQyUuSFqfv6FfV6ap6tlt+DTgKrAa2AHu71fYCd3bLW4B9VXW2qo4D08At/e5fkrR4S3JNP8k64GbgKeD6qjoNcy8MwHXdaquBkz2bzXRjCz3ftiRTSaZmZ2eXYoqSJJYg+kmuAr4LfKmqfvt2qy4wVgutWFV7qmqiqibGx8cHnaIkqTNQ9JO8h7ngP1pVj3fDryZZ1T2+CjjTjc8Aa3s2XwOcGmT/kqTFGeTdOwEeAY5W1Td6HjoIbO2WtwJP9IxPJrksyXpgA/B0v/uXJC3e2ADb3gbcA7yQ5Plu7GvAbmB/knuBE8BdAFV1OMl+4Ahz7/zZXlVvDLB/SdIi9R39qvovFr5OD7DpAtvsAnb1u09J0mD8RK4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNWRs2DtMshl4AFgBPFxVu4ex33U7vv/H5Vd23zGMXUrSogyjU0ONfpIVwL8AfwvMAD9LcrCqjgxzHr4ASGrVsM/0bwGmq+plgCT7gC3AUKPfq/cF4FLR+0J0ofkt9sXKFzq9yT8Lw3OhYz3K/wapquHtLPl7YHNV/WN3/x7gr6vq8/PW2wZs6+7eABzrc5fXAr/qc9vlyONxnsfirTwe5y2XY/GXVTU+f3DYZ/pZYOxPXnWqag+wZ+CdJVNVNTHo8ywXHo/zPBZv5fE4b7kfi2G/e2cGWNtzfw1washzkKRmDTv6PwM2JFmf5L3AJHBwyHOQpGYN9fJOVZ1L8nngP5l7y+a3qurwu7jLgS8RLTMej/M8Fm/l8ThvWR+Lof4gV5I0Wn4iV5IaYvQlqSHLMvpJNic5lmQ6yY5Rz2eUkqxN8uMkR5McTnLfqOc0aklWJHkuyfdGPZdRS/L+JAeS/Lz7M/KxUc9plJJ8uft78mKSx5K8b9RzWmrLLvo9X/Xwd8BG4LNJNo52ViN1DvhKVX0YuBXY3vjxALgPODrqSVwiHgB+UFUfAj5Cw8clyWrgi8BEVd3E3JtNJkc7q6W37KJPz1c9VNXrwJtf9dCkqjpdVc92y68x95d69WhnNTpJ1gB3AA+Pei6jluRq4BPAIwBV9XpV/Xqkkxq9MeDyJGPAFSzDzxEtx+ivBk723J+h4cj1SrIOuBl4asRTGaVvAl8F/jDieVwKPgjMAt/uLnc9nOTKUU9qVKrqF8DXgRPAaeA3VfXD0c5q6S3H6F/UVz20JslVwHeBL1XVb0c9n1FI8mngTFU9M+q5XCLGgI8CD1XVzcDvgGZ/BpZkJXNXBdYDHwCuTHL3aGe19JZj9P2qh3mSvIe54D9aVY+Pej4jdBvwmSSvMHfZ75NJvjPaKY3UDDBTVW/+y+8Acy8CrfoUcLyqZqvq98DjwMdHPKcltxyj71c99EgS5q7ZHq2qb4x6PqNUVTurak1VrWPuz8WPqmrZncldrKr6JXAyyQ3d0CZG+DXnl4ATwK1Jruj+3mxiGf5ge+j/56x32wi+6uFSdxtwD/BCkue7sa9V1b+Pbkq6hHwBeLQ7QXoZ+NyI5zMyVfVUkgPAs8y96+05luFXMvg1DJLUkOV4eUeSdAFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSH/DzYnjXtG8K8tAAAAAElFTkSuQmCC\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "c592c58d5c1d43d3adfda01696fc5479": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "3750c368c8a64d76902f07d8844faab0": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_b2bc6c02ecc148b2b70b23d853f623d5",
                    "IPY_MODEL_5802c4bca6f4467c930c0dcdbfef4c80"
                ],
                "layout": "IPY_MODEL_c592c58d5c1d43d3adfda01696fc5479"
            }
        },
        "2ff4d6aa31ae41b09f07752d37bc1802": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "898d230b6a8c4b6e9cf25c00fa3154cb": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "b2bc6c02ecc148b2b70b23d853f623d5": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_2ff4d6aa31ae41b09f07752d37bc1802",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_898d230b6a8c4b6e9cf25c00fa3154cb",
                "value": 10
            }
        },
        "8beadccd881645fdbd863b0f2481dd61": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "5802c4bca6f4467c930c0dcdbfef4c80": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_8beadccd881645fdbd863b0f2481dd61",
                "outputs": [
                    {
                        "output_type": "error",
                        "ename": "AttributeError",
                        "evalue": "'Rectangle' object has no property 'normed'",
                        "traceback": [
                            "\u001b[1;31m---------------------------------------------------------------------------\u001b[0m",
                            "\u001b[1;31mAttributeError\u001b[0m                            Traceback (most recent call last)",
                            "\u001b[1;32mC:\\ProgramData\\Anaconda3\\lib\\site-packages\\ipywidgets\\widgets\\interaction.py\u001b[0m in \u001b[0;36mupdate\u001b[1;34m(self, *args)\u001b[0m\n\u001b[0;32m    254\u001b[0m                     \u001b[0mvalue\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mwidget\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mget_interact_value\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m    255\u001b[0m                     \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[1;33m[\u001b[0m\u001b[0mwidget\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0m_kwarg\u001b[0m\u001b[1;33m]\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mvalue\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m--> 256\u001b[1;33m                 \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mresult\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mf\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;33m**\u001b[0m\u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m    257\u001b[0m                 \u001b[0mshow_inline_matplotlib_plots\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m    258\u001b[0m                 \u001b[1;32mif\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mauto_display\u001b[0m \u001b[1;32mand\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mresult\u001b[0m \u001b[1;32mis\u001b[0m \u001b[1;32mnot\u001b[0m \u001b[1;32mNone\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n",
                            "\u001b[1;32m<ipython-input-44-67bbd1a2b4b9>\u001b[0m in \u001b[0;36mgraph_error\u001b[1;34m(Z)\u001b[0m\n\u001b[0;32m     12\u001b[0m     \u001b[0mYfit\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0marray\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mYfit\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m     13\u001b[0m \u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m---> 14\u001b[1;33m     \u001b[0mhist\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mYfit\u001b[0m \u001b[1;33m-\u001b[0m \u001b[0mFFy\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mbins\u001b[0m \u001b[1;33m=\u001b[0m \u001b[1;36m100\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mnormed\u001b[0m\u001b[1;33m=\u001b[0m\u001b[1;32mTrue\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m",
                            "\u001b[1;32mC:\\ProgramData\\Anaconda3\\lib\\site-packages\\matplotlib\\pyplot.py\u001b[0m in \u001b[0;36mhist\u001b[1;34m(x, bins, range, density, weights, cumulative, bottom, histtype, align, orientation, rwidth, log, color, label, stacked, data, **kwargs)\u001b[0m\n\u001b[0;32m   2683\u001b[0m         \u001b[0morientation\u001b[0m\u001b[1;33m=\u001b[0m\u001b[1;34m'vertical'\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mrwidth\u001b[0m\u001b[1;33m=\u001b[0m\u001b[1;32mNone\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mlog\u001b[0m\u001b[1;33m=\u001b[0m\u001b[1;32mFalse\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mcolor\u001b[0m\u001b[1;33m=\u001b[0m\u001b[1;32mNone\u001b[0m\u001b[1;33m,\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m   2684\u001b[0m         label=None, stacked=False, *, data=None, **kwargs):\n\u001b[1;32m-> 2685\u001b[1;33m     return gca().hist(\n\u001b[0m\u001b[0;32m   2686\u001b[0m         \u001b[0mx\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mbins\u001b[0m\u001b[1;33m=\u001b[0m\u001b[0mbins\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mrange\u001b[0m\u001b[1;33m=\u001b[0m\u001b[0mrange\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mdensity\u001b[0m\u001b[1;33m=\u001b[0m\u001b[0mdensity\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mweights\u001b[0m\u001b[1;33m=\u001b[0m\u001b[0mweights\u001b[0m\u001b[1;33m,\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m   2687\u001b[0m         \u001b[0mcumulative\u001b[0m\u001b[1;33m=\u001b[0m\u001b[0mcumulative\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mbottom\u001b[0m\u001b[1;33m=\u001b[0m\u001b[0mbottom\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mhisttype\u001b[0m\u001b[1;33m=\u001b[0m\u001b[0mhisttype\u001b[0m\u001b[1;33m,\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n",
                            "\u001b[1;32mC:\\ProgramData\\Anaconda3\\lib\\site-packages\\matplotlib\\__init__.py\u001b[0m in \u001b[0;36minner\u001b[1;34m(ax, data, *args, **kwargs)\u001b[0m\n\u001b[0;32m   1436\u001b[0m     \u001b[1;32mdef\u001b[0m \u001b[0minner\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0max\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;33m*\u001b[0m\u001b[0margs\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0mdata\u001b[0m\u001b[1;33m=\u001b[0m\u001b[1;32mNone\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;33m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m   1437\u001b[0m         \u001b[1;32mif\u001b[0m \u001b[0mdata\u001b[0m \u001b[1;32mis\u001b[0m \u001b[1;32mNone\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m-> 1438\u001b[1;33m             \u001b[1;32mreturn\u001b[0m \u001b[0mfunc\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0max\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;33m*\u001b[0m\u001b[0mmap\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0msanitize_sequence\u001b[0m\u001b[1;33m,\u001b[0m \u001b[0margs\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;33m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m   1439\u001b[0m \u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m   1440\u001b[0m         \u001b[0mbound\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mnew_sig\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mbind\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0max\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;33m*\u001b[0m\u001b[0margs\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;33m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n",
                            "\u001b[1;32mC:\\ProgramData\\Anaconda3\\lib\\site-packages\\matplotlib\\axes\\_axes.py\u001b[0m in \u001b[0;36mhist\u001b[1;34m(self, x, bins, range, density, weights, cumulative, bottom, histtype, align, orientation, rwidth, log, color, label, stacked, **kwargs)\u001b[0m\n\u001b[0;32m   6818\u001b[0m             \u001b[1;32mif\u001b[0m \u001b[0mpatch\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m   6819\u001b[0m                 \u001b[0mp\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mpatch\u001b[0m\u001b[1;33m[\u001b[0m\u001b[1;36m0\u001b[0m\u001b[1;33m]\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m-> 6820\u001b[1;33m                 \u001b[0mp\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mupdate\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m   6821\u001b[0m                 \u001b[1;32mif\u001b[0m \u001b[0mlbl\u001b[0m \u001b[1;32mis\u001b[0m \u001b[1;32mnot\u001b[0m \u001b[1;32mNone\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m   6822\u001b[0m                     \u001b[0mp\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mset_label\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mlbl\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n",
                            "\u001b[1;32mC:\\ProgramData\\Anaconda3\\lib\\site-packages\\matplotlib\\artist.py\u001b[0m in \u001b[0;36mupdate\u001b[1;34m(self, props)\u001b[0m\n\u001b[0;32m    994\u001b[0m                     \u001b[0mfunc\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mgetattr\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mself\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;34mf\"set_{k}\"\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;32mNone\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m    995\u001b[0m                     \u001b[1;32mif\u001b[0m \u001b[1;32mnot\u001b[0m \u001b[0mcallable\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mfunc\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m--> 996\u001b[1;33m                         raise AttributeError(f\"{type(self).__name__!r} object \"\n\u001b[0m\u001b[0;32m    997\u001b[0m                                              f\"has no property {k!r}\")\n\u001b[0;32m    998\u001b[0m                     \u001b[0mret\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mappend\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mfunc\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mv\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n",
                            "\u001b[1;31mAttributeError\u001b[0m: 'Rectangle' object has no property 'normed'"
                        ]
                    }
                ]
            }
        },
        "38f9d4b7192d48248d7ba5ba91e2b829": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "380af0ca8d8a41879e45619cc02aecad": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_63fecfdf60a649ffbca224d33e7cac30",
                    "IPY_MODEL_f60957bfeaf24f7e800c60497c1889b8"
                ],
                "layout": "IPY_MODEL_38f9d4b7192d48248d7ba5ba91e2b829"
            }
        },
        "77766548058642fbadd9a82ec167c31b": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "4925e162c1a94b66b613ede121c142ef": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "63fecfdf60a649ffbca224d33e7cac30": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_77766548058642fbadd9a82ec167c31b",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_4925e162c1a94b66b613ede121c142ef",
                "value": 1
            }
        },
        "09e6778196f741db8932d3de2cd9bc03": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "f60957bfeaf24f7e800c60497c1889b8": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_09e6778196f741db8932d3de2cd9bc03",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 432x288 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAYoAAAD4CAYAAADy46FuAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAAWa0lEQVR4nO3df4xd5X3n8fdn8YbS7EIMOFnXJjUJdFVAu85iGaRuqmxpwE2qQlrYGq2Cq0VyQETaqlupsFkJSoQUUmWRUDZURFgY1PCjsBFIhU3dUDWqRIAhoeFHQhl+tDhY4MaIQlJYmXz3j/sMvTPceWY8P+5g835JR/fc73me4+fRGH/mnOfcS6oKSZJm8y9WegCSpHc2g0KS1GVQSJK6DApJUpdBIUnqWrXSA1hqxx57bG3YsGGlhyFJB5WHH374H6pqzahjh1xQbNiwgYmJiZUehiQdVJL83WzHvPUkSeoyKCRJXQaFJKnLoJAkdRkUkqQug0KS1GVQSJK65gyKJDuSvJTksaHabUkeadtzSR5p9Q1J/mno2B8P9Tk1yaNJJpNcmyStfng732SSB5JsGOqzLclTbdu2lBOXJM3PfD5wdyPwZeCmqUJV/fbUfpIvAa8MtX+6qjaOOM91wHbg28A9wBbgXuBC4OWqOiHJVuBq4LeTHA1cDmwCCng4yd1V9fK8ZydJWrQ5g6KqvjX8W/6wdlXwn4Ff6Z0jyVrgyKq6v72/CTiHQVCcDVzRmt4BfLmd9yxgV1Xta312MQiXW+Ya87vNhkv/7K39577wyRUciaRD0WLXKD4KvFhVTw3Vjk/y3SR/leSjrbYO2D3UZnerTR17HqCq9jO4OjlmuD6izzRJtieZSDKxd+/eRU5JkjRssUFxPtN/w98DfLCqPgL8HvC1JEcCGdF36v/BOtuxXp/pxarrq2pTVW1as2bkd1pJkhZowUGRZBXwm8BtU7WqeqOqftT2HwaeBn6BwdXA+qHu64EX2v5u4Lihcx4F7Buuj+gjSRqTxVxR/Crwg6p665ZSkjVJDmv7HwJOBJ6pqj3Aq0lOb+sPFwB3tW53A1NPNJ0L3FdVBXwDODPJ6iSrgTNbTZI0RnMuZie5BfgYcGyS3cDlVXUDsJW3Lyz/MnBlkv3Am8BFU4vRwMUMnqA6gsEi9r2tfgNwc5JJBlcSWwGqal+SzwMPtXZXDp1LkjQm83nq6fxZ6r8zonYncOcs7SeAU0bUXwfOm6XPDmDHXGOUJC0fP5ktSeoyKCRJXQaFJKnLoJAkdRkUkqQug0KS1GVQSJK6DApJUpdBIUnqMigkSV0GhSSpy6CQJHUZFJKkLoNCktRlUEiSugwKSVKXQSFJ6jIoJEldBoUkqcugkCR1zRkUSXYkeSnJY0O1K5L8MMkjbfvE0LHLkkwmeTLJWUP1U5M82o5dmyStfniS21r9gSQbhvpsS/JU27Yt2awlSfM2nyuKG4EtI+rXVNXGtt0DkOQkYCtwcuvzlSSHtfbXAduBE9s2dc4LgZer6gTgGuDqdq6jgcuB04DNwOVJVh/wDCVJizJnUFTVt4B98zzf2cCtVfVGVT0LTAKbk6wFjqyq+6uqgJuAc4b67Gz7dwBntKuNs4BdVbWvql4GdjE6sCRJy2gxaxSfTfK9dmtq6jf9dcDzQ212t9q6tj+zPq1PVe0HXgGO6ZzrbZJsTzKRZGLv3r2LmJIkaaaFBsV1wIeBjcAe4EutnhFtq1NfaJ/pxarrq2pTVW1as2ZNZ9iSpAO1oKCoqher6s2q+inwVQZrCDD4rf+4oabrgRdaff2I+rQ+SVYBRzG41TXbuSRJY7SgoGhrDlM+BUw9EXU3sLU9yXQ8g0XrB6tqD/BqktPb+sMFwF1DfaaeaDoXuK+tY3wDODPJ6nZr68xWkySN0aq5GiS5BfgYcGyS3QyeRPpYko0MbgU9B3wGoKoeT3I78ASwH7ikqt5sp7qYwRNURwD3tg3gBuDmJJMMriS2tnPtS/J54KHW7sqqmu+iuiRpicwZFFV1/ojyDZ32VwFXjahPAKeMqL8OnDfLuXYAO+YaoyRp+fjJbElSl0EhSeoyKCRJXQaFJKnLoJAkdRkUkqQug0KS1GVQSJK6DApJUpdBIUnqMigkSV0GhSSpy6CQJHUZFJKkLoNCktRlUEiSugwKSVKXQSFJ6jIoJEldBoUkqcugkCR1zRkUSXYkeSnJY0O1P0rygyTfS/L1JO9r9Q1J/inJI23746E+pyZ5NMlkkmuTpNUPT3Jbqz+QZMNQn21JnmrbtqWcuCRpfuZzRXEjsGVGbRdwSlX9O+BvgcuGjj1dVRvbdtFQ/TpgO3Bi26bOeSHwclWdAFwDXA2Q5GjgcuA0YDNweZLVBzA3SdISmDMoqupbwL4ZtT+vqv3t7beB9b1zJFkLHFlV91dVATcB57TDZwM72/4dwBntauMsYFdV7auqlxmE08zAkiQts6VYo/ivwL1D749P8t0kf5Xko622Dtg91GZ3q00dex6ghc8rwDHD9RF9pkmyPclEkom9e/cudj6SpCGLCooknwP2A3/SSnuAD1bVR4DfA76W5EggI7rX1GlmOdbrM71YdX1VbaqqTWvWrDmQKUiS5rDgoGiLy78O/Jd2O4mqeqOqftT2HwaeBn6BwdXA8O2p9cALbX83cFw75yrgKAa3ut6qj+gjSRqTBQVFki3AHwC/UVU/GaqvSXJY2/8Qg0XrZ6pqD/BqktPb+sMFwF2t293A1BNN5wL3teD5BnBmktVtEfvMVpMkjdGquRokuQX4GHBskt0MnkS6DDgc2NWecv12e8Lpl4Erk+wH3gQuqqqphfCLGTxBdQSDNY2pdY0bgJuTTDK4ktgKUFX7knweeKi1u3LoXJKkMZkzKKrq/BHlG2Zpeydw5yzHJoBTRtRfB86bpc8OYMdcY5QkLR8/mS1J6jIoJEldBoUkqcugkCR1GRSSpC6DQpLUZVBIkroMCklSl0EhSeoyKCRJXQaFJKnLoJAkdRkUkqQug0KS1GVQSJK6DApJUpdBIUnqMigkSV0GhSSpy6CQJHXNGRRJdiR5KcljQ7Wjk+xK8lR7XT107LIkk0meTHLWUP3UJI+2Y9cmSasfnuS2Vn8gyYahPtvan/FUkm1LNmtJ0rzN54riRmDLjNqlwDer6kTgm+09SU4CtgIntz5fSXJY63MdsB04sW1T57wQeLmqTgCuAa5u5zoauBw4DdgMXD4cSJKk8ZgzKKrqW8C+GeWzgZ1tfydwzlD91qp6o6qeBSaBzUnWAkdW1f1VVcBNM/pMnesO4Ix2tXEWsKuq9lXVy8Au3h5YkqRlttA1ig9U1R6A9vr+Vl8HPD/UbnerrWv7M+vT+lTVfuAV4JjOud4myfYkE0km9u7du8ApSZJGWerF7IyoVae+0D7Ti1XXV9Wmqtq0Zs2aeQ1UkjQ/Cw2KF9vtJNrrS62+GzhuqN164IVWXz+iPq1PklXAUQxudc12LknSGC00KO4Gpp5C2gbcNVTf2p5kOp7BovWD7fbUq0lOb+sPF8zoM3Wuc4H72jrGN4Azk6xui9hntpokaYxWzdUgyS3Ax4Bjk+xm8CTSF4Dbk1wI/D1wHkBVPZ7kduAJYD9wSVW92U51MYMnqI4A7m0bwA3AzUkmGVxJbG3n2pfk88BDrd2VVTVzUV2StMzmDIqqOn+WQ2fM0v4q4KoR9QnglBH112lBM+LYDmDHXGOUJC0fP5ktSeoyKCRJXQaFJKnLoJAkdRkUkqQug0KS1GVQSJK6DApJUpdBIUnqMigkSV0GhSSpy6CQJHUZFJKkLoNCktRlUEiSugwKSVKXQSFJ6jIoJEldBoUkqcugkCR1LTgokvzbJI8Mbf+Y5HeTXJHkh0P1Twz1uSzJZJInk5w1VD81yaPt2LVJ0uqHJ7mt1R9IsmFRs5UkHbAFB0VVPVlVG6tqI3Aq8BPg6+3wNVPHquoegCQnAVuBk4EtwFeSHNbaXwdsB05s25ZWvxB4uapOAK4Brl7oeCVJC7NUt57OAJ6uqr/rtDkbuLWq3qiqZ4FJYHOStcCRVXV/VRVwE3DOUJ+dbf8O4Iypqw1J0ngsVVBsBW4Zev/ZJN9LsiPJ6lZbBzw/1GZ3q61r+zPr0/pU1X7gFeCYJRqzJGkeFh0USd4D/Abwp610HfBhYCOwB/jSVNMR3atT7/WZOYbtSSaSTOzdu3f+g5ckzWkprih+DfhOVb0IUFUvVtWbVfVT4KvA5tZuN3DcUL/1wAutvn5EfVqfJKuAo4B9MwdQVddX1aaq2rRmzZolmJIkacpSBMX5DN12amsOUz4FPNb27wa2tieZjmewaP1gVe0BXk1yelt/uAC4a6jPtrZ/LnBfW8eQJI3JqsV0TvKzwMeBzwyVv5hkI4NbRM9NHauqx5PcDjwB7Acuqao3W5+LgRuBI4B72wZwA3BzkkkGVxJbFzNeSdKBW1RQVNVPmLG4XFWf7rS/CrhqRH0COGVE/XXgvMWMUZK0OH4yW5LUZVBIkroMCklSl0EhSeoyKCRJXQaFJKnLoJAkdRkUkqQug0KS1GVQSJK6DApJUpdBIUnqMigkSV0GhSSpy6CQJHUZFJKkLoNCktRlUEiSugwKSVKXQSFJ6jIoJEldiwqKJM8leTTJI0kmWu3oJLuSPNVeVw+1vyzJZJInk5w1VD+1nWcyybVJ0uqHJ7mt1R9IsmEx45UkHbiluKL4T1W1sao2tfeXAt+sqhOBb7b3JDkJ2AqcDGwBvpLksNbnOmA7cGLbtrT6hcDLVXUCcA1w9RKMV5J0AJbj1tPZwM62vxM4Z6h+a1W9UVXPApPA5iRrgSOr6v6qKuCmGX2mznUHcMbU1YYkaTwWGxQF/HmSh5Nsb7UPVNUegPb6/lZfBzw/1Hd3q61r+zPr0/pU1X7gFeCYmYNIsj3JRJKJvXv3LnJKkqRhqxbZ/5eq6oUk7wd2JflBp+2oK4Hq1Ht9pheqrgeuB9i0adPbjkuSFm5RVxRV9UJ7fQn4OrAZeLHdTqK9vtSa7waOG+q+Hnih1dePqE/rk2QVcBSwbzFjliQdmAUHRZL3JvnXU/vAmcBjwN3AttZsG3BX278b2NqeZDqewaL1g+321KtJTm/rDxfM6DN1rnOB+9o6hiRpTBZz6+kDwNfb2vIq4GtV9X+TPATcnuRC4O+B8wCq6vEktwNPAPuBS6rqzXaui4EbgSOAe9sGcANwc5JJBlcSWxcxXknSAiw4KKrqGeDfj6j/CDhjlj5XAVeNqE8Ap4yov04LGknSyvCT2ZKkLoNCktRlUEiSugwKSVKXQSFJ6jIoJEldBoUkqcugkCR1GRSSpC6DQpLUZVBIkroMCklSl0EhSeoyKCRJXQaFJKnLoJAkdRkUkqQug0KS1GVQSJK6DApJUteCgyLJcUn+Msn3kzye5L+1+hVJfpjkkbZ9YqjPZUkmkzyZ5Kyh+qlJHm3Hrk2SVj88yW2t/kCSDYuYqyRpARZzRbEf+O9V9YvA6cAlSU5qx66pqo1tuwegHdsKnAxsAb6S5LDW/jpgO3Bi27a0+oXAy1V1AnANcPUixitJWoAFB0VV7amq77T9V4HvA+s6Xc4Gbq2qN6rqWWAS2JxkLXBkVd1fVQXcBJwz1Gdn278DOGPqakOSNB5LskbRbgl9BHiglT6b5HtJdiRZ3WrrgOeHuu1utXVtf2Z9Wp+q2g+8Ahwz4s/fnmQiycTevXuXYkqSpGbRQZHkXwF3Ar9bVf/I4DbSh4GNwB7gS1NNR3SvTr3XZ3qh6vqq2lRVm9asWXNgE5AkdS0qKJL8SwYh8SdV9X8AqurFqnqzqn4KfBXY3JrvBo4b6r4eeKHV14+oT+uTZBVwFLBvMWOWJB2YxTz1FOAG4PtV9b+G6muHmn0KeKzt3w1sbU8yHc9g0frBqtoDvJrk9HbOC4C7hvpsa/vnAve1dQxJ0pisWkTfXwI+DTya5JFW+x/A+Uk2MrhF9BzwGYCqejzJ7cATDJ6YuqSq3mz9LgZuBI4A7m0bDILo5iSTDK4kti5ivJKkBVhwUFTVXzN6DeGeTp+rgKtG1CeAU0bUXwfOW+gYJUmL5yezJUldBoUkqcugkCR1GRSSpC6DQpLUZVBIkroMCklSl0EhSeoyKCRJXQaFJKnLoJAkdRkUkqQug0KS1GVQSJK6DApJUpdBIUnqMigkSV0GhSSpy6CQJHUZFJKkLoNCktR1UARFki1JnkwymeTSlR6PJL2brFrpAcwlyWHA/wY+DuwGHkpyd1U9sbIjm78Nl/7ZW/vPfeGTh8yfJend4R0fFMBmYLKqngFIcitwNrAsQTH8D+18DP9jPJ++B3r+xZjvn2WgSOo5GIJiHfD80PvdwGnDDZJsB7a3t68leXLGOY4F/mE5Bperl+OsC7ageb7D5jAfy/bzfIdxnoeWd/o8f362AwdDUGREraa9qboeuH7WEyQTVbVpqQf2TuM8Dy3O89ByMM/zYFjM3g0cN/R+PfDCCo1Fkt51DoageAg4McnxSd4DbAXuXuExSdK7xjv+1lNV7U/yWeAbwGHAjqp6/ABPM+ttqUOM8zy0OM9Dy0E7z1TV3K0kSe9aB8OtJ0nSCjIoJEldB1VQJDk6ya4kT7XX1bO0G/mVH73+SS5r7Z9MctZQ/aokzyd5bXlnN/dXlWTg2nb8e0n+w1x9FzLn5TbOeSY5JslfJnktyZfHM8P+WIeOL+U8P57k4SSPttdfGc8sxz7PzUkeadvfJPnUeGY5/v8+2/EPtr+7v7+8s5tDVR00G/BF4NK2fylw9Yg2hwFPAx8C3gP8DXBSrz9wUmt3OHB8639YO3Y6sBZ4bZnnNuu4h9p8AriXwWdLTgceWI45H2LzfC/wH4GLgC+P8e/quOf5EeDn2v4pwA8P0Xn+LLCq7a8FXpp6fyjNc+icdwJ/Cvz+uP7ujtoOqisKBl/dsbPt7wTOGdHmra/8qKr/B0x95Uev/9nArVX1RlU9C0y281BV366qPUs8j1F6455yNnBTDXwbeF+StXP0PeA5L7OxzrOqflxVfw28vpyTGmHc8/xuVU19vuhx4GeSHL5Mcxs27nn+pKr2t/rPMOPDt8to3P99kuQc4BkGP88VdbAFxQem/tFur+8f0WbUV36sm6N/r8+4zGcMs7U5mOY87nmulJWc528B362qNxY8+vkb+zyTnJbkceBR4KKh4FhOY51nkvcCfwD84RKNf1HecZ+jSPIXwL8Zcehz8z3FiNpcv3UspM9Sm88YZmtzMM153PNcKSsyzyQnA1cDZ86n/RIY+zyr6gHg5CS/COxMcm9VLfcV47jn+YfANVX1WjKq+3i944Kiqn51tmNJXkyytqr2tEu6l0Y0633lx2z93wlfEzKfMczW5j2dvu+0OY97nitl7PNMsh74OnBBVT29JLOY24r9PKvq+0l+zGBNZmJRs5jbuOd5GnBuki8C7wN+muT1qhrrAxlvWckFkgPdgD9i+sLPF0e0WcXgvt7x/PPC0cm9/sDJTF/YfYYZC7ss/2L2rOMeavNJpi+WPbiccz4U5jl0zt9hvIvZ4/55vq+1+61xzXGF5nk8/7yY/fMM/sE99lCb54zzXsEKL2av2B+8wB/WMcA3gafa69Gt/nPAPUPtPgH8LYMnDT43V/927HOt/ZPArw3Vv8jgN4WfttcrlnF+bxs3g6d1Lmr7YfA/cXqawf3ZTcsx5zH8HMc9z+eAfcBr7Wd40nLPcdzzBP4n8GPgkaHt/YfgPD/NYHH3EeA7wDmH6t/boTZXsMJB4Vd4SJK6DranniRJY2ZQSJK6DApJUpdBIUnqMigkSV0GhSSpy6CQJHX9f+GZNkT93yB+AAAAAElFTkSuQmCC\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "170574d3fbb2442a9ae4fc8a49f50fd0": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "f77128d9f6844f0e99c4440cf5623510": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_401f28e9ede240acaf80a9b6e0a73f61",
                    "IPY_MODEL_3879386bbc87497b9799d66b6f918718"
                ],
                "layout": "IPY_MODEL_170574d3fbb2442a9ae4fc8a49f50fd0"
            }
        },
        "f77f51e52e3842f180c9cfe5e76e3d48": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "cb22064224fb402fb1b7886f66f89485": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "401f28e9ede240acaf80a9b6e0a73f61": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_f77f51e52e3842f180c9cfe5e76e3d48",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_cb22064224fb402fb1b7886f66f89485",
                "value": 10
            }
        },
        "5332b5910163406c82995db7a06c4c07": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "3879386bbc87497b9799d66b6f918718": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_5332b5910163406c82995db7a06c4c07",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 432x288 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAX0AAAD4CAYAAAAAczaOAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAAOUklEQVR4nO3db4hdd53H8fdnM1r7h2JKpyUmYRMhVNOC1B261YIsxqVZKqYPtjBCu0G6BJaoVQRJfNJHgTwQsQvbQmh1s1gaQiw06K5riYosLK3TP9AmMXRousmY2Iy7qMUHqanffTCn5nactJl7p/fG+b1fEO65v3vOPb+cJu97cube21QVkqQ2/MWoJyBJGh6jL0kNMfqS1BCjL0kNMfqS1JCxUU/gnVx77bW1bt26UU9Dkv6sPPPMM7+qqvH545d89NetW8fU1NSopyFJf1aS/M9C417ekaSGGH1JaojRl6SGGH1JaojRl6SGGH1JaojRl6SGGH1JaojRl6SGXPKfyB3Euh3f/+PyK7vvGOFMJOnS4Jm+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXE6EtSQ4y+JDXkHaOf5FtJziR5sWfsmiRPJnmpu13Z89jOJNNJjiW5vWf8r5K80D32z0my9L8dSdLbuZgz/X8FNs8b2wEcqqoNwKHuPkk2ApPAjd02DyZZ0W3zELAN2ND9mv+ckqR32TtGv6p+CvzfvOEtwN5ueS9wZ8/4vqo6W1XHgWngliSrgKur6r+rqoB/69lGkjQk/V7Tv76qTgN0t9d146uBkz3rzXRjq7vl+eMLSrItyVSSqdnZ2T6nKEmab6l/kLvQdfp6m/EFVdWeqpqoqonx8fElm5wkta7f6L/aXbKhuz3Tjc8Aa3vWWwOc6sbXLDAuSRqifqN/ENjaLW8FnugZn0xyWZL1zP3A9unuEtBrSW7t3rXzDz3bSJKGZOydVkjyGPA3wLVJZoD7gd3A/iT3AieAuwCq6nCS/cAR4Bywvare6J7qn5h7J9DlwH90vyRJQ/SO0a+qz17goU0XWH8XsGuB8SngpkXNTpK0pPxEriQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkOMviQ1xOhLUkMGin6SLyc5nOTFJI8leV+Sa5I8meSl7nZlz/o7k0wnOZbk9sGnL0lajL6jn2Q18EVgoqpuAlYAk8AO4FBVbQAOdfdJsrF7/EZgM/BgkhWDTV+StBiDXt4ZAy5PMgZcAZwCtgB7u8f3And2y1uAfVV1tqqOA9PALQPuX5K0CH1Hv6p+AXwdOAGcBn5TVT8Erq+q0906p4Hruk1WAyd7nmKmG/sTSbYlmUoyNTs72+8UJUnzDHJ5ZyVzZ+/rgQ8AVya5++02WWCsFlqxqvZU1URVTYyPj/c7RUnSPINc3vkUcLyqZqvq98DjwMeBV5OsAuhuz3TrzwBre7Zfw9zlIEnSkAwS/RPArUmuSBJgE3AUOAhs7dbZCjzRLR8EJpNclmQ9sAF4eoD9S5IWaazfDavqqSQHgGeBc8BzwB7gKmB/knuZe2G4q1v/cJL9wJFu/e1V9caA85ckLULf0QeoqvuB++cNn2XurH+h9XcBuwbZpySpf34iV5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaMlD0k7w/yYEkP09yNMnHklyT5MkkL3W3K3vW35lkOsmxJLcPPn1J0mIMeqb/APCDqvoQ8BHgKLADOFRVG4BD3X2SbAQmgRuBzcCDSVYMuH9J0iL0Hf0kVwOfAB4BqKrXq+rXwBZgb7faXuDObnkLsK+qzlbVcWAauKXf/UuSFm+QM/0PArPAt5M8l+ThJFcC11fVaYDu9rpu/dXAyZ7tZ7qxP5FkW5KpJFOzs7MDTFGS1GuQ6I8BHwUeqqqbgd/RXcq5gCwwVgutWFV7qmqiqibGx8cHmKIkqdcg0Z8BZqrqqe7+AeZeBF5Nsgqguz3Ts/7anu3XAKcG2L8kaZH6jn5V/RI4meSGbmgTcAQ4CGztxrYCT3TLB4HJJJclWQ9sAJ7ud/+SpMUbG3D7LwCPJnkv8DLwOeZeSPYnuRc4AdwFUFWHk+xn7oXhHLC9qt4YcP+SpEUYKPpV9TwwscBDmy6w/i5g1yD7lCT1z0/kSlJDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNWTg6CdZkeS5JN/r7l+T5MkkL3W3K3vW3ZlkOsmxJLcPum9J0uIsxZn+fcDRnvs7gENVtQE41N0nyUZgErgR2Aw8mGTFEuxfknSRBop+kjXAHcDDPcNbgL3d8l7gzp7xfVV1tqqOA9PALYPsX5K0OIOe6X8T+Crwh56x66vqNEB3e103vho42bPeTDcmSRqSvqOf5NPAmap65mI3WWCsLvDc25JMJZmanZ3td4qSpHkGOdO/DfhMkleAfcAnk3wHeDXJKoDu9ky3/gywtmf7NcCphZ64qvZU1URVTYyPjw8wRUlSr76jX1U7q2pNVa1j7ge0P6qqu4GDwNZuta3AE93yQWAyyWVJ1gMbgKf7nrkkadHG3oXn3A3sT3IvcAK4C6CqDifZDxwBzgHbq+qNd2H/kqQLWJLoV9VPgJ90y/8LbLrAeruAXUuxT0nS4vmJXElqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqiNGXpIYYfUlqSN/RT7I2yY+THE1yOMl93fg1SZ5M8lJ3u7Jnm51JppMcS3L7UvwGJEkXb5Az/XPAV6rqw8CtwPYkG4EdwKGq2gAc6u7TPTYJ3AhsBh5MsmKQyUuSFqfv6FfV6ap6tlt+DTgKrAa2AHu71fYCd3bLW4B9VXW2qo4D08At/e5fkrR4S3JNP8k64GbgKeD6qjoNcy8MwHXdaquBkz2bzXRjCz3ftiRTSaZmZ2eXYoqSJJYg+kmuAr4LfKmqfvt2qy4wVgutWFV7qmqiqibGx8cHnaIkqTNQ9JO8h7ngP1pVj3fDryZZ1T2+CjjTjc8Aa3s2XwOcGmT/kqTFGeTdOwEeAY5W1Td6HjoIbO2WtwJP9IxPJrksyXpgA/B0v/uXJC3e2ADb3gbcA7yQ5Plu7GvAbmB/knuBE8BdAFV1OMl+4Ahz7/zZXlVvDLB/SdIi9R39qvovFr5OD7DpAtvsAnb1u09J0mD8RK4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNcToS1JDjL4kNWRs2DtMshl4AFgBPFxVu4ex33U7vv/H5Vd23zGMXUrSogyjU0ONfpIVwL8AfwvMAD9LcrCqjgxzHr4ASGrVsM/0bwGmq+plgCT7gC3AUKPfq/cF4FLR+0J0ofkt9sXKFzq9yT8Lw3OhYz3K/wapquHtLPl7YHNV/WN3/x7gr6vq8/PW2wZs6+7eABzrc5fXAr/qc9vlyONxnsfirTwe5y2XY/GXVTU+f3DYZ/pZYOxPXnWqag+wZ+CdJVNVNTHo8ywXHo/zPBZv5fE4b7kfi2G/e2cGWNtzfw1washzkKRmDTv6PwM2JFmf5L3AJHBwyHOQpGYN9fJOVZ1L8nngP5l7y+a3qurwu7jLgS8RLTMej/M8Fm/l8ThvWR+Lof4gV5I0Wn4iV5IaYvQlqSHLMvpJNic5lmQ6yY5Rz2eUkqxN8uMkR5McTnLfqOc0aklWJHkuyfdGPZdRS/L+JAeS/Lz7M/KxUc9plJJ8uft78mKSx5K8b9RzWmrLLvo9X/Xwd8BG4LNJNo52ViN1DvhKVX0YuBXY3vjxALgPODrqSVwiHgB+UFUfAj5Cw8clyWrgi8BEVd3E3JtNJkc7q6W37KJPz1c9VNXrwJtf9dCkqjpdVc92y68x95d69WhnNTpJ1gB3AA+Pei6jluRq4BPAIwBV9XpV/Xqkkxq9MeDyJGPAFSzDzxEtx+ivBk723J+h4cj1SrIOuBl4asRTGaVvAl8F/jDieVwKPgjMAt/uLnc9nOTKUU9qVKrqF8DXgRPAaeA3VfXD0c5q6S3H6F/UVz20JslVwHeBL1XVb0c9n1FI8mngTFU9M+q5XCLGgI8CD1XVzcDvgGZ/BpZkJXNXBdYDHwCuTHL3aGe19JZj9P2qh3mSvIe54D9aVY+Pej4jdBvwmSSvMHfZ75NJvjPaKY3UDDBTVW/+y+8Acy8CrfoUcLyqZqvq98DjwMdHPKcltxyj71c99EgS5q7ZHq2qb4x6PqNUVTurak1VrWPuz8WPqmrZncldrKr6JXAyyQ3d0CZG+DXnl4ATwK1Jruj+3mxiGf5ge+j/56x32wi+6uFSdxtwD/BCkue7sa9V1b+Pbkq6hHwBeLQ7QXoZ+NyI5zMyVfVUkgPAs8y96+05luFXMvg1DJLUkOV4eUeSdAFGX5IaYvQlqSFGX5IaYvQlqSFGX5IaYvQlqSH/DzYnjXtG8K8tAAAAAElFTkSuQmCC\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "802ceeed63bb4eaab472c11c0e4fda43": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "3a60080935434b739a9cdf37d5de5a18": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_b4636f73598a446cbd9969fe82a52ed2",
                    "IPY_MODEL_bee172c217a3419f997ce85807fba041"
                ],
                "layout": "IPY_MODEL_802ceeed63bb4eaab472c11c0e4fda43"
            }
        },
        "f5128cb047004e3d83beaa2e25ea0902": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "a394629571eb48a7bb0690f3d65c9e36": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "b4636f73598a446cbd9969fe82a52ed2": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_f5128cb047004e3d83beaa2e25ea0902",
                "max": 20,
                "min": 1,
                "style": "IPY_MODEL_a394629571eb48a7bb0690f3d65c9e36",
                "value": 20
            }
        },
        "55a7bc25ca2e41afa1aea4517fd93f9b": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "bee172c217a3419f997ce85807fba041": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_55a7bc25ca2e41afa1aea4517fd93f9b",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAlwAAAI/CAYAAACifAdEAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAAYZElEQVR4nO3df4xld3nf8c9TL9AASbHrhTq2VZvKooVIKXTlktAiVKe1AxGmlVw5apJV68pKZVKoWrWmkUL+seT0R9SkClQu0DgtwnEJKasEWiw3UVSpQBYwP2zjeBM7eOONvUlUoK3kxM7TP+a4Oxpm7GXuPjM7M6+XdHXv/Z5z7vnO2TPjt++9c6e6OwAAzPkTuz0BAID9TnABAAwTXAAAwwQXAMAwwQUAMExwAQAMO7TbE3g+F198cV9xxRW7PQ0AgOf1mc985ve6+/DG8fM+uK644oocP358t6cBAPC8quq3Nxv3kiIAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDDu32BM4HV9z6y98w9ujtb9mFmQAA+5FnuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGPW9wVdUHqurJqvrSurGLquqeqnp4ub5w3bJ3VdWJqnqoqq5dN/6XquqLy7Kfrqo6918OAMD552ye4frZJNdtGLs1yb3dfVWSe5f7qapXJ7kxyWuWbd5TVRcs27w3yc1JrlouGx8TAGBfet7g6u5fS/IHG4avT3LncvvOJG9bN35Xdz/V3Y8kOZHk6qq6JMm3dff/7O5O8nPrtgEA2Ne2+x6uV3T3qSRZrl++jF+a5LF1651cxi5dbm8cBwDY9871m+Y3e19WP8f45g9SdXNVHa+q46dPnz5nkwMA2A3bDa4nlpcJs1w/uYyfTHL5uvUuS/L4Mn7ZJuOb6u47uvtIdx85fPjwNqcIAHB+2G5wHUtydLl9NMlH143fWFUvqqors/bm+E8vLzt+vapev/x24g+t2wYAYF879HwrVNWHkrwpycVVdTLJu5PcnuTuqropyVeS3JAk3X1/Vd2d5IEkTye5pbufWR7qH2TtNx6/JcnHlwsAwL73vMHV3d+/xaJrtlj/tiS3bTJ+PMl3fFOzAwDYB3zSPADAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwLCVgquq/lFV3V9VX6qqD1XVn6yqi6rqnqp6eLm+cN3676qqE1X1UFVdu/r0AQDOf9sOrqq6NMk/THKku78jyQVJbkxya5J7u/uqJPcu91NVr16WvybJdUneU1UXrDZ9AIDz36ovKR5K8i1VdSjJi5M8nuT6JHcuy+9M8rbl9vVJ7urup7r7kSQnkly94v4BAM572w6u7v6dJP8qyVeSnEry1e7+RJJXdPepZZ1TSV6+bHJpksfWPcTJZQwAYF9b5SXFC7P2rNWVSb49yUuq6geea5NNxnqLx765qo5X1fHTp09vd4oAAOeFVV5S/J4kj3T36e7+oyQfSfLdSZ6oqkuSZLl+cln/ZJLL121/WdZegvwG3X1Hdx/p7iOHDx9eYYoAALtvleD6SpLXV9WLq6qSXJPkwSTHkhxd1jma5KPL7WNJbqyqF1XVlUmuSvLpFfYPALAnHNruht39qar6cJLPJnk6yeeS3JHkpUnurqqbshZlNyzr319Vdyd5YFn/lu5+ZsX5AwCc97YdXEnS3e9O8u4Nw09l7dmuzda/Lcltq+wTAGCv8UnzAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMCwlYKrql5WVR+uqi9X1YNV9V1VdVFV3VNVDy/XF65b/11VdaKqHqqqa1efPgDA+W/VZ7h+Ksl/7e4/n+Q7kzyY5NYk93b3VUnuXe6nql6d5MYkr0lyXZL3VNUFK+4fAOC8t+3gqqpvS/LGJO9Pku7+w+7+X0muT3LnstqdSd623L4+yV3d/VR3P5LkRJKrt7t/AIC9YpVnuF6Z5HSS/1BVn6uq91XVS5K8ortPJcly/fJl/UuTPLZu+5PLGADAvrZKcB1K8rok7+3u1yb5P1lePtxCbTLWm65YdXNVHa+q46dPn15higAAu2+V4DqZ5GR3f2q5/+GsBdgTVXVJkizXT65b//J121+W5PHNHri77+juI9195PDhwytMEQBg9207uLr7d5M8VlWvWoauSfJAkmNJji5jR5N8dLl9LMmNVfWiqroyyVVJPr3d/QMA7BWHVtz+R5J8sKpemOS3kvzdrEXc3VV1U5KvJLkhSbr7/qq6O2tR9nSSW7r7mRX3DwBw3lspuLr7viRHNll0zRbr35bktlX2CQCw1/ikeQCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGErB1dVXVBVn6uqX1ruX1RV91TVw8v1hevWfVdVnaiqh6rq2lX3DQCwF5yLZ7jekeTBdfdvTXJvd1+V5N7lfqrq1UluTPKaJNcleU9VXXAO9g8AcF5bKbiq6rIkb0nyvnXD1ye5c7l9Z5K3rRu/q7uf6u5HkpxIcvUq+wcA2AtWfYbr3yT5p0n+eN3YK7r7VJIs1y9fxi9N8ti69U4uYwAA+9q2g6uqvi/Jk939mbPdZJOx3uKxb66q41V1/PTp09udIgDAeWGVZ7jekOStVfVokruS/LWq+k9JnqiqS5JkuX5yWf9kksvXbX9Zksc3e+DuvqO7j3T3kcOHD68wRQCA3bft4Orud3X3Zd19RdbeDP/fu/sHkhxLcnRZ7WiSjy63jyW5sapeVFVXJrkqyae3PXMAgD3i0MBj3p7k7qq6KclXktyQJN19f1XdneSBJE8nuaW7nxnYPwDAeeWcBFd3/2qSX11u/36Sa7ZY77Ykt52LfQIA7BU+aR4AYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGDYtoOrqi6vql+pqger6v6qescyflFV3VNVDy/XF67b5l1VdaKqHqqqa8/FFwAAcL5b5Rmup5P84+7+C0len+SWqnp1kluT3NvdVyW5d7mfZdmNSV6T5Lok76mqC1aZPADAXrDt4OruU9392eX215M8mOTSJNcnuXNZ7c4kb1tuX5/kru5+qrsfSXIiydXb3T8AwF5xTt7DVVVXJHltkk8leUV3n0rWoizJy5fVLk3y2LrNTi5jAAD72srBVVUvTfILSd7Z3V97rlU3GestHvPmqjpeVcdPnz696hQBAHbVSsFVVS/IWmx9sLs/sgw/UVWXLMsvSfLkMn4yyeXrNr8syeObPW5339HdR7r7yOHDh1eZIgDArlvltxQryfuTPNjdP7lu0bEkR5fbR5N8dN34jVX1oqq6MslVST693f0DAOwVh1bY9g1JfjDJF6vqvmXsnye5PcndVXVTkq8kuSFJuvv+qro7yQNZ+w3HW7r7mRX2DwCwJ2w7uLr7f2Tz92UlyTVbbHNbktu2u08AgL3IJ80DAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMOzQbk8AAOBcuuLWX/6GsUdvf8suzOQMwbWF8/EfCwDYm3Y8uKrquiQ/leSCJO/r7tt3eg7bJcIAgO3Y0eCqqguS/EySv57kZJJfr6pj3f3ATs7jXJqIMGEHAGdns/9mno92+hmuq5Oc6O7fSpKquivJ9Un2bHBt5psJpr1yogAA27fTwXVpksfW3T+Z5C/v8Bx2xaphJcxWt1n07tXjerZfy7lebxWr7GPV+e3Evs+3f5OzfVb8XO97LxyvvTDHs9n2bO3Vn3P7TXX3zu2s6oYk13b331/u/2CSq7v7Rzasd3OSm5e7r0ry0PDULk7ye8P72CscizMcizWOwxmOxRmOxRmOxRrHYc2f7e7DGwd3+hmuk0kuX3f/siSPb1ypu+9IcsdOTaqqjnf3kZ3a3/nMsTjDsVjjOJzhWJzhWJzhWKxxHJ7bTn/w6a8nuaqqrqyqFya5McmxHZ4DAMCO2tFnuLr76ap6e5L/lrWPhfhAd9+/k3MAANhpO/45XN39sSQf2+n9Po8de/lyD3AsznAs1jgOZzgWZzgWZzgWaxyH57Cjb5oHADiI/PFqAIBhByq4quq6qnqoqk5U1a2bLK+q+ull+Req6nW7Mc9pVXV5Vf1KVT1YVfdX1Ts2WedNVfXVqrpvufzYbsx1WlU9WlVfXL7G45ssPyjnxKvW/VvfV1Vfq6p3blhn354TVfWBqnqyqr60buyiqrqnqh5eri/cYtvn/Lmy12xxLP5lVX15+R74xap62RbbPuf3016yxXH48ar6nXXfA2/eYtuDcE78/Lrj8GhV3bfFtvvmnFhZdx+IS9bepP+bSV6Z5IVJPp/k1RvWeXOSjyepJK9P8qndnvfQsbgkyeuW29+a5Dc2ORZvSvJLuz3XHTgWjya5+DmWH4hzYsPXfEGS383aZ8kciHMiyRuTvC7Jl9aN/Yskty63b03yE1scq+f8ubLXLlsci7+R5NBy+yc2OxbLsuf8ftpLly2Ow48n+SfPs92BOCc2LP/XSX5sv58Tq14O0jNc///PCnX3HyZ59s8KrXd9kp/rNZ9M8rKqumSnJzqtu09192eX219P8mDW/goA3+hAnBMbXJPkN7v7t3d7Ijulu38tyR9sGL4+yZ3L7TuTvG2TTc/m58qestmx6O5PdPfTy91PZu0zFPe1Lc6Js3EgzolnVVUl+dtJPrSjk9qDDlJwbfZnhTZGxtmss69U1RVJXpvkU5ss/q6q+nxVfbyqXrOzM9sxneQTVfWZ5S8cbHTgzomsfT7eVj88D8I58axXdPepZO1/UpK8fJN1DuL58fey9qzvZp7v+2k/ePvy0uoHtniZ+aCdE381yRPd/fAWyw/COXFWDlJw1SZjG39F82zW2Teq6qVJfiHJO7v7axsWfzZrLyl9Z5J/m+S/7PD0dsobuvt1Sb43yS1V9cYNyw/aOfHCJG9N8p83WXxQzolvxkE7P340ydNJPrjFKs/3/bTXvTfJn0vyF5OcytpLaRsdqHMiyffnuZ/d2u/nxFk7SMF1Nn9W6Kz+9NB+UFUvyFpsfbC7P7JxeXd/rbv/93L7Y0leUFUX7/A0x3X348v1k0l+MWsvB6x3YM6Jxfcm+Wx3P7FxwUE5J9Z54tmXj5frJzdZ58CcH1V1NMn3Jfk7vbw5Z6Oz+H7a07r7ie5+prv/OMm/z+Zf30E6Jw4l+VtJfn6rdfb7OfHNOEjBdTZ/VuhYkh9afjPt9Um++uxLCvvJ8pr7+5M82N0/ucU6f2ZZL1V1ddbOld/fuVnOq6qXVNW3Pns7a28M/tKG1Q7EObHOlv+3ehDOiQ2OJTm63D6a5KObrHMg/lxZVV2X5J8leWt3/98t1jmb76c9bcP7N/9mNv/6DsQ5sfieJF/u7pObLTwI58Q3Zbfftb+Tl6z9xtlvZO03SH50GfvhJD+83K4kP7Ms/2KSI7s956Hj8Fey9hT3F5Lct1zevOFYvD3J/Vn7DZtPJvnu3Z73wHF45fL1fX75Wg/sObF8rS/OWkD9qXVjB+KcyFpknkryR1l7huKmJH86yb1JHl6uL1rW/fYkH1u37Tf8XNnLly2OxYmsvS/p2Z8X/27jsdjq+2mvXrY4Dv9x+TnwhaxF1CUH9ZxYxn/22Z8P69bdt+fEqhefNA8AMOwgvaQIALArBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAM+3/7mgBtLZmy4QAAAABJRU5ErkJggg==\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "02dd6f7a5c90404abc73175482804ee4": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "f503eeb083af4e128ecb13dde2c6b10b": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_c3cbf9900b67418fa19fcb6b3678cdca",
                    "IPY_MODEL_16d25586a2494073a5b0165545e44e35"
                ],
                "layout": "IPY_MODEL_02dd6f7a5c90404abc73175482804ee4"
            }
        },
        "1165a88f2e194b6ebd520406913c2e9b": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "aea3531058454f5aa8b7e9a8a9a1ef67": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "c3cbf9900b67418fa19fcb6b3678cdca": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_1165a88f2e194b6ebd520406913c2e9b",
                "max": 50,
                "min": 1,
                "style": "IPY_MODEL_aea3531058454f5aa8b7e9a8a9a1ef67",
                "value": 25
            }
        },
        "7309d937050e4e72aff18fa18b6d8209": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "16d25586a2494073a5b0165545e44e35": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_7309d937050e4e72aff18fa18b6d8209",
                "outputs": [
                    {
                        "output_type": "error",
                        "ename": "FileNotFoundError",
                        "evalue": "[Errno 2] No such file or directory: 'pickles/25'",
                        "traceback": [
                            "\u001b[1;31m---------------------------------------------------------------------------\u001b[0m",
                            "\u001b[1;31mFileNotFoundError\u001b[0m                         Traceback (most recent call last)",
                            "\u001b[1;32mC:\\ProgramData\\Anaconda3\\lib\\site-packages\\ipywidgets\\widgets\\interaction.py\u001b[0m in \u001b[0;36mupdate\u001b[1;34m(self, *args)\u001b[0m\n\u001b[0;32m    254\u001b[0m                     \u001b[0mvalue\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mwidget\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mget_interact_value\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m    255\u001b[0m                     \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[1;33m[\u001b[0m\u001b[0mwidget\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0m_kwarg\u001b[0m\u001b[1;33m]\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mvalue\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m--> 256\u001b[1;33m                 \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mresult\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mf\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;33m**\u001b[0m\u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m    257\u001b[0m                 \u001b[0mshow_inline_matplotlib_plots\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m    258\u001b[0m                 \u001b[1;32mif\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mauto_display\u001b[0m \u001b[1;32mand\u001b[0m \u001b[0mself\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mresult\u001b[0m \u001b[1;32mis\u001b[0m \u001b[1;32mnot\u001b[0m \u001b[1;32mNone\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n",
                            "\u001b[1;32m<ipython-input-50-217a6eb13b79>\u001b[0m in \u001b[0;36mgraph_error\u001b[1;34m(Z)\u001b[0m\n\u001b[0;32m      1\u001b[0m \u001b[1;32mdef\u001b[0m \u001b[0mgraph_error\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mZ\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m----> 2\u001b[1;33m     \u001b[0mf\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mgetFF\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mZ\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m      3\u001b[0m \u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m      4\u001b[0m \u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m      5\u001b[0m     \u001b[0mFFdata\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mdb\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mEPDL\u001b[0m\u001b[1;33m[\u001b[0m\u001b[0mZ\u001b[0m\u001b[1;33m-\u001b[0m\u001b[1;36m1\u001b[0m\u001b[1;33m]\u001b[0m\u001b[1;33m[\u001b[0m\u001b[1;33m(\u001b[0m\u001b[1;36m7\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;36m93\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;36m0\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;36m0\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;36m0\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;36m941\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m]\u001b[0m\u001b[1;33m[\u001b[0m\u001b[1;36m1\u001b[0m\u001b[1;33m]\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n",
                            "\u001b[1;32m<ipython-input-21-7a5ea8fd40bb>\u001b[0m in \u001b[0;36mgetFF\u001b[1;34m(Z)\u001b[0m\n\u001b[0;32m     19\u001b[0m     \u001b[1;32mimport\u001b[0m \u001b[0mdill\u001b[0m \u001b[1;32mas\u001b[0m \u001b[0mpickle\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m     20\u001b[0m     \u001b[0mpath\u001b[0m \u001b[1;33m=\u001b[0m \u001b[1;34m\"pickles/\"\u001b[0m \u001b[1;33m+\u001b[0m \u001b[0mstr\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mZ\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[1;32m---> 21\u001b[1;33m     \u001b[1;32mwith\u001b[0m \u001b[0mopen\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mpath\u001b[0m\u001b[1;33m,\u001b[0m \u001b[1;34m'rb'\u001b[0m\u001b[1;33m)\u001b[0m \u001b[1;32mas\u001b[0m \u001b[0mfile\u001b[0m\u001b[1;33m:\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0m\u001b[0;32m     22\u001b[0m         \u001b[0mparam\u001b[0m \u001b[1;33m=\u001b[0m \u001b[0mpickle\u001b[0m\u001b[1;33m.\u001b[0m\u001b[0mload\u001b[0m\u001b[1;33m(\u001b[0m\u001b[0mfile\u001b[0m\u001b[1;33m)\u001b[0m\u001b[1;33m\u001b[0m\u001b[1;33m\u001b[0m\u001b[0m\n\u001b[0;32m     23\u001b[0m \u001b[1;33m\u001b[0m\u001b[0m\n",
                            "\u001b[1;31mFileNotFoundError\u001b[0m: [Errno 2] No such file or directory: 'pickles/25'"
                        ]
                    }
                ]
            }
        },
        "8fc25f25640349b2a3f9938eb01c5192": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "54f1ef4a0f10405ea1bed25832aa9ed9": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_c6f08875243c4129ae5e56a8c5890bb3",
                    "IPY_MODEL_e9513e8104e849d295b8972b4b307b13"
                ],
                "layout": "IPY_MODEL_8fc25f25640349b2a3f9938eb01c5192"
            }
        },
        "5e6b7be8607d456cbf82a6e1e2250af7": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "ddd2420598cb4b748664e892ba4085b1": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "c6f08875243c4129ae5e56a8c5890bb3": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_5e6b7be8607d456cbf82a6e1e2250af7",
                "max": 26,
                "min": 1,
                "style": "IPY_MODEL_ddd2420598cb4b748664e892ba4085b1",
                "value": 21
            }
        },
        "941fbd0a533b48378cb33190a5a5ee37": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "e9513e8104e849d295b8972b4b307b13": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_941fbd0a533b48378cb33190a5a5ee37",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAlwAAAI/CAYAAACifAdEAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAAYqElEQVR4nO3dbYyl9Xnf8d9V1nZjO6mhLJQAKrhCbiFSandFnbi1rJIWYkeGVqIiapJVS4VS4dSuWrXQSHHeIJE+RE2q2BW1nWxay4Q6TkGJ3RrRRFGl2s7axg+ACZtAYAOBTaLabiuRQK6+mJt2NJ5hNzN7zePnI43OOffDuf9/7jk7X845c6a6OwAAzPkTOz0AAID9TnABAAwTXAAAwwQXAMAwwQUAMExwAQAMO7TTAzid888/vy+77LKdHgYAwGl99rOf/d3uPrx2+a4PrssuuyzHjx/f6WEAAJxWVf3Wesu9pAgAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMCwQzs9gN3gstt+6RuWPXHnO3ZgJADAfuQZLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABh22uCqqg9V1XNV9eVVy86rqvur6rHl8txV626vqhNV9WhVXbtq+V+qqi8t636yqursTwcAYPc5k2e4fibJdWuW3Zbkge6+IskDy+1U1ZVJbkpy1bLP+6rqnGWf9ye5JckVy9fa+wQA2JdOG1zd/atJfn/N4uuTHFuuH0tyw6rld3f38939eJITSa6uqouSfEt3/4/u7iQ/u2ofAIB9bbPv4bqwu59JkuXygmX5xUmeWrXdyWXZxcv1tcsBAPa9s/2m+fXel9Uvs3z9O6m6paqOV9XxU6dOnbXBAQDshM0G17PLy4RZLp9blp9Mcumq7S5J8vSy/JJ1lq+ru+/q7iPdfeTw4cObHCIAwO6w2eC6L8nR5frRJPeuWn5TVb2qqi7PypvjP7O87Pj1qnrz8tuJP7BqHwCAfe3Q6Taoqo8keVuS86vqZJL3JrkzyT1VdXOSJ5PcmCTd/VBV3ZPk4SQvJLm1u19c7uofZOU3Hr8pySeWLwCAfe+0wdXd37vBqms22P6OJHess/x4km/7Y40OAGAf8EnzAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMCwLQVXVf2jqnqoqr5cVR+pqj9ZVedV1f1V9dhyee6q7W+vqhNV9WhVXbv14QMA7H6bDq6qujjJP0xypLu/Lck5SW5KcluSB7r7iiQPLLdTVVcu669Kcl2S91XVOVsbPgDA7rfVlxQPJfmmqjqU5NVJnk5yfZJjy/pjSW5Yrl+f5O7ufr67H09yIsnVWzw+AMCut+ng6u7fTvKvkjyZ5JkkX+3uTya5sLufWbZ5JskFyy4XJ3lq1V2cXJYBAOxrW3lJ8dysPGt1eZJvTfKaqvq+l9tlnWW9wX3fUlXHq+r4qVOnNjtEAIBdYSsvKX5Xkse7+1R3/2GSjyX5ziTPVtVFSbJcPrdsfzLJpav2vyQrL0F+g+6+q7uPdPeRw4cPb2GIAAA7byvB9WSSN1fVq6uqklyT5JEk9yU5umxzNMm9y/X7ktxUVa+qqsuTXJHkM1s4PgDAnnBoszt296er6qNJPpfkhSSfT3JXktcmuaeqbs5KlN24bP9QVd2T5OFl+1u7+8Utjh8AYNfbdHAlSXe/N8l71yx+PivPdq23/R1J7tjKMQEA9hqfNA8AMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBsS8FVVa+rqo9W1Veq6pGq+o6qOq+q7q+qx5bLc1dtf3tVnaiqR6vq2q0PHwBg99vqM1w/keS/dPefT/LtSR5JcluSB7r7iiQPLLdTVVcmuSnJVUmuS/K+qjpni8cHANj1Nh1cVfUtSd6a5INJ0t1/0N3/M8n1SY4tmx1LcsNy/fokd3f38939eJITSa7e7PEBAPaKrTzD9fokp5L8dFV9vqo+UFWvSXJhdz+TJMvlBcv2Fyd5atX+J5dlAAD72laC61CSNyV5f3e/Mcn/zvLy4QZqnWW97oZVt1TV8ao6furUqS0MEQBg520luE4mOdndn15ufzQrAfZsVV2UJMvlc6u2v3TV/pckeXq9O+7uu7r7SHcfOXz48BaGCACw8zYdXN39O0meqqo3LIuuSfJwkvuSHF2WHU1y73L9viQ3VdWrquryJFck+cxmjw8AsFcc2uL+P5Tkw1X1yiS/meTvZiXi7qmqm5M8meTGJOnuh6rqnqxE2QtJbu3uF7d4fACAXW9LwdXdDyY5ss6qazbY/o4kd2zlmAAAe41PmgcAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhm05uKrqnKr6fFX94nL7vKq6v6oeWy7PXbXt7VV1oqoeraprt3psAIC94Gw8w/XuJI+sun1bkge6+4okDyy3U1VXJrkpyVVJrkvyvqo65ywcHwBgV9tScFXVJUnekeQDqxZfn+TYcv1YkhtWLb+7u5/v7seTnEhy9VaODwCwF2z1Ga5/k+SfJvmjVcsu7O5nkmS5vGBZfnGSp1Ztd3JZBgCwr206uKrqe5I8192fPdNd1lnWG9z3LVV1vKqOnzp1arNDBADYFbbyDNdbkryzqp5IcneSv1ZV/zHJs1V1UZIsl88t259Mcumq/S9J8vR6d9zdd3X3ke4+cvjw4S0MEQBg5206uLr79u6+pLsvy8qb4f9bd39fkvuSHF02O5rk3uX6fUluqqpXVdXlSa5I8plNjxwAYI84NHCfdya5p6puTvJkkhuTpLsfqqp7kjyc5IUkt3b3iwPHBwDYVc5KcHX3ryT5leX67yW5ZoPt7khyx9k4JgDAXuGT5gEAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYZsOrqq6tKp+uaoeqaqHqurdy/Lzqur+qnpsuTx31T63V9WJqnq0qq49GxMAANjttvIM1wtJ/nF3/4Ukb05ya1VdmeS2JA909xVJHlhuZ1l3U5KrklyX5H1Vdc5WBg8AsBdsOri6+5nu/txy/etJHklycZLrkxxbNjuW5Ibl+vVJ7u7u57v78SQnkly92eMDAOwVZ+U9XFV1WZI3Jvl0kgu7+5lkJcqSXLBsdnGSp1btdnJZBgCwr205uKrqtUl+Psl7uvtrL7fpOst6g/u8paqOV9XxU6dObXWIAAA7akvBVVWvyEpsfbi7P7YsfraqLlrWX5TkuWX5ySSXrtr9kiRPr3e/3X1Xdx/p7iOHDx/eyhABAHbcVn5LsZJ8MMkj3f3jq1bdl+Tocv1okntXLb+pql5VVZcnuSLJZzZ7fACAveLQFvZ9S5LvT/KlqnpwWfbPk9yZ5J6qujnJk0luTJLufqiq7knycFZ+w/HW7n5xC8cHANgTNh1c3f3fs/77spLkmg32uSPJHZs9JgDAXuST5gEAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYNihnR7AbnXZbb/0DcueuPMdOzASAGCv8wwXAMAwz3ABAPvKbnyVatuDq6quS/ITSc5J8oHuvnO7x7BZu/EEAgC737YGV1Wdk+Snkvz1JCeT/FpV3dfdD2/nOM4mEQYAO2e9n8O70XY/w3V1khPd/ZtJUlV3J7k+yZ4NrvVMnHwRB8BBt1fiaj3bHVwXJ3lq1e2TSf7yNo9hT9rL32Sns1FMnu05r3ecnfrvOjGWs32fu+n+/jj7btdxNrvvmf7P08T3w5keZytz2e3b7YUx7qbHLWdPdff2HazqxiTXdvffX25/f5Kru/uH1mx3S5JblptvSPLo8NDOT/K7w8fYrQ7y3JODPf+DPPfkYM//IM89OdjzN/d5f7a7D69duN3PcJ1Mcumq25ckeXrtRt19V5K7tmtQVXW8u49s1/F2k4M89+Rgz/8gzz052PM/yHNPDvb8zX3n5r7dn8P1a0muqKrLq+qVSW5Kct82jwEAYFtt6zNc3f1CVb0ryX/NysdCfKi7H9rOMQAAbLdt/xyu7v54ko9v93FPY9tevtyFDvLck4M9/4M89+Rgz/8gzz052PM39x2yrW+aBwA4iPwtRQCAYQcquKrquqp6tKpOVNVt66yvqvrJZf0Xq+pNOzHOs62qLq2qX66qR6rqoap69zrbvK2qvlpVDy5fP7ITY51SVU9U1ZeWuR1fZ/1+PfdvWHVOH6yqr1XVe9Zss6/OfVV9qKqeq6ovr1p2XlXdX1WPLZfnbrDvy/4bsdttMPd/WVVfWb6vf6GqXrfBvi/7GNntNpj7j1bVb6/63n77Bvvu6fOebDj/n1s19yeq6sEN9t3r537dn3G77nHf3QfiKytv0v+NJK9P8sokX0hy5Zpt3p7kE0kqyZuTfHqnx32W5n5Rkjct1785ya+vM/e3JfnFnR7r4H+DJ5Kc/zLr9+W5XzPHc5L8TlY+I2bfnvskb03ypiRfXrXsXyS5bbl+W5If2+C/z8v+G7HbvzaY+99Icmi5/mPrzX1Z97KPkd3+tcHcfzTJPznNfnv+vG80/zXr/3WSH9mn537dn3G77XF/kJ7h+n9/Vqi7/yDJS39WaLXrk/xsr/hUktdV1UXbPdCzrbuf6e7PLde/nuSRrHzqP//fvjz3a1yT5De6+7d2eiCTuvtXk/z+msXXJzm2XD+W5IZ1dj2TfyN2tfXm3t2f7O4XlpufysrnH+47G5z3M7Hnz3vy8vOvqkryt5N8ZFsHtU1e5mfcrnrcH6TgWu/PCq2NjjPZZk+rqsuSvDHJp9dZ/R1V9YWq+kRVXbW9IxvXST5ZVZ+tlb9ksNa+P/dZ+dy7jf7B3c/nPkku7O5nkpV/nJNcsM42B+F74O9l5Znc9ZzuMbJXvWt5OfVDG7ykdBDO+19N8mx3P7bB+n1z7tf8jNtVj/uDFFy1zrK1v6J5JtvsWVX12iQ/n+Q93f21Nas/l5WXmr49yb9N8p+3eXjT3tLdb0ry3Uluraq3rlm/38/9K5O8M8l/Wmf1fj/3Z2q/fw/8cJIXknx4g01O9xjZi96f5M8l+YtJnsnKy2pr7evzvvjevPyzW/vi3J/mZ9yGu62zbOT8H6TgOpM/K3RGf3poL6qqV2TlG/HD3f2xteu7+2vd/b+W6x9P8oqqOn+bhzmmu59eLp9L8gtZeRp5tX177hffneRz3f3s2hX7/dwvnn3pJeLl8rl1ttm33wNVdTTJ9yT5O728cWWtM3iM7Dnd/Wx3v9jdf5Tk32f9Oe3b854kVXUoyd9K8nMbbbMfzv0GP+N21eP+IAXXmfxZofuS/MDyG2tvTvLVl56O3MuW1+8/mOSR7v7xDbb5M8t2qaqrs/K98XvbN8o5VfWaqvrml65n5U3EX16z2b4896ts+H+4+/ncr3JfkqPL9aNJ7l1nm335p8eq6rok/yzJO7v7/2ywzZk8RvacNe/D/JtZf0778ryv8l1JvtLdJ9dbuR/O/cv8jNtdj/ud+q2CnfjKym+i/XpWfiPhh5dlP5jkB5frleSnlvVfSnJkp8d8lub9V7LyFOkXkzy4fL19zdzfleShrPyGxqeSfOdOj/sszv/1y7y+sMzxwJz7ZW6vzkpA/alVy/btuc9KWD6T5A+z8n+vNyf500keSPLYcnnesu23Jvn4qn2/4d+IvfS1wdxPZOU9Ki899v/d2rlv9BjZS18bzP0/LI/nL2blh+hF+/G8bzT/ZfnPvPRYX7Xtfjv3G/2M21WPe580DwAw7CC9pAgAsCMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAz7v5tB+AnIfYf5AAAAAElFTkSuQmCC\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "3d0a1fc2a81545819a98d2dfd4bd3923": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "0800bcbb34114edaa132030aa4ccda41": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_e08600bfe250430da4a3c41c49b70477",
                    "IPY_MODEL_f73f2f930b7e44b0ab92e0c840b5d534"
                ],
                "layout": "IPY_MODEL_3d0a1fc2a81545819a98d2dfd4bd3923"
            }
        },
        "db3ce652925f489c820c72ce9ea8e61b": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "33a26bb9c9db4916827f20cd2c1aab17": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "e08600bfe250430da4a3c41c49b70477": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_db3ce652925f489c820c72ce9ea8e61b",
                "max": 21,
                "min": 1,
                "style": "IPY_MODEL_33a26bb9c9db4916827f20cd2c1aab17",
                "value": 21
            }
        },
        "293db04304914d47a733769cf8506d08": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "f73f2f930b7e44b0ab92e0c840b5d534": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_293db04304914d47a733769cf8506d08",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAlwAAAI/CAYAAACifAdEAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAAYqElEQVR4nO3dbYyl9Xnf8d9V1nZjO6mhLJQAKrhCbiFSandFnbi1rJIWYkeGVqIiapJVS4VS4dSuWrXQSHHeIJE+RE2q2BW1nWxay4Q6TkGJ3RrRRFGl2s7axg+ACZtAYAOBTaLabiuRQK6+mJt2NJ5hNzN7zePnI43OOffDuf9/7jk7X845c6a6OwAAzPkTOz0AAID9TnABAAwTXAAAwwQXAMAwwQUAMExwAQAMO7TTAzid888/vy+77LKdHgYAwGl99rOf/d3uPrx2+a4PrssuuyzHjx/f6WEAAJxWVf3Wesu9pAgAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMCwQzs9gN3gstt+6RuWPXHnO3ZgJADAfuQZLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABh22uCqqg9V1XNV9eVVy86rqvur6rHl8txV626vqhNV9WhVXbtq+V+qqi8t636yqursTwcAYPc5k2e4fibJdWuW3Zbkge6+IskDy+1U1ZVJbkpy1bLP+6rqnGWf9ye5JckVy9fa+wQA2JdOG1zd/atJfn/N4uuTHFuuH0tyw6rld3f38939eJITSa6uqouSfEt3/4/u7iQ/u2ofAIB9bbPv4bqwu59JkuXygmX5xUmeWrXdyWXZxcv1tcsBAPa9s/2m+fXel9Uvs3z9O6m6paqOV9XxU6dOnbXBAQDshM0G17PLy4RZLp9blp9Mcumq7S5J8vSy/JJ1lq+ru+/q7iPdfeTw4cObHCIAwO6w2eC6L8nR5frRJPeuWn5TVb2qqi7PypvjP7O87Pj1qnrz8tuJP7BqHwCAfe3Q6Taoqo8keVuS86vqZJL3JrkzyT1VdXOSJ5PcmCTd/VBV3ZPk4SQvJLm1u19c7uofZOU3Hr8pySeWLwCAfe+0wdXd37vBqms22P6OJHess/x4km/7Y40OAGAf8EnzAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMCwLQVXVf2jqnqoqr5cVR+pqj9ZVedV1f1V9dhyee6q7W+vqhNV9WhVXbv14QMA7H6bDq6qujjJP0xypLu/Lck5SW5KcluSB7r7iiQPLLdTVVcu669Kcl2S91XVOVsbPgDA7rfVlxQPJfmmqjqU5NVJnk5yfZJjy/pjSW5Yrl+f5O7ufr67H09yIsnVWzw+AMCut+ng6u7fTvKvkjyZ5JkkX+3uTya5sLufWbZ5JskFyy4XJ3lq1V2cXJYBAOxrW3lJ8dysPGt1eZJvTfKaqvq+l9tlnWW9wX3fUlXHq+r4qVOnNjtEAIBdYSsvKX5Xkse7+1R3/2GSjyX5ziTPVtVFSbJcPrdsfzLJpav2vyQrL0F+g+6+q7uPdPeRw4cPb2GIAAA7byvB9WSSN1fVq6uqklyT5JEk9yU5umxzNMm9y/X7ktxUVa+qqsuTXJHkM1s4PgDAnnBoszt296er6qNJPpfkhSSfT3JXktcmuaeqbs5KlN24bP9QVd2T5OFl+1u7+8Utjh8AYNfbdHAlSXe/N8l71yx+PivPdq23/R1J7tjKMQEA9hqfNA8AMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBsS8FVVa+rqo9W1Veq6pGq+o6qOq+q7q+qx5bLc1dtf3tVnaiqR6vq2q0PHwBg99vqM1w/keS/dPefT/LtSR5JcluSB7r7iiQPLLdTVVcmuSnJVUmuS/K+qjpni8cHANj1Nh1cVfUtSd6a5INJ0t1/0N3/M8n1SY4tmx1LcsNy/fokd3f38939eJITSa7e7PEBAPaKrTzD9fokp5L8dFV9vqo+UFWvSXJhdz+TJMvlBcv2Fyd5atX+J5dlAAD72laC61CSNyV5f3e/Mcn/zvLy4QZqnWW97oZVt1TV8ao6furUqS0MEQBg520luE4mOdndn15ufzQrAfZsVV2UJMvlc6u2v3TV/pckeXq9O+7uu7r7SHcfOXz48BaGCACw8zYdXN39O0meqqo3LIuuSfJwkvuSHF2WHU1y73L9viQ3VdWrquryJFck+cxmjw8AsFcc2uL+P5Tkw1X1yiS/meTvZiXi7qmqm5M8meTGJOnuh6rqnqxE2QtJbu3uF7d4fACAXW9LwdXdDyY5ss6qazbY/o4kd2zlmAAAe41PmgcAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhm05uKrqnKr6fFX94nL7vKq6v6oeWy7PXbXt7VV1oqoeraprt3psAIC94Gw8w/XuJI+sun1bkge6+4okDyy3U1VXJrkpyVVJrkvyvqo65ywcHwBgV9tScFXVJUnekeQDqxZfn+TYcv1YkhtWLb+7u5/v7seTnEhy9VaODwCwF2z1Ga5/k+SfJvmjVcsu7O5nkmS5vGBZfnGSp1Ztd3JZBgCwr206uKrqe5I8192fPdNd1lnWG9z3LVV1vKqOnzp1arNDBADYFbbyDNdbkryzqp5IcneSv1ZV/zHJs1V1UZIsl88t259Mcumq/S9J8vR6d9zdd3X3ke4+cvjw4S0MEQBg5206uLr79u6+pLsvy8qb4f9bd39fkvuSHF02O5rk3uX6fUluqqpXVdXlSa5I8plNjxwAYI84NHCfdya5p6puTvJkkhuTpLsfqqp7kjyc5IUkt3b3iwPHBwDYVc5KcHX3ryT5leX67yW5ZoPt7khyx9k4JgDAXuGT5gEAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYZsOrqq6tKp+uaoeqaqHqurdy/Lzqur+qnpsuTx31T63V9WJqnq0qq49GxMAANjttvIM1wtJ/nF3/4Ukb05ya1VdmeS2JA909xVJHlhuZ1l3U5KrklyX5H1Vdc5WBg8AsBdsOri6+5nu/txy/etJHklycZLrkxxbNjuW5Ibl+vVJ7u7u57v78SQnkly92eMDAOwVZ+U9XFV1WZI3Jvl0kgu7+5lkJcqSXLBsdnGSp1btdnJZBgCwr205uKrqtUl+Psl7uvtrL7fpOst6g/u8paqOV9XxU6dObXWIAAA7akvBVVWvyEpsfbi7P7YsfraqLlrWX5TkuWX5ySSXrtr9kiRPr3e/3X1Xdx/p7iOHDx/eyhABAHbcVn5LsZJ8MMkj3f3jq1bdl+Tocv1okntXLb+pql5VVZcnuSLJZzZ7fACAveLQFvZ9S5LvT/KlqnpwWfbPk9yZ5J6qujnJk0luTJLufqiq7knycFZ+w/HW7n5xC8cHANgTNh1c3f3fs/77spLkmg32uSPJHZs9JgDAXuST5gEAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYNihnR7AbnXZbb/0DcueuPMdOzASAGCv8wwXAMAwz3ABAPvKbnyVatuDq6quS/ITSc5J8oHuvnO7x7BZu/EEAgC737YGV1Wdk+Snkvz1JCeT/FpV3dfdD2/nOM4mEQYAO2e9n8O70XY/w3V1khPd/ZtJUlV3J7k+yZ4NrvVMnHwRB8BBt1fiaj3bHVwXJ3lq1e2TSf7yNo9hT9rL32Sns1FMnu05r3ecnfrvOjGWs32fu+n+/jj7btdxNrvvmf7P08T3w5keZytz2e3b7YUx7qbHLWdPdff2HazqxiTXdvffX25/f5Kru/uH1mx3S5JblptvSPLo8NDOT/K7w8fYrQ7y3JODPf+DPPfkYM//IM89OdjzN/d5f7a7D69duN3PcJ1Mcumq25ckeXrtRt19V5K7tmtQVXW8u49s1/F2k4M89+Rgz/8gzz052PM/yHNPDvb8zX3n5r7dn8P1a0muqKrLq+qVSW5Kct82jwEAYFtt6zNc3f1CVb0ryX/NysdCfKi7H9rOMQAAbLdt/xyu7v54ko9v93FPY9tevtyFDvLck4M9/4M89+Rgz/8gzz052PM39x2yrW+aBwA4iPwtRQCAYQcquKrquqp6tKpOVNVt66yvqvrJZf0Xq+pNOzHOs62qLq2qX66qR6rqoap69zrbvK2qvlpVDy5fP7ITY51SVU9U1ZeWuR1fZ/1+PfdvWHVOH6yqr1XVe9Zss6/OfVV9qKqeq6ovr1p2XlXdX1WPLZfnbrDvy/4bsdttMPd/WVVfWb6vf6GqXrfBvi/7GNntNpj7j1bVb6/63n77Bvvu6fOebDj/n1s19yeq6sEN9t3r537dn3G77nHf3QfiKytv0v+NJK9P8sokX0hy5Zpt3p7kE0kqyZuTfHqnx32W5n5Rkjct1785ya+vM/e3JfnFnR7r4H+DJ5Kc/zLr9+W5XzPHc5L8TlY+I2bfnvskb03ypiRfXrXsXyS5bbl+W5If2+C/z8v+G7HbvzaY+99Icmi5/mPrzX1Z97KPkd3+tcHcfzTJPznNfnv+vG80/zXr/3WSH9mn537dn3G77XF/kJ7h+n9/Vqi7/yDJS39WaLXrk/xsr/hUktdV1UXbPdCzrbuf6e7PLde/nuSRrHzqP//fvjz3a1yT5De6+7d2eiCTuvtXk/z+msXXJzm2XD+W5IZ1dj2TfyN2tfXm3t2f7O4XlpufysrnH+47G5z3M7Hnz3vy8vOvqkryt5N8ZFsHtU1e5mfcrnrcH6TgWu/PCq2NjjPZZk+rqsuSvDHJp9dZ/R1V9YWq+kRVXbW9IxvXST5ZVZ+tlb9ksNa+P/dZ+dy7jf7B3c/nPkku7O5nkpV/nJNcsM42B+F74O9l5Znc9ZzuMbJXvWt5OfVDG7ykdBDO+19N8mx3P7bB+n1z7tf8jNtVj/uDFFy1zrK1v6J5JtvsWVX12iQ/n+Q93f21Nas/l5WXmr49yb9N8p+3eXjT3tLdb0ry3Uluraq3rlm/38/9K5O8M8l/Wmf1fj/3Z2q/fw/8cJIXknx4g01O9xjZi96f5M8l+YtJnsnKy2pr7evzvvjevPyzW/vi3J/mZ9yGu62zbOT8H6TgOpM/K3RGf3poL6qqV2TlG/HD3f2xteu7+2vd/b+W6x9P8oqqOn+bhzmmu59eLp9L8gtZeRp5tX177hffneRz3f3s2hX7/dwvnn3pJeLl8rl1ttm33wNVdTTJ9yT5O728cWWtM3iM7Dnd/Wx3v9jdf5Tk32f9Oe3b854kVXUoyd9K8nMbbbMfzv0GP+N21eP+IAXXmfxZofuS/MDyG2tvTvLVl56O3MuW1+8/mOSR7v7xDbb5M8t2qaqrs/K98XvbN8o5VfWaqvrml65n5U3EX16z2b4896ts+H+4+/ncr3JfkqPL9aNJ7l1nm335p8eq6rok/yzJO7v7/2ywzZk8RvacNe/D/JtZf0778ryv8l1JvtLdJ9dbuR/O/cv8jNtdj/ud+q2CnfjKym+i/XpWfiPhh5dlP5jkB5frleSnlvVfSnJkp8d8lub9V7LyFOkXkzy4fL19zdzfleShrPyGxqeSfOdOj/sszv/1y7y+sMzxwJz7ZW6vzkpA/alVy/btuc9KWD6T5A+z8n+vNyf500keSPLYcnnesu23Jvn4qn2/4d+IvfS1wdxPZOU9Ki899v/d2rlv9BjZS18bzP0/LI/nL2blh+hF+/G8bzT/ZfnPvPRYX7Xtfjv3G/2M21WPe580DwAw7CC9pAgAsCMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAz7v5tB+AnIfYf5AAAAAElFTkSuQmCC\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "2e27c37c8a744e2bba3e14658af0e972": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "ffd59b46be7d4359b01758aacdefc036": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_d186dfd24688413a9e6fdd5e271d2416",
                    "IPY_MODEL_caf9b8e2e24a4bc38abe20ca79e2e649"
                ],
                "layout": "IPY_MODEL_2e27c37c8a744e2bba3e14658af0e972"
            }
        },
        "0113daf86fcc4c0fb6819a553f2a7a3b": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "b50b923ae869429b99133275afc659f1": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "d186dfd24688413a9e6fdd5e271d2416": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_0113daf86fcc4c0fb6819a553f2a7a3b",
                "max": 21,
                "min": 1,
                "style": "IPY_MODEL_b50b923ae869429b99133275afc659f1",
                "value": 11
            }
        },
        "67c0cfd092f049789fffb2bbbb2eeaec": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "caf9b8e2e24a4bc38abe20ca79e2e649": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_67c0cfd092f049789fffb2bbbb2eeaec",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAlwAAAJOCAYAAABiAtkgAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAAhCklEQVR4nO3dfbRld13n+c/XBMJDRMBU0iGJBDQLTexu0SwE7XHSDQ5R1DBrYE2cQdMMGp2J3epymgnaNmqTNmuGUehu0aYBSctDJh0ZiA9jkwkii+4WLEAHQ2SSBkwVCUkBzZMygcB3/ji79OTmVqpSt751b1W9Xmvddc/ZZz/8zj51677v3vvcW90dAADmfMV2DwAA4HgnuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLtigqt5eVT90mMt+TVV9rqpOOtLj2omq6iNV9Yzl9k9X1auO4Lo/V1VPXG6/tqpecgTX/WtV9bNHan1r662q+vWq+s9V9e4jtM7/uqr2LPvjyVV1c1VddITW/fCq+q2q+nRV/dsHuey5VdVVdfKRGAsc7wQXx6UlBD6/fJP62PIN+9Sh7Txj//3uvr27T+3uLx3h7VxUVV9ens/6x9OO5Ha2orv/WXcfNFQPNWiX/fihrY6rqv5+Vb1zw7p/tLv/6VbXvYm/k+Q7k5zd3U85Qut8aZIfW/bH+7r7gu5+e5JU1c9V1eu2sO7nJDkjyVd393OPwFi33WZxXlU/VlW7q+qeqnrthsceWlXXL1/LfaRiFjYSXBzPvre7T03yTUmenORF2zucLbtj+aa7/vEfN860HGX5ig3THtRRiO08anGMHzF5fJKPdPdfPNgFH+B5Pz7JzVsa1YE9Psn/2933Dq1/p7gjyUuSvOYAj78zyfOSfOyojYgTjuDiuNfdH0vy77IKryRJVT21qv5DVX2qqv7kQD/VVtXXVtXbquoTVfXxqnp9VT16eew3knxNkt9ajja9cP00S1VdWlW7N6zvJ6vqhuX2s6rqfVX1meWU0c8d7nNcjhpdVVX/PslfJnniMo4rqurWJLcu8/1wVd1WVZ+sqhuq6nFr67jf/Jts5weq6s+X/fEzGx77q6MtVfWwqnrdMt+nquqPquqMqroqyX+R5F8u++xfHmjby7SvW9vEaVV1Y1V9tqr+oKoev8x3v1Nb+4+iVdU3JPm1JE9btvep5fH7HAU5hP3yo1V1a61OFf5KVdUm++YFSV61tq2f38o+r6pTqupzSU5K8idV9Z+W6R+pqmdU1cVJfjrJf7ts708O8Jp9w7I/PlWr05Hft0z/+ST/ZG35F2yy7FdU1ZVV9Z+W1/K6qnrsAbbzVVX16qq6s6o+WlUvqeXUeq2OMv77qvrlZRwfqqpvW6bvqaq7q+qyDc/9pVV1e1XdVatTwA9fHruoqvZW1U8ty91ZVc9fHrs8yX+f5IXLc/qtJOnuN3X3m5N8YuO4u/sL3f2y7n5nkiN6ZBruo7t9+DjuPpJ8JMkzlttnJ3l/kpcv98/K6j/e787qh47vXO7vWh5/e5IfWm5/3fL4KUl2JXlHkpdttp3l/rlJOsnJSR6R5LNJzlt7/I+SXLrcvijJ31zG8LeS3JXk2Qd4Phcl2fsAz/ftSW5PcsGy7Ycs47gxyWOTPDzJ30vy8STfvDyff5HkHWvruM/8m2zj/CSfS/Idy/K/lOTetf38c0let9z+kSS/teyDk5J8S5JHbdy/D7TtZdrXLbdfu+zL/dt+eZJ3btznG/bH/tfw7++fd+3x1yZ5yXL7UPbLbyd5dFaBvS/JxQd4He6zra3u8437YZN/23+1zw+w7EOS3JZVmD10Gc9nkzzpEJf/iSR/mNXX0ClJ/lWSN26235O8eXn8kUlOT/LuJD+ytl/uTfL85d/DS7L69/ory3r/q2Vcpy7zvyzJDct++cqs/i394trXwr1JfmF5ft+d1Q8Zj9n42m7yfF6S5LUP8Hz3Jrlou///8nF8fjjCxfHszVX12SR7ktyd5MXL9Ocl+d3u/t3u/nJ335hkd1b/cd9Hd9/W3Td29z3dvS+ryPgvD2Xj3f2XSd6S5PuTpKrOS/L1WX0jSXe/vbvfv4zh/0nyxoOs+3HL0YH1j0euPf7a7r65u+/t7i8u036xuz/Z3Z/P6if/13T3e7v7nqxOsT6tqs5dW8f6/Bs9J8lvd/c7luV/NsmXDzDWLyb56qxC4Uvd/Z7u/swDPLeDbTtJfmdt2z+zjP2cg6zzUBzKfrm6uz/V3bcn+f2sHS09Aus+2PPeiqcmOTWr8X+hu9+WVTx+/yEu/yNJfqa79y7j/7kkz6kNpz+r6owk35XkJ7r7L7r77iS/nOTStdk+3N2/3qvrG/+PJOck+YXla+utSb6Q5OuWo4c/nOQnl/3y2ST/bMO6vrgs+8Xu/t2sfhB40iHvFdgGgovj2bO7+yuz+on465Octkx/fJLnrodLVhc7n7lxBVV1elVdu5wi+UyS162t51C8IX/9ze2/S/LmJcRSVd9aVb9fVfuq6tNJfvQg676jux+94WP9WqE9myyzPu1xSf58/53u/lxWR/bOOsg61pf/q8eXbd/vFM3iN7I6jXttVd1RVf9rVT3kAdZ9sG3f5/Fl7J9cxrRVh7Jf1q/t+cusIuZIrftgz3srHpdkT3evh/Gfb9j+A3l8kv9z7evklqxOu52xyXwPSXLn2rz/KqsjXfvdtXb780nS3RunnZrVkeRHJHnP2rp+b5m+3yf6vtedPZjXBLaF4OK4191/kNVphpcuk/Yk+Y0N4fLI7r56k8V/MavTJn+rux+V1dGx9et3+iCbf2tW1x59U1bh9Ya1x96Q1dGuc7r7q7K61uh+1wY9CJuNZX3aHVl9Y0ySLEfHvjrJRw+yjv3uzOqoxP7lH7Esf/+Nro48/Hx3n5/k25J8T5IfPMg2DrYv17d9alanm+5Isj86H7E27994EOs9lP1yuLa6zw/mUJ7bOXXfN1F8TQ79ue1J8l0bvlYe1t0bl9+T5J4kp63N96juvuAQt7Pu41nF1wVr6/qqXr0B5lBsZX/CGMHFieJlSb5zCZ/XJfneqnpmVZ1Uqwu8L6qqszdZ7iuzOl3xqao6K8k/2vD4XUmeeKCNLj+FX5/kf8sqEG7csO5Pdvf/V1VPyeoI2KQ3JHl+VX1TVZ2S1Wmad3X3Rw5x+euTfE9V/Z2qemhW19Bs+n9IVf3dqvqby0XTn8nqFND+C5IfcJ89gO9e2/Y/Xca+ZznV+9Ekz1tez/8hydeuLXdXkrOX5Taz1f3yQCbXnaye27kbgmrdu7IK0hdW1UNq9eaQ701y7SGu/9eSXFV//QaFXVV1ycaZuvvOrH64+N+r6lHLxfZfW1WHdPp9w7q+nORfJ/nlqjp92e5ZVfXMQ1zF/f591epNLA/L6vqx/V/z62+yOGV5PEkeujy+lR9+4H4EFyeE5Zvyv0nys929J8klWV1IvC+rn87/UTb/evj5rC54/nSS30nypg2P/2KSf7yc+vifD7D5NyR5RpJ/u+E0yP+U5BeW68z+SZLrDvI0Hlf3/z1c/81Blvkr3X1TVtdd/WZWR6u+Nve9LuZgy9+c5Irl+dyZ5D9ndZHxZv5GVoH2maxOQ/1BVqGbrC54f06t3vH3zw91+8t2X5zVqcRvyer6qP1+OKvX8BNZvXHgP6w99rasfq3Cx6rq45s8ry3tlwcyue7F/l9W+omqeu8m2/9Cku/L6vqqjyd5RZIf7O4/O8T1vzyro7BvXf6d/mGSbz3AvD+Y1YX5H8jq38b12eQ0/SH6X7K62P8Pl1P5/3cO/RqtVyc5f/mafPMy7R9nddTsyqyOUn9+mbbfB5dpZ2V1KvzzWTsyCUdCdTv6CgAwyREuAIBhggsAYJjgAgAYJrgAAIbt+D8Se9ppp/W555673cMAADio97znPR/v7l0bp+/44Dr33HOze/fug88IALDNqurPN5vulCIAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDTt7uAewE5175O/eb9pGrn7UNIwEAjkeOcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAw7aHBV1Wuq6u6q+tO1aY+tqhur6tbl82PWHntRVd1WVR+sqmeuTf+Wqnr/8tg/r6o68k8HAGDnOZQjXK9NcvGGaVcmuam7z0ty03I/VXV+kkuTXLAs84qqOmlZ5leTXJ7kvOVj4zoBAI5LBw2u7n5Hkk9umHxJkmuW29ckefba9Gu7+57u/nCS25I8parOTPKo7v6P3d1J/s3aMgAAx7XDvYbrjO6+M0mWz6cv089Ksmdtvr3LtLOW2xunb6qqLq+q3VW1e9++fYc5RACAneFIXzS/2XVZ/QDTN9Xdr+zuC7v7wl27dh2xwQEAbIfDDa67ltOEWT7fvUzfm+SctfnOTnLHMv3sTaYDABz3Dje4bkhy2XL7siRvWZt+aVWdUlVPyOri+Hcvpx0/W1VPXd6d+INrywAAHNdOPtgMVfXGJBclOa2q9iZ5cZKrk1xXVS9IcnuS5yZJd99cVdcl+UCSe5Nc0d1fWlb1P2b1jseHJ/m/lg8AgOPeQYOru7//AA89/QDzX5Xkqk2m707yjQ9qdAAAxwG/aR4AYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYNiWgquqfrKqbq6qP62qN1bVw6rqsVV1Y1Xdunx+zNr8L6qq26rqg1X1zK0PHwBg5zvs4Kqqs5L8wyQXdvc3JjkpyaVJrkxyU3efl+Sm5X6q6vzl8QuSXJzkFVV10taGDwCw8231lOLJSR5eVScneUSSO5JckuSa5fFrkjx7uX1Jkmu7+57u/nCS25I8ZYvbBwDY8Q47uLr7o0lemuT2JHcm+XR3vzXJGd195zLPnUlOXxY5K8metVXsXabdT1VdXlW7q2r3vn37DneIAAA7wlZOKT4mq6NWT0jyuCSPrKrnPdAim0zrzWbs7ld294XdfeGuXbsOd4gAADvCVk4pPiPJh7t7X3d/Mcmbknxbkruq6swkWT7fvcy/N8k5a8ufndUpSACA49pWguv2JE+tqkdUVSV5epJbktyQ5LJlnsuSvGW5fUOSS6vqlKp6QpLzkrx7C9sHADgmnHy4C3b3u6rq+iTvTXJvkvcleWWSU5NcV1UvyCrKnrvMf3NVXZfkA8v8V3T3l7Y4fgCAHe+wgytJuvvFSV68YfI9WR3t2mz+q5JctZVtAgAca/ymeQCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGDYloKrqh5dVddX1Z9V1S1V9bSqemxV3VhVty6fH7M2/4uq6raq+mBVPXPrwwcA2Pm2eoTr5Ul+r7u/PsnfTnJLkiuT3NTd5yW5abmfqjo/yaVJLkhycZJXVNVJW9w+AMCOd9jBVVWPSvIdSV6dJN39he7+VJJLklyzzHZNkmcvty9Jcm1339PdH05yW5KnHO72AQCOFVs5wvXEJPuS/HpVva+qXlVVj0xyRnffmSTL59OX+c9Ksmdt+b3LtPupqsurandV7d63b98WhggAsP22ElwnJ/nmJL/a3U9O8hdZTh8eQG0yrTebsbtf2d0XdveFu3bt2sIQAQC231aCa2+Svd39ruX+9VkF2F1VdWaSLJ/vXpv/nLXlz05yxxa2DwBwTDjs4OrujyXZU1VPWiY9PckHktyQ5LJl2mVJ3rLcviHJpVV1SlU9Icl5Sd59uNsHADhWnLzF5f9BktdX1UOTfCjJ87OKuOuq6gVJbk/y3CTp7pur6rqsouzeJFd095e2uH0AgB1vS8HV3X+c5MJNHnr6Aea/KslVW9kmAMCxxm+aBwAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIZtObiq6qSqel9V/fZy/7FVdWNV3bp8fszavC+qqtuq6oNV9cytbhsA4FhwJI5w/XiSW9buX5nkpu4+L8lNy/1U1flJLk1yQZKLk7yiqk46AtsHANjRthRcVXV2kmcledXa5EuSXLPcvibJs9emX9vd93T3h5PcluQpW9k+AMCxYKtHuF6W5IVJvrw27YzuvjNJls+nL9PPSrJnbb69y7T7qarLq2p3Ve3et2/fFocIALC9Dju4qup7ktzd3e851EU2mdabzdjdr+zuC7v7wl27dh3uEAEAdoSTt7Dstyf5vqr67iQPS/Koqnpdkruq6szuvrOqzkxy9zL/3iTnrC1/dpI7trB9AIBjwmEf4eruF3X32d19blYXw7+tu5+X5IYkly2zXZbkLcvtG5JcWlWnVNUTkpyX5N2HPXIAgGPEVo5wHcjVSa6rqhckuT3Jc5Oku2+uquuSfCDJvUmu6O4vDWwfAGBHOSLB1d1vT/L25fYnkjz9APNdleSqI7FNAIBjhd80DwAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAw7LCDq6rOqarfr6pbqurmqvrxZfpjq+rGqrp1+fyYtWVeVFW3VdUHq+qZR+IJAADsdFs5wnVvkp/q7m9I8tQkV1TV+UmuTHJTd5+X5KblfpbHLk1yQZKLk7yiqk7ayuABAI4Fhx1c3X1nd793uf3ZJLckOSvJJUmuWWa7Jsmzl9uXJLm2u+/p7g8nuS3JUw53+wAAx4ojcg1XVZ2b5MlJ3pXkjO6+M1lFWZLTl9nOSrJnbbG9y7TN1nd5Ve2uqt379u07EkMEANg2Ww6uqjo1yW8m+Ynu/swDzbrJtN5sxu5+ZXdf2N0X7tq1a6tDBADYVlsKrqp6SFax9fruftMy+a6qOnN5/Mwkdy/T9yY5Z23xs5PcsZXtAwAcC7byLsVK8uokt3T3L609dEOSy5bblyV5y9r0S6vqlKp6QpLzkrz7cLcPAHCsOHkLy357kh9I8v6q+uNl2k8nuTrJdVX1giS3J3luknT3zVV1XZIPZPUOxyu6+0tb2D4AwDHhsIOru9+Zza/LSpKnH2CZq5JcdbjbBAA4FvlN8wAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAw7OTtHsBOde6Vv3O/aR+5+lnbMBIA4MHYid/DHeECABgmuAAAhh31U4pVdXGSlyc5Kcmruvvqoz2Gw7XZIcpDtd2HMgHgeLSV781H01ENrqo6KcmvJPnOJHuT/FFV3dDdHzia49gOB/oHsVmIHeo/nq1EnHgEYDvsxOurjoajfYTrKUlu6+4PJUlVXZvkkiTHfXAdyFbCZ7uqfqf9NLGVaD0a2zjUZR/MNo70OnfS89vKfAdypJ/LVpY91HFPPOfteg120lgOtPxmtvL8DnV9O+nr7MGM50jbad9XJlR3H72NVT0nycXd/UPL/R9I8q3d/WMb5rs8yeXL3Scl+eDw0E5L8vHhbfDgeV12Hq/JzuM12Zm8LjvP0XpNHt/duzZOPNpHuGqTafcrvu5+ZZJXzg9npap2d/eFR2t7HBqvy87jNdl5vCY7k9dl59nu1+Rov0txb5Jz1u6fneSOozwGAICj6mgH1x8lOa+qnlBVD01yaZIbjvIYAACOqqN6SrG7762qH0vy77L6tRCv6e6bj+YYDuConb7kQfG67Dxek53Ha7IzeV12nm19TY7qRfMAACciv2keAGCY4AIAGHbCB1dVXVxVH6yq26rqyu0ez4muqs6pqt+vqluq6uaq+vHtHhMrVXVSVb2vqn57u8fCSlU9uqqur6o/W75mnrbdYzrRVdVPLv93/WlVvbGqHrbdYzoRVdVrquruqvrTtWmPraobq+rW5fNjjuaYTujgWvtTQ9+V5Pwk319V52/vqE549yb5qe7+hiRPTXKF12TH+PEkt2z3ILiPlyf5ve7++iR/O16fbVVVZyX5h0ku7O5vzOrNYZdu76hOWK9NcvGGaVcmuam7z0ty03L/qDmhgytrf2qou7+QZP+fGmKbdPed3f3e5fZns/oGctb2joqqOjvJs5K8arvHwkpVPSrJdyR5dZJ09xe6+1PbOiiS1bv/H15VJyd5RPyuyW3R3e9I8skNky9Jcs1y+5okzz6aYzrRg+usJHvW7u+Nb+47RlWdm+TJSd61zUMheVmSFyb58jaPg7/2xCT7kvz6cqr3VVX1yO0e1Imsuz+a5KVJbk9yZ5JPd/dbt3dUrDmju+9MVj/cJzn9aG78RA+uQ/pTQxx9VXVqkt9M8hPd/ZntHs+JrKq+J8nd3f2e7R4L93Fykm9O8qvd/eQkf5GjfIqE+1quCbokyROSPC7JI6vqeds7KnaKEz24/KmhHaiqHpJVbL2+u9+03eMh357k+6rqI1mddv97VfW67R0SWf3/tbe79x8Bvj6rAGP7PCPJh7t7X3d/McmbknzbNo+Jv3ZXVZ2ZJMvnu4/mxk/04PKnhnaYqqqsrkm5pbt/abvHQ9LdL+rus7v73Ky+Rt7W3X5q32bd/bEke6rqScukpyf5wDYOidWpxKdW1SOW/8ueHm9k2EluSHLZcvuyJG85mhs/qn/aZ6fZwX9q6ET27Ul+IMn7q+qPl2k/3d2/u31Dgh3rHyR5/fID44eSPH+bx3NC6+53VdX1Sd6b1Tuu3xd/4mdbVNUbk1yU5LSq2pvkxUmuTnJdVb0gqzh+7lEdkz/tAwAw60Q/pQgAME5wAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADD/n+6e9kJ4my78gAAAABJRU5ErkJggg==\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "a9c2144e84e64c0cb1037d323eaa3a08": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "b1eebb3ff7aa434b99a326ca82376515": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_b510b9b7211f402eb4fd9abd05f2b6a3",
                    "IPY_MODEL_6adab2995e97453686116fa77236f409"
                ],
                "layout": "IPY_MODEL_a9c2144e84e64c0cb1037d323eaa3a08"
            }
        },
        "8b849e4d38ec4d319fdb7c4127cb2816": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "73458e57d7634c92884892545ce99439": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "b510b9b7211f402eb4fd9abd05f2b6a3": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_8b849e4d38ec4d319fdb7c4127cb2816",
                "max": 21,
                "min": 1,
                "style": "IPY_MODEL_73458e57d7634c92884892545ce99439",
                "value": 1
            }
        },
        "a2d350b9dc0c42a2a7e3584324a4c12a": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "6adab2995e97453686116fa77236f409": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_a2d350b9dc0c42a2a7e3584324a4c12a",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAlwAAAJOCAYAAABiAtkgAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAAjUklEQVR4nO3dfZRld13n+8/XBMJDRAjpxJBEEjQLTRyvaK8QHWcuIzBEfEhmDayJ9woZB43MjTOjaxxMZBxAjTKOdwSvokZUogiZiA+0IjNkosjSEWLzZAwxpiUx3aRJGhgeAhpI+N4/zm44qa7qqtSpX1V19+u11ll1ap/9dPau6n7X3vucU90dAADG+YKtXgEAgKOd4AIAGExwAQAMJrgAAAYTXAAAgwkuAIDBBBesoKreWlXftc5pv6Sq7q2q4zZ6vbajqrqjqp4x3f+hqnr1Bs773qp60nT/NVX1Yxs471+oqh/eqPnNzbeq6ler6n9X1Y0bNM9/VlV7p+3xlKq6uaqetkHzfmRV/V5VfayqfvMhTntWVXVVHb8R6wJHK8HFUW0Kgb+b/pP64PQf9omDlvOMg993953dfWJ3P7DBy3laVX12ej7zt6/byOUsort/vLtXDdW1Bu20Hd+/6HpV1b+sqj9ZMu8XdvePLjrvZXxDkmcmOaO7z9+gef5Uku+dtse7u/u87n5rklTVS6vqtQvM+zlJTk3y+O5+7gas65ZbS5xX1Y9W1U1VdX9VvXSTVo1jlODiWPCt3X1ikq9O8pQkV27t6izsruk/3fnbny0daTrK8gVLhj2koxBbedTiCD9i8sQkd3T3Jx/qhId53k9McvNCa7WyJyb56+6+f9D8t6s9SV6U5E1bvSIc/QQXx4zu/mCS/5FZeCVJquqCqvpfVfXRqnrvSqdoqupLq+oPq+rDVfWhqvqNqnrs9NivJ/mSJL83HW160fxplqq6pKp2L5nf91fVrun+N1fVu6vq49Mpo5eu9zlOR42uqqo/TfKpJE+a1uPyqrotyW3TeN9dVXuq6iNVtauqnjA3j0PGX2Y5z6uqv522x4uXPPa5oy1V9Yiqeu003ker6s+r6tSquirJP0rys9M2+9mVlj0N+7K5RZxcVddX1Seq6o+r6onTeIec2jp4FK2qviLJLyT5uml5H50ef9BRkDVslxdW1W01O1X4c1VVy2ybFyR59dyyXrbINq+qE6rq3iTHJXlvVf3NNPyOqnpGVV2Y5IeS/Itpee9dYZ99xbQ9Plqz05HfNg1/WZL/NDf9C5aZ9guq6oqq+ptpX15XVSetsJwvqqpfrqr9VfWBqvqxmk6t1+wo459W1U9P6/H+qvr6afjeqrqnqi5d8tx/qqrurKq7a3YK+JHTY0+rqn1V9e+n6fZX1XdOj12W5P9O8qLpOf3ecuva3dd095uTfGK5x2FDdbeb21F7S3JHkmdM989IclOSV07fn57kw0mendkfH8+cvt8xPf7WJN813f+y6fETkuxI8rYkr1huOdP3ZyXpJMcneVRm/6CfM/f4nye5ZLr/tCT/YFqHr0pyd5KLV3g+T0uy7zDP961J7kxy3rTsh03rcX2Sk5I8Msk3JvlQkq+Zns//l+Rtc/N40PjLLOPcJPcm+cfT9P81yf1z2/mlSV473f+eJL83bYPjknxtkscs3b6HW/Y07Mum+6+ZtuXBZb8yyZ8s3eZLtsfBffgvD4479/hrkvzYdH8t2+X3kzw2s8A+kOTCFfbDg5a16DZfuh2W+dn+3DZfYdqHZXY054eSPHxan08kefIap/++JG/P7HfohCS/mOT1y233JL87Pf7oJKckuTHJ98xtl/uTfOf08/Bjmf28/tw03386rdeJ0/ivSLJr2i5fmNnP0k/M/S7cn+RHpuf37Mz+yHjc0n27hn8nXpvkpVv975Xb0X1zhItjwe9W1SeS7E1yT5KXTMO/I8kfdPcfdPdnu/v6JLsz+4f7Qbp7T3df3933dfeBzCLj/1zLwrv7U0nemOTbk6Sqzkny5Zn9R5Lufmt33zStw18kef0q837CdHRg/vboucdf0903d/f93f2ZadhPdPdHuvvvMvvL/1e6+13dfV9mp1i/rqrOmpvH/PhLPSfJ73f326bpfzjJZ1dY188keXxmofBAd7+zuz9+mOe22rKT5E1zy37xtO5nrjLPtVjLdnl5d3+0u+9M8keZO1q6AfNe7Xkv4oIkJ2a2/p/u7j/MLB6/fY3Tf0+SF3f3vmn9X5rkObXk9GdVnZrkm5J8X3d/srvvSfLTSS6ZG+327v7Vnl3f+N+SnJnkR6bfrbck+XSSL5uOHn53ku+ftssnkvz4knl9Zpr2M939B5n9IfDkNW8V2ESCi2PBxd39hZn9RfzlSU6ehj8xyXPnwyWzi51PWzqDqjqlqq6dTpF8PLO/iE9eOt5hvC6f/8/t/0ryu1OIpaqeWlV/VFUHqupjSV64yrzv6u7HLrnNXyu0d5lp5oc9IcnfHvymu+/N7Mje6avMY376zz0+LfvDK4z765mdxr22qu6qqp+sqocdZt6rLftBj0/r/pFpnRa1lu3ywbn7n8osYjZq3qs970U8Icne7p4P479dsvzDeWKS35n7PbklyQOZXWi/dLyHJdk/N+4vZnak66C75+7/XZJ099JhJ2Z2JPlRSd45N6//Pg0/6MP94OvOHso+gU0luDhmdPcfZ3aa4aemQXuT/PqScHl0d798mcl/IrPTJl/V3Y/J7OjY/PU7vcri35LZtUdfnVl4vW7usddldrTrzO7+osyuNTrk2qCHYLl1mR92V2b/MSZJpqNjj0/ygVXmcdD+zI5KHJz+UdP0hy50duThZd19bpKvT/ItSZ6/yjJW25bzyz4xs9NNdyU5GJ2Pmhv3ix/CfNeyXdZr0W2+mrU8tzPrwS+i+JKs/bntTfJNS35XHtHdS6ffm+S+JCfPjfeY7j5vjcuZ96HM4uu8uXl9Uc9eALMWi2xP2HCCi2PNK5I8cwqf1yb51qp6VlUdV7MLvJ9WVWcsM90XZna64qNVdXqS/7Dk8buTPGmlhU5/hb8hyX/JLBCuXzLvj3T331fV+ZkdARvpdUm+s6q+uqpOyOw0zTu6+441Tv+GJN9SVd9QVQ/P7BqaZf8tqap/UlX/YLpo+uOZnQI6+FYZh91mh/HsuWX/6LTue6dTvR9I8h3T/vxXSb50brq7k5wxTbecRbfL4YycdzJ7bmctCap578gsSF9UVQ+r2YtDvjXJtWuc/y8kuao+/wKFHVV10dKRunt/Zn9c/L9V9ZjpYvsvrao1nX5fMq/PJvmlJD9dVadMyz29qp61xlms+vM1bYtHZPbze/z0b8Ax8d55bD7BxTFl+k/515L8cHfvTXJRZhcSH8jsr/P/kOV/L16W2QXPH8vsJeS/veTxn0jyH6dTHz+wwuJfl+QZSX5zyWmQ/yfJj0zXmf2nJNet8jSeUIe+D9c/X2Waz+nuGzK77uq3Mjta9aV58HUxq01/c5LLp+ezP8n/TrJvhdG/OLNA+3hmp6H+OLPQTWYXvD+nZq/4+5m1Ln9a7ksyO5X4tZldH3XQd2e2Dz+c2QsH/tfcY3+Y2dsqfLCqPrTM81pouxzOyHlPDr5Z6Yer6l3LLP/TSb4ts+urPpTkVUme391/tcb5vzKzo7BvmX5O357kqSuM+/zMLsx/X2Y/G2/IMqfp1+gHM7vY/+3Tqfz/mbVfo/XLSc6dfid/d4Vxfimzo2jfntn1gH+X5HnrXFc4rOp21BUAYCRHuAAABhNcAACDCS4AgMEEFwDAYNv+w2FPPvnkPuuss7Z6NQAAVvXOd77zQ929Y+nwbR9cZ511Vnbv3r36iAAAW6yq/na54U4pAgAMtmpwVdWvVNU9VfWXc8P+S1X9VVX9RVX9TlU9du6xK6tqT1XdOv+OwFX1tVV10/TYz0wfTAoAcNRbyxGu1yS5cMmw65N8ZXd/VZK/zuyT71NV52b27snnTdO8au5jEn4+yWVJzpluS+cJAHBUWjW4uvttmX2Exvywt8x9NMnbkxz87LmLklzb3fd19+2ZfSTD+VV1WpLHdPef9eyt7X8tycUb9BwAALa1jbiG618lefN0//TMPo/uoH3TsNPz4M9aOzh8WVV1WVXtrqrdBw4c2IBVBADYOgsFV1W9OMn9SX7j4KBlRuvDDF9Wd1/d3Tu7e+eOHYe8shIA4Iiy7reFqKpLk3xLkqf35z8Be1+SM+dGOyPJXdPwM5YZDgBw1FvXEa6qujDJDyb5tu7+1NxDu5JcUlUnVNXZmV0cf2N370/yiaq6YHp14vOTvHHBdQcAOCKseoSrql6f5GlJTq6qfUlektmrEk9Icv307g5v7+4XdvfNVXVdkvdldqrx8u5+YJrVv87sFY+PzOyarzcHAOAYUJ8/G7g97dy5s73TPABwJKiqd3b3zqXDvdM8AMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGO36rV4Ajz1lXvOmQYXe8/Ju3YE0A4MjgCBcAwGCCCwBgMMEFADCY4AIAGExwAQAMJrgAAAYTXAAAgwkuAIDBBBcAwGCCCwBgMMEFADCY4AIAGExwAQAMJrgAAAYTXAAAgwkuAIDBBBcAwGCCCwBgMMEFADCY4AIAGExwAQAMJrgAAAYTXAAAgwkuAIDBBBcAwGCCCwBgMMEFADCY4AIAGExwAQAMJrgAAAYTXAAAgwkuAIDBBBcAwGCCCwBgMMEFADCY4AIAGExwAQAMJrgAAAYTXAAAgwkuAIDBBBcAwGCCCwBgMMEFADCY4AIAGExwAQAMJrgAAAYTXAAAgwkuAIDBBBcAwGCCCwBgMMEFADCY4AIAGExwAQAMJrgAAAYTXAAAgwkuAIDBBBcAwGCCCwBgMMEFADCY4AIAGExwAQAMJrgAAAYTXAAAgwkuAIDBBBcAwGCCCwBgsFWDq6p+paruqaq/nBt2UlVdX1W3TV8fN/fYlVW1p6purapnzQ3/2qq6aXrsZ6qqNv7pAABsP2s5wvWaJBcuGXZFkhu6+5wkN0zfp6rOTXJJkvOmaV5VVcdN0/x8ksuSnDPdls4TAOCotGpwdffbknxkyeCLklwz3b8mycVzw6/t7vu6+/Yke5KcX1WnJXlMd/9Zd3eSX5ubBgDgqLbea7hO7e79STJ9PWUafnqSvXPj7ZuGnT7dXzp8WVV1WVXtrqrdBw4cWOcqAgBsDxt90fxy12X1YYYvq7uv7u6d3b1zx44dG7ZyAABbYb3Bdfd0mjDT13um4fuSnDk33hlJ7pqGn7HMcACAo956g2tXkkun+5cmeePc8Euq6oSqOjuzi+NvnE47fqKqLphenfj8uWkAAI5qx682QlW9PsnTkpxcVfuSvCTJy5NcV1UvSHJnkucmSXffXFXXJXlfkvuTXN7dD0yz+teZveLxkUnePN0AAI56qwZXd3/7Cg89fYXxr0py1TLDdyf5yoe0dgAARwHvNA8AMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMNhCwVVV319VN1fVX1bV66vqEVV1UlVdX1W3TV8fNzf+lVW1p6purapnLb76AADb37qDq6pOT/Jvk+zs7q9MclySS5JckeSG7j4nyQ3T96mqc6fHz0tyYZJXVdVxi60+AMD2t+gpxeOTPLKqjk/yqCR3JbkoyTXT49ckuXi6f1GSa7v7vu6+PcmeJOcvuHwAgG1v3cHV3R9I8lNJ7kyyP8nHuvstSU7t7v3TOPuTnDJNcnqSvXOz2DcNO0RVXVZVu6tq94EDB9a7igAA28IipxQfl9lRq7OTPCHJo6vqOw43yTLDerkRu/vq7t7Z3Tt37Nix3lUEANgWFjml+Iwkt3f3ge7+TJLfTvL1Se6uqtOSZPp6zzT+viRnzk1/RmanIAEAjmqLBNedSS6oqkdVVSV5epJbkuxKcuk0zqVJ3jjd35Xkkqo6oarOTnJOkhsXWD4AwBHh+PVO2N3vqKo3JHlXkvuTvDvJ1UlOTHJdVb0gsyh77jT+zVV1XZL3TeNf3t0PLLj+AADb3rqDK0m6+yVJXrJk8H2ZHe1abvyrkly1yDIBAI403mkeAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYAsFV1U9tqreUFV/VVW3VNXXVdVJVXV9Vd02fX3c3PhXVtWeqrq1qp61+OoDAGx/ix7hemWS/97dX57k/0hyS5IrktzQ3eckuWH6PlV1bpJLkpyX5MIkr6qq4xZcPgDAtrfu4KqqxyT5x0l+OUm6+9Pd/dEkFyW5ZhrtmiQXT/cvSnJtd9/X3bcn2ZPk/PUuHwDgSLHIEa4nJTmQ5Fer6t1V9eqqenSSU7t7f5JMX0+Zxj89yd656fdNww5RVZdV1e6q2n3gwIEFVhEAYOstElzHJ/maJD/f3U9J8slMpw9XUMsM6+VG7O6ru3tnd+/csWPHAqsIALD1FgmufUn2dfc7pu/fkFmA3V1VpyXJ9PWeufHPnJv+jCR3LbB8AIAjwrqDq7s/mGRvVT15GvT0JO9LsivJpdOwS5O8cbq/K8klVXVCVZ2d5JwkN653+QAAR4rjF5z+3yT5jap6eJL3J/nOzCLuuqp6QZI7kzw3Sbr75qq6LrMouz/J5d39wILLBwDY9hYKru5+T5Kdyzz09BXGvyrJVYssEwDgSOOd5gEABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMJjgAgAYTHABAAwmuAAABls4uKrquKp6d1X9/vT9SVV1fVXdNn193Ny4V1bVnqq6taqeteiyAQCOBBtxhOvfJbll7vsrktzQ3eckuWH6PlV1bpJLkpyX5MIkr6qq4zZg+QAA29pCwVVVZyT55iSvnht8UZJrpvvXJLl4bvi13X1fd9+eZE+S8xdZPgDAkWDRI1yvSPKiJJ+dG3Zqd+9PkunrKdPw05PsnRtv3zTsEFV1WVXtrqrdBw4cWHAVAQC21rqDq6q+Jck93f3OtU6yzLBebsTuvrq7d3b3zh07dqx3FQEAtoXjF5j2Hyb5tqp6dpJHJHlMVb02yd1VdVp376+q05LcM42/L8mZc9OfkeSuBZYPAHBEWPcRru6+srvP6O6zMrsY/g+7+zuS7Epy6TTapUneON3fleSSqjqhqs5Ock6SG9e95gAAR4hFjnCt5OVJrquqFyS5M8lzk6S7b66q65K8L8n9SS7v7gcGLB8AYFvZkODq7rcmeet0/8NJnr7CeFcluWojlgkAcKTwTvMAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGW3dwVdWZVfVHVXVLVd1cVf9uGn5SVV1fVbdNXx83N82VVbWnqm6tqmdtxBMAANjuFjnCdX+Sf9/dX5HkgiSXV9W5Sa5IckN3n5Pkhun7TI9dkuS8JBcmeVVVHbfIygMAHAnWHVzdvb+73zXd/0SSW5KcnuSiJNdMo12T5OLp/kVJru3u+7r79iR7kpy/3uUDABwpNuQarqo6K8lTkrwjyandvT+ZRVmSU6bRTk+yd26yfdOw5eZ3WVXtrqrdBw4c2IhVBADYMgsHV1WdmOS3knxfd3/8cKMuM6yXG7G7r+7und29c8eOHYuuIgDAlloouKrqYZnF1m90929Pg++uqtOmx09Lcs80fF+SM+cmPyPJXYssHwDgSLDIqxQryS8nuaW7/+vcQ7uSXDrdvzTJG+eGX1JVJ1TV2UnOSXLjepcPAHCkOH6Baf9hkucluamq3jMN+6EkL09yXVW9IMmdSZ6bJN19c1Vdl+R9mb3C8fLufmCB5QMAHBHWHVzd/SdZ/rqsJHn6CtNcleSq9S4TAOBI5J3mAQAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABhMcAEADCa4AAAGE1wAAIMJLgCAwQQXAMBgggsAYDDBBQAwmOACABjs+K1eAY4OZ13xpkOG3fHyb96CNQGA7UdwbUPiBQCOLpseXFV1YZJXJjkuyau7++WbvQ5Hi7WG2SIBt9y0AMBDs6nBVVXHJfm5JM9Msi/Jn1fVru5+32aux1psRsysNP0iFpnfdlqXxFE9AI4em32E6/wke7r7/UlSVdcmuSjJlgbXWsNgo2PmSA2zzbLIOi4Sx2td7kYfJdzKwNxOR0uFNvNcYrEy22Zl23HbVHdv3sKqnpPkwu7+run75yV5and/75LxLkty2fTtk5PcusIsT07yoUGry+ps/61nH2w9+2Dr2Qdbzz74vCd2946lAzf7CFctM+yQ4uvuq5NcverMqnZ3986NWDEeOtt/69kHW88+2Hr2wdazD1a32e/DtS/JmXPfn5Hkrk1eBwCATbXZwfXnSc6pqrOr6uFJLkmya5PXAQBgU23qKcXuvr+qvjfJ/8jsbSF+pbtvXmCWq552ZCjbf+vZB1vPPth69sHWsw9WsakXzQMAHIt8liIAwGCCCwBgsG0RXFV1UlVdX1W3TV8ft8J4F1bVrVW1p6quWMv0VXXlNP6tVfWsueFXVdXeqrp37LPbvlbannOPV1X9zPT4X1TV16w27Xr2xbFsM/dBVT2+qv6oqu6tqp/dnGe4/W3yPnhmVb2zqm6avn7j5jzL7W2T98H5VfWe6fbeqvpnm/Mst7fN/v9gevxLpn+PfmDss9smunvLb0l+MskV0/0rkvznZcY5LsnfJHlSkocneW+Scw83fZJzp/FOSHL2NP1x02MXJDktyb1b/fy3aJuvuD3nxnl2kjdn9v5pFyR5x4h9cazetmAfPDrJNyR5YZKf3ernvx1uW7APnpLkCdP9r0zyga3eBlt924J98Kgkx0/3T0tyz8Hvj9XbZu+DuXn+VpLfTPIDW70NNuO2LY5wZfbxPtdM969JcvEy43zuY4G6+9NJDn4s0OGmvyjJtd19X3ffnmTPNJ9099u7e/8GP48jyeG250EXJfm1nnl7ksdW1WmrTPuQ98UxbFP3QXd/srv/JMnfj3xSR5jN3gfv7u6D7z14c5JHVNUJg57bkWKz98Gnuvv+afgjssybbx+DNvv/g1TVxUnen9nvwTFhuwTXqQfjZ/p6yjLjnJ5k79z3+6Zhh5v+cNMc69aybVYax77YGJu9DzjUVu6Df57k3d1937rX/uiw6fugqp5aVTcnuSnJC+cC7Fi1qfugqh6d5AeTvGyD1v+IsGnvw1VV/zPJFy/z0IvXOotlhq32l8l6pjlWrGXbrDSOfbExNnsfcKgt2QdVdV6S/5zkn65l/KPcpu+D7n5HkvOq6iuSXFNVb+7uY/nI72bvg5cl+enuvrdqucmPTpsWXN39jJUeq6q7q+q07t4/HaK8Z5nRDvexQCtN76OEVraWbbPSOA8/zLT2xdpt9j7gUJu+D6rqjCS/k+T53f03G/Isjmxb9nvQ3bdU1Sczu55u90LP4si22fvgqUmeU1U/meSxST5bVX/f3Uf1i3m2yynFXUkune5fmuSNy4xzuI8FWmn6XUkuqaoTqursJOckuXHA+h+J1vIxS7uSPH96dcoFST42HRa2LzbGZu8DDrWp+6CqHpvkTUmu7O4/Hfi8jiSbvQ/Orqrjp/tPTPLkJHcMe3ZHhk3dB939j7r7rO4+K8krkvz40R5bSbbNqxQfn+SGJLdNX0+ahj8hyR/MjffsJH+d2SsiXrza9NNjL57GvzXJN80N/8nMiv2z09eXbvV22ILtfsj2zOwVbC+c7leSn5sevynJzhH74li+bcE+uCPJR5LcO/3cnzv6OW7322bugyT/Mcknk7xn7nbKVm+Drb5t8j54XmYXar8nybuSXLzVz3873Db736K5cV6aY+RVij7aBwBgsO1yShEA4KgluAAABhNcAACDCS4AgMEEFwDAYIILAGAwwQUAMNj/DxPS5lTlH8xOAAAAAElFTkSuQmCC\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        },
        "ee832eeb377b46ecb13a6e3789f57947": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "af62e297abf445f79ee2f1855a949b44": {
            "model_name": "VBoxModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "_dom_classes": [
                    "widget-interact"
                ],
                "children": [
                    "IPY_MODEL_96b0f88d03c8431ba9c3af675ac0aa39",
                    "IPY_MODEL_17bc630276b844a0a35965c0abdfd260"
                ],
                "layout": "IPY_MODEL_ee832eeb377b46ecb13a6e3789f57947"
            }
        },
        "1789105642a0449cadcbe069319c24f1": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "596e45b61604496c9b91aef5691f19f8": {
            "model_name": "SliderStyleModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description_width": ""
            }
        },
        "96b0f88d03c8431ba9c3af675ac0aa39": {
            "model_name": "IntSliderModel",
            "model_module": "@jupyter-widgets/controls",
            "model_module_version": "1.5.0",
            "state": {
                "description": "Z",
                "layout": "IPY_MODEL_1789105642a0449cadcbe069319c24f1",
                "max": 21,
                "min": 1,
                "style": "IPY_MODEL_596e45b61604496c9b91aef5691f19f8",
                "value": 6
            }
        },
        "056cf1323b694303b3bb9a3adda5a148": {
            "model_name": "LayoutModel",
            "model_module": "@jupyter-widgets/base",
            "model_module_version": "1.2.0",
            "state": {}
        },
        "17bc630276b844a0a35965c0abdfd260": {
            "model_name": "OutputModel",
            "model_module": "@jupyter-widgets/output",
            "model_module_version": "1.0.0",
            "state": {
                "layout": "IPY_MODEL_056cf1323b694303b3bb9a3adda5a148",
                "outputs": [
                    {
                        "output_type": "display_data",
                        "data": {
                            "text/plain": "<Figure size 720x720 with 1 Axes>",
                            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAlwAAAJOCAYAAABiAtkgAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8vihELAAAACXBIWXMAAAsTAAALEwEAmpwYAAAelElEQVR4nO3de5Std13f8c/XhGsQBTnQkASCmqLgatGmiKKWikoUJNhVaqyX2KLxghYvrSuwtGo1Ftey3qpoES+xsExTLyWKtzSIFFHi4eIlhJQUkBwTk4OIXLShwW//2M/Rncmcc+bMzPfMnJzXa61Zs/ezf8/z/PbeZ86859nP7KnuDgAAcz5srycAAHBvJ7gAAIYJLgCAYYILAGCY4AIAGCa4AACGCS64l6qq76iqly6XH1VV76+qM3Zp2z9eVd+2XH5KVR3aje0u2/v0qrppt7a3YdtfUFW3LI/FJ+7C9h5RVa+uqvdV1X+qqhdU1Ut2Y67L9r+mqm5f5vtRJ7juq6rqK3ZrLsDOCC7YBVX1jqr66+Ub45GPH9nreR3R3e/s7gd194eONa6qvryqXrOF7X11d3/XbsytqrqqPnZt2/+rux+7G9vexPcl+brlsXjjLmzvsiTvSvLg7v7m7v6e7v6KJKmq85f7duZ2NlxV90ny/Uk+Z5nvn+/CfPfUVuO8qj5pCdn3L8H5vJMxP5i0rf8IgE19fnf/z+MNqqozu/uuDcvOOF4M7WT8btrLfe+CRye5YTsrHuV+PzrJm3vmHaQfkeT+2eZ8T1VV9bAkv57kG5P8fJL7Jjl3TycFu8ARLhi2HDX6nar6gap6d5LvqKqfqaofq6pfraoPJPmnVfXxy8tA76mqG6rqmWvbuMf4TfbzmKr67eXlrWuTPGzttrsdbVnm9LZl7Nur6our6uOT/HiST1mOLLznaPteln33hv2/oKretRzt++K15Xd7aWv9KFpVvXpZ/AfLPr9w41GQLTwuP1pVr1juy+uq6mM2eWzuV1XvT3LGsq//s8VtH/Uxr6qfSXJpkm9Z5v5ZtfYybpIj9+09y+2fcpR5/WBV3bp8/OCy7O8nuWlt/VduXHdZ/0lV9dpl/n9QVU/ZbNwy9l9X1Y1V9RdV9RtV9ei127qqvraq3ro8jt9VVR9TVb9bVe+tqqur6r5r459RVW9a9vvaqvoHa7e9o6r+bVX9YVX9ZVX9t6q6f1WdleTXkjyy/u4o8CM3meo3JfmN7n5Zd9/Z3e/r7huPdr/glNHdPnz42OFHknck+ayj3PblSe5K8vVZHVV+QJKfSfKXSZ6c1Q8+H57k5iQvyOon+s9M8r4kj122sXH8/TfZz+9m9RLU/ZJ8xrL+S5fbzk/Sy/7PSvLetW2fneTxa3N9zYbt3mPfy7LvXm5/ynL/juz7nyT5wNr2X5XkKzY8Hq9Zu95JPnbt+lOSHFou32cLj8u7kzxxuW8vS3LVMZ6nv93XFrd9vMf8bx+H5fp3bPaYH2M+/yHJ7yV5eJIDSV6b5Lu2sn6Sc5L8eZLPW+b32cv1Axsf9yTPWu7rxy+P07cmee2Gx+WaJA9O8vgkdya5LslHJ/mIJG9Ocuky9pOS3JHkk7MK2Euz+vd/v7WvheuTPDLJQ5PcmOSrNz63x3hMXpnkh5bH4o4kv5zkUXv9Ne7Dx04/HOGC3fM/lp/4j3x85dptt3b3f+7uu7r7r5dlL+/u3+nuv0nyhCQPSvLC7v5gd78yya8k+aK1bfzt+O7+v+s7rqpHJfnHSb6tV0cFXp3VN6qj+Zskn1BVD+ju27r7eC9bHXXfa47s+7eTvCLJvzjONrfiSTn+4/KL3X19r16mfVlWj+VubXsr93snvjjJf+juO7r7cJLvTPKlW1z3S5L8anf/6jK/a5MczCrANvqqJP+xu29cHqfvSfKE9aNcSb63u9+7/Fv44yS/2d1v6+6/zOrI1JFfMvjKJP+lu1/X3R/q7iuzCrQnrW3rh7v71u5+d1b/Dp+wxfuUrF4+vDTJ85I8Ksnbk/zcCawP+5Lggt3zrO7+yLWPn1i77ZZNxq8ve2SSW5b4OuJPsjqKcaxtrK//F939gQ3r38My5guTfHWS25aX4z7uGNs+3r5zlH1v9nLRidrK4/Jna5f/KquI2q1tH+9+79Qjc/fn6UQet0cnefZ65Cf5tKyOWG429ofWxr07SeXu9/X2tct/vcn1I4/ro5N884b9nrdh3tt9To7s65e6+/eXyP3OJJ9aVR9xAtuAfUdwwcmx2UnV68tuTXJeVa1/TT4qyZ8eZxtH3JbkIct5Muvrbz6Z7t/o7s/O6pvzW5IcicOj7eN4J4Vvtu9bl8sfSPLAtdv+3nG2tW4rj8t27fQxP56trHtrVgGzvv9bjzJ2o1uS/NcNkX9Wd7/wKGO/asPYB3T3a7e4r43bumLDth7Y3Vs5CrWVx+QPN4w7crlOdKKwnwgu2B9el1WYfEtV3Wc5+fnzk1y1lZW7+0+yejnpO6vqvlX1acv691Cr94565hJIdyZ5f5Ijv313e5Jz10+QPgFH9v3pSZ6R5L8vy9+U5J9V1QNr9fYPz9mw3u1ZnSu0mR09Lscxue0kOZzVS7dHu2/J6qWyb62qA7X67bx/n+Slxxi/7qVJPr+qnlZVZywnpj+lqjb7jb4fT/L8qnp8klTVR1TVs7d+V+7mJ5J8dVV9cq2cVVVPr6oP38K6tyf5qOMcrfrpJF9QVU+o1VtjfFtW5/y9Z5vzhX1BcMHu+eW6+/tw/dJWV+zuDyZ5ZpLPzep9nV6U5Mu6+y0nsP9/mdWJzO9O8u1JfvYo4z4syTdndSTl3Vmd5P61y22vzOptCP6sqt51Avv+syR/sWzzZVmdJH1k7j+Q5INZfbO9crl93XckuXJ5eepu533t0uOyqcltL9v/qyRXJPmd5b49aZNh351VKP9hkj9K8oZl2Va2f0uSi7M66f9wVkee/l02+X+9u38pyfcmuaqq3pvVOVqfe6L3adnWwazO4/qRrJ7zm7P6RYitrPuWrCLzbctjco+XT5dz6V6Q1XmAdyT52Kz+bcMprbon3j4GAIAjHOECABgmuAAAhgkuAIBhggsAYNi+/+PVD3vYw/r888/f62kAABzX61//+nd194GNy/d9cJ1//vk5ePDgXk8DAOC4qmrTv/LhJUUAgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCa4k51/+ipx/+Sv2ehoAwL2U4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBg2HGDq6p+qqruqKo/Xlv20Kq6tqreunx+yNptz6+qm6vqpqp62tryf1RVf7Tc9sNVVbt/dwAA9p+tHOH6mSQXbVh2eZLruvuCJNct11NVj0tySZLHL+u8qKrOWNb5sSSXJblg+di4TQCAe6XjBld3vzrJuzcsvjjJlcvlK5M8a235Vd19Z3e/PcnNSZ5YVWcneXB3/253d5KfXVsHAOBebbvncD2iu29LkuXzw5fl5yS5ZW3coWXZOcvljcs3VVWXVdXBqjp4+PDhbU4RAGB/2O2T5jc7L6uPsXxT3f3i7r6wuy88cODArk0OAGAvbDe4bl9eJszy+Y5l+aEk562NOzfJrcvyczdZDgBwr7fd4LomyaXL5UuTvHxt+SVVdb+qekxWJ8dfv7zs+L6qetLy24lftrYOAMC92pnHG1BVP5fkKUkeVlWHknx7khcmubqqnpPknUmenSTdfUNVXZ3kzUnuSvLc7v7Qsqmvyeo3Hh+Q5NeWDwCAe73jBld3f9FRbnrqUcZfkeSKTZYfTPIJJzQ7AIB7Ae80DwAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDdhRcVfWNVXVDVf1xVf1cVd2/qh5aVddW1VuXzw9ZG//8qrq5qm6qqqftfPoAAPvftoOrqs5J8m+SXNjdn5DkjCSXJLk8yXXdfUGS65brqarHLbc/PslFSV5UVWfsbPoAAPvfTl9SPDPJA6rqzCQPTHJrkouTXLncfmWSZy2XL05yVXff2d1vT3JzkifucP8AAPvetoOru/80yfcleWeS25L8ZXf/ZpJHdPdty5jbkjx8WeWcJLesbeLQsuwequqyqjpYVQcPHz683SkCAOwLO3lJ8SFZHbV6TJJHJjmrqr7kWKtssqw3G9jdL+7uC7v7wgMHDmx3igAA+8JOXlL8rCRv7+7D3f3/kvxikk9NcntVnZ0ky+c7lvGHkpy3tv65Wb0ECQBwr7aT4HpnkidV1QOrqpI8NcmNSa5Jcuky5tIkL18uX5Pkkqq6X1U9JskFSa7fwf4BAE4JZ253xe5+XVX9fJI3JLkryRuTvDjJg5JcXVXPySrKnr2Mv6Gqrk7y5mX8c7v7QzucPwDAvrft4EqS7v72JN++YfGdWR3t2mz8FUmu2Mk+AQBONd5pHgBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACG7Si4quojq+rnq+otVXVjVX1KVT20qq6tqrcunx+yNv75VXVzVd1UVU/b+fQBAPa/nR7h+qEkv97dH5fkHya5McnlSa7r7guSXLdcT1U9LsklSR6f5KIkL6qqM3a4fwCAfW/bwVVVD07yGUl+Mkm6+4Pd/Z4kFye5chl2ZZJnLZcvTnJVd9/Z3W9PcnOSJ253/wAAp4qdHOH66CSHk/x0Vb2xql5SVWcleUR335Yky+eHL+PPSXLL2vqHlmX3UFWXVdXBqjp4+PDhHUwRAGDv7SS4zkzySUl+rLs/MckHsrx8eBS1ybLebGB3v7i7L+zuCw8cOLCDKQIA7L2dBNehJIe6+3XL9Z/PKsBur6qzk2T5fMfa+PPW1j83ya072D8AwClh28HV3X+W5Jaqeuyy6KlJ3pzkmiSXLssuTfLy5fI1SS6pqvtV1WOSXJDk+u3uHwDgVHHmDtf/+iQvq6r7Jnlbkn+VVcRdXVXPSfLOJM9Oku6+oaquzirK7kry3O7+0A73DwCw7+0ouLr7TUku3OSmpx5l/BVJrtjJPgEATjXeaR4AYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhOw6uqjqjqt5YVb+yXH9oVV1bVW9dPj9kbezzq+rmqrqpqp62030DAJwKduMI1/OS3Lh2/fIk13X3BUmuW66nqh6X5JIkj09yUZIXVdUZu7B/AIB9bUfBVVXnJnl6kpesLb44yZXL5SuTPGtt+VXdfWd3vz3JzUmeuJP9AwCcCnZ6hOsHk3xLkr9ZW/aI7r4tSZbPD1+Wn5PklrVxh5Zl91BVl1XVwao6ePjw4R1OEQBgb207uKrqGUnu6O7Xb3WVTZb1ZgO7+8XdfWF3X3jgwIHtThEAYF84cwfrPjnJM6vq85LcP8mDq+qlSW6vqrO7+7aqOjvJHcv4Q0nOW1v/3CS37mD/AACnhG0f4eru53f3ud19flYnw7+yu78kyTVJLl2GXZrk5cvla5JcUlX3q6rHJLkgyfXbnjkAwCliJ0e4juaFSa6uquckeWeSZydJd99QVVcneXOSu5I8t7s/NLB/AIB9ZVeCq7tfleRVy+U/T/LUo4y7IskVu7FPAIBThXeaBwAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGCY4AIAGCa4AACGCS4AgGGCCwBgmOACABgmuAAAhgkuAIBh2w6uqjqvqn6rqm6sqhuq6nnL8odW1bVV9dbl80PW1nl+Vd1cVTdV1dN24w4AAOx3OznCdVeSb+7uj0/ypCTPrarHJbk8yXXdfUGS65brWW67JMnjk1yU5EVVdcZOJg8AcCrYdnB1923d/Ybl8vuS3JjknCQXJ7lyGXZlkmctly9OclV339ndb09yc5Inbnf/AACnil05h6uqzk/yiUlel+QR3X1bsoqyJA9fhp2T5Ja11Q4tyzbb3mVVdbCqDh4+fHg3pggAsGd2HFxV9aAkv5DkG7r7vccausmy3mxgd7+4uy/s7gsPHDiw0ykCAOypHQVXVd0nq9h6WXf/4rL49qo6e7n97CR3LMsPJTlvbfVzk9y6k/0DAJwKdvJbipXkJ5Pc2N3fv3bTNUkuXS5fmuTla8svqar7VdVjklyQ5Prt7h8A4FRx5g7WfXKSL03yR1X1pmXZC5K8MMnVVfWcJO9M8uwk6e4bqurqJG/O6jccn9vdH9rB/gEATgnbDq7ufk02Py8rSZ56lHWuSHLFdvcJAHAq8k7zAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMMEFwDAMMEFADBMcAEADBNcAADDBBcAwDDBBQAwTHABAAwTXAAAwwQXAMAwwQUAMExwAQAME1wAAMME15rzL3/FXk8BANhl++H7u+ACABgmuAAAhp15sndYVRcl+aEkZyR5SXe/8GTP4Vg2HnZ8xwufvkczAQDuLU5qcFXVGUl+NMlnJzmU5Per6prufvPJnMeJWA+wI/F1ZJkYA4D9Zz9+nz7ZR7iemOTm7n5bklTVVUkuTrJvg2vdxqNfu3US3saQ28m4d7zw6Uc9SrfZescav77OsZZtFqUnsu7GL4j1McfaxvH2ebQvuKPtc7MvzK180R5t3Y23beXx2Or+j7fdY83tWPPdyr6PtY3jrXO02050/yc67nj3eTe2sZ1xx9vniayzm/dht9e9N27jyHaSY/9/c7Txx1p2vHV2Oo+tbnc7409kvpuN2+z/thMZt3HsfjhZ/ojq7pO3s6p/nuSi7v6K5fqXJvnk7v66DeMuS3LZcvWxSW4antrDkrxreB+cOM/L/uM52X88J/uT52X/OVnPyaO7+8DGhSf7CFdtsuwexdfdL07y4vnprFTVwe6+8GTtj63xvOw/npP9x3OyP3le9p+9fk5O9m8pHkpy3tr1c5PcepLnAABwUp3s4Pr9JBdU1WOq6r5JLklyzUmeAwDASXVSX1Ls7ruq6uuS/EZWbwvxU919w8mcw1GctJcvOSGel/3Hc7L/eE72J8/L/rOnz8lJPWkeAOB05J3mAQCGCS4AgGGnfXBV1UVVdVNV3VxVl+/1fE53VfVTVXVHVf3xXs+Fv1NV51XVb1XVjVV1Q1U9b6/ndLqrqvtX1fVV9QfLc/Kdez0nVqrqjKp6Y1X9yl7PhZWqekdV/VFVvamqDu7JHE7nc7iWPzX0v7P2p4aSfNF+/lND93ZV9RlJ3p/kZ7v7E/Z6PqxU1dlJzu7uN1TVhyd5fZJn+VrZO1VVSc7q7vdX1X2SvCbJ87r79/Z4aqe9qvqmJBcmeXB3P2Ov58MquJJc2N179ma0p/sRrr/9U0Pd/cEkR/7UEHuku1+d5N17PQ/urrtv6+43LJffl+TGJOfs7axOb73y/uXqfZaP0/cn6H2iqs5N8vQkL9nrubC/nO7BdU6SW9auH4pvInBMVXV+kk9M8ro9nsppb3np6k1J7khybXd7TvbeDyb5liR/s8fz4O46yW9W1euXPx940p3uwbWlPzUErFTVg5L8QpJv6O737vV8Tnfd/aHufkJWf7XjiVXlZfg9VFXPSHJHd79+r+fCPTy5uz8pyecmee5y+spJdboHlz81BFu0nCf0C0le1t2/uNfz4e9093uSvCrJRXs7k9Pek5M8czlf6Kokn1lVL93bKZEk3X3r8vmOJL+U1SlFJ9XpHlz+1BBswXKC9k8mubG7v3+v50NSVQeq6iOXyw9I8llJ3rKnkzrNdffzu/vc7j4/q+8nr+zuL9njaZ32quqs5Zd9UlVnJfmcJCf9N+FP6+Dq7ruSHPlTQzcmuXqf/Kmh01ZV/VyS303y2Ko6VFXP2es5kWT1k/uXZvUT+5uWj8/b60md5s5O8ltV9YdZ/fB4bXd7GwK4p0ckeU1V/UGS65O8ort//WRP4rR+WwgAgJPhtD7CBQBwMgguAIBhggsAYJjgAgAYJrgAAIYJLgCAYYILAGDY/wc1GdKj0lwHKQAAAABJRU5ErkJggg==\n"
                        },
                        "metadata": {
                            "needs_background": "light"
                        }
                    }
                ]
            }
        }
    }
}
</script>
</head>
<body>

<script type="application/vnd.jupyter.widget-view+json">
{
    "version_major": 2,
    "version_minor": 0,
    "model_id": "62b8ac902f8f432b9a34cc3ed6845c41"
}
</script>

<script type="application/vnd.jupyter.widget-view+json">
{
    "version_major": 2,
    "version_minor": 0,
    "model_id": "af62e297abf445f79ee2f1855a949b44"
}
</script>

</body>
</html>
